In this notebook we develop code to find tangle exteriors of strongly invertible knots, which we use to create diagrams of branching sets of their surgeries. We apply that code to the census L-space knots. In a second step we use that to determine which of these branching sets are Khovanov thick as an obstruction for the knots having quasi-alternating slopes.

First we load the necassary functions.

In [40]:
import pandas
import time 
import snappy 
import math

exceptional_filllings = pandas.read_csv("exceptional_fillings.csv") 
# We load Dunfield's list from of all exceptional surgeries along the SnapPy CensusKnots.
DBChomologies = pandas.read_csv("DBChomologies_non_alternating.csv") 
# We load a list of all non-alternating links in the HTLinkExteriors together with their homologies 
#(if the homology is cyclic) and their crossing numbers.

def double_branched_cover(link):
    """
    Returns the double branched cover of the link.
    """
    L=link.copy()
    for i in range(L.num_cusps()):
        L.dehn_fill((2,0),i)
    for cov in L.covers(2):
        if (2.0, 0.0) not in cov.cusp_info('filling'):
            return cov

def better_is_isometric_to(X,Y,index=100,try_hard=False):
    """
    Returns True if X and Y are isometric.
    Returns False if SnapPy cannot verify it.
    The higher the index the harder SnapPy tries.
    """  
    w=False
    for i in (0,index):
        try:
            w=X.is_isometric_to(Y)
        except RuntimeError:
            pass
        except snappy.SnapPeaFatalError:
            pass
        if w==True:
            return w
        X.randomize()
        Y.randomize()
        i=i+1
    if try_hard==True:
        pos_triangX=better_find_positive_triangulation(X,tries=index)
        pos_triangY=better_find_positive_triangulation(Y,tries=index)
        if pos_triangX is not None:
            if pos_triangY is not None:
                return better_is_isometric_to(pos_triangX,pos_triangY)
    return False



def is_exceptional(knot,slope):
    """
    Checks if the slope is exceptional.
    """
    exceptional_slopes_strings=exceptional_filllings.loc[(exceptional_filllings['cusped'] == knot)]['slope'].tolist()
    exceptional_slopes=[]
    for string in exceptional_slopes_strings:
        string_without_brackets=string[1:-1]
        exceptional_slopes.append(tuple(map(int, string_without_brackets.split(', '))))
    if slope in exceptional_slopes:
        return True
    else:
        return False

def possible_DBC(homologies,max_crossings=15):
    """
    Takes a list of orders of homologies and returns a list consisting of all DBC of links in the HT link table with that homologies together with the link names.
    """
    DBCList=[]
    LINKS=[]
    for order in homologies:
        LINKS=LINKS+DBChomologies.loc[(DBChomologies['homology']==order) & (DBChomologies['crossings']<=max_crossings)]['knot'].tolist()
    for link in LINKS:
        L=snappy.Manifold(link)
        D=double_branched_cover(L)
        DBCList.append([D,link])
    return DBCList

### The following two functions are written by Dunfield and search for positive triangulations.

def all_positive(manifold):
    return manifold.solution_type() == 'all tetrahedra positively oriented'

def find_positive_triangulation(manifold, tries=100):
    M = manifold.copy()
    for i in range(tries):
        if all_positive(M):
            return M
        M.randomize()
    for d in M.dual_curves():
        X = M.drill(d)
        X = X.filled_triangulation()
        X.dehn_fill((1,0))
        for i in range(tries):
            if all_positive(X):
                return X
            X.randomize()

    # In the closed case, here is another trick.
    if all(not c for c in M.cusp_info('is_complete')):
        for i in range(tries):
            # Drills out a random edge
            X = M.__class__(M.filled_triangulation())
            if all_positive(X):
                return X
            M.randomize()
            
def better_find_positive_triangulation(M,tries=1):
    '''
    Search for a positive triangulation, but ignores errors.
    '''
    RandomizeCount=0
    while RandomizeCount<tries:
        try:
            X=find_positive_triangulation(M)
            return X
        except snappy.SnapPeaFatalError:
            M.randomize()
            RandomizeCount=RandomizeCount+1
    return None

def find_br_set(knot,slope,try_hard=False,index=10,tries=1,max_cro=15):
    '''
    Tries to find a branching set of the slope.
    '''
    K=snappy.Manifold(knot)
    K.dehn_fill(slope)
    DBC=possible_DBC([K.homology().order()],max_crossings=max_cro) 
    for D in DBC:
        w=better_is_isometric_to(D[0],K,index)
        if w==True:
            return [[knot,slope,D[1]]]
    if try_hard:
        X=better_find_positive_triangulation(K,tries)
        if X is not None:
            for D in DBC:
                Y=better_find_positive_triangulation(D[0],tries)
                if Y is not None:
                    w=better_is_isometric_to(X,Y,index)
                    if w==True:
                        return [[knot,slope,D[1]]]
    return False


#### This is Dunfield's util.py from his exceptional census

####  for a snappy manifold M descibed as a single filling of a cusp (so do filled_triangulation() as needed) 
####  the command regina_name(M) gives what regina identifies M as

"""

This file provides functions for working with Regina (with a little
help from SnapPy) to:

1. Give a standard name ("identify") manifolds, especially Seifert and
   graph manifolds.

2. Find essential tori.

3. Try to compute the JSJ decomposition.

"""

import regina
import snappy
import re
import networkx as nx

def appears_hyperbolic(M):
    acceptable = ['all tetrahedra positively oriented',
                  'contains negatively oriented tetrahedra']
    return M.solution_type() in acceptable and M.volume() > 0

def children(packet):
    child = packet.firstChild()
    while child:
        yield child
        child = child.nextSibling()

def to_regina(data):
    if hasattr(data, '_to_string'):
        data = data._to_string()
    if isinstance(data, str):
        if data.find('(') > -1:
            data = closed_isosigs(data)[0]
        return regina.Triangulation3(data)
    assert isinstance(data, regina.Triangulation3)
    return data

def extract_vector(surface):
    """
    Extract the raw vector of the (almost) normal surface in Regina's
    NS_STANDARD coordinate system.
    """
    S = surface
    T = S.triangulation()
    n = T.countTetrahedra()
    ans = []
    for i in range(n):
        for j in range(4):
            ans.append(S.triangles(i, j))
        for j in range(3):
            ans.append(S.quads(i, j))
    A = regina.NormalSurface(T, regina.NS_STANDARD, ans)
    assert A.sameSurface(S)
    return ans

def haken_sum(S1, S2):
    T = S1.triangulation()
    assert S1.locallyCompatible(S2)
    v1, v2 = extract_vector(S1), extract_vector(S2)
    sum_vec = [x1 + x2 for x1, x2 in zip(v1, v2)]
    A = regina.NormalSurface(T, regina.NS_STANDARD, sum_vec)
    assert S1.locallyCompatible(A) and S2.locallyCompatible(A)
    assert S1.eulerChar() + S2.eulerChar() == A.eulerChar()
    return A


def census_lookup(regina_tri):
    """
    Should the input triangulation be in Regina's census, return the
    name of the manifold, dropping the triangulation number.
    """
    hits = regina.Census.lookup(regina_tri)
    hit = hits[0]
    if hit is not None:
        name = hit.name()
        match = re.search('(.*) : #\d+$', name)
        if match:
            return match.group(1)
        else:
            return match

def standard_lookup(regina_tri):
    match = regina.StandardTriangulation.isStandardTriangulation(regina_tri)
    if match:
        return match.manifold()

def closed_isosigs(snappy_manifold, trys=20, max_tets=50):
    """
    Generate a slew of 1-vertex triangulations of a closed manifold
    using SnapPy.
    
    >>> M = snappy.Manifold('m004(1,2)')
    >>> len(closed_isosigs(M, trys=5)) > 0
    True
    """
    M = snappy.Manifold(snappy_manifold)
    assert M.cusp_info('complete?') == [False]
    surgery_descriptions = [M.copy()]

    try:
        for curve in M.dual_curves():
            N = M.drill(curve)
            N.dehn_fill((1,0), 1)
            surgery_descriptions.append(N.filled_triangulation([0]))
    except snappy.SnapPeaFatalError:
        pass

    if len(surgery_descriptions) == 1:
        # Try again, but unfill the cusp first to try to find more
        # dual curves.
        try:
            filling = M.cusp_info(0).filling
            N = M.copy()
            N.dehn_fill((0, 0), 0)
            N.randomize()
            for curve in N.dual_curves():
                D = N.drill(curve)
                D.dehn_fill([filling, (1,0)])
                surgery_descriptions.append(D.filled_triangulation([0]))
        except snappy.SnapPeaFatalError:
            pass

    ans = set()
    for N in surgery_descriptions:
        for i in range(trys):
            T = N.filled_triangulation()
            if T._num_fake_cusps() == 1:
                n = T.num_tetrahedra()
                if n <= max_tets:
                    ans.add((n, T.triangulation_isosig(decorated=False)))
            N.randomize()

    return [iso for n, iso in sorted(ans)]

def best_match(matches):
    """
    Prioritize the most concise description that Regina provides to
    try to avoid things like the Seifert fibered space of a node being
    a solid torus or having several nodes that can be condensed into a
    single Seifert fibered piece.
    """
    
    def score(m):
        if isinstance(m, regina.SFSpace):
            s = 0
        elif isinstance(m, regina.GraphLoop):
            s = 1
        elif isinstance(m, regina.GraphPair):
            s = 2
        elif isinstance(m, regina.GraphTriple):
            s = 3
        elif m is None:
            s = 10000
        else:
            s = 4
        return (s, str(m))
    return min(matches, key=score)

def identify_with_torus_boundary(regina_tri):
    """
    Use the combined power of Regina and SnapPy to try to give a name
    to the input manifold.
    """
    
    kind, name = None, None
    
    P = regina_tri.clone()
    P.finiteToIdeal()
    P.intelligentSimplify()
    M = snappy.Manifold(P.isoSig())
    M.simplify()
    if appears_hyperbolic(M):
        for i in range(100):
            if M.solution_type() == 'all tetrahedra positively oriented':
                break
            M.randomize()
        
        if not M.verify_hyperbolicity(bits_prec=100):
            raise RuntimeError('Cannot prove hyperbolicity for ' +
                               M.triangulation_isosig())
        kind = 'hyperbolic'
        ids = M.identify()
        if ids:
            name = ids[0].name()
    else:
        match = standard_lookup(regina_tri)
        if match is None:
            Q = P.clone()
            Q.idealToFinite()
            Q.intelligentSimplify()
            match = standard_lookup(Q)
        if match is not None:
            kind = match.__class__.__name__
            name = str(match)
        else:
            name = P.isoSig()
    return kind, name
            
def is_toroidal(regina_tri):
    """
    Checks for essential tori and returns the pieces of the
    associated partial JSJ decomposition.
    
    >>> T = to_regina('hLALAkbccfefgglpkusufk')  # m004(4,1)
    >>> is_toroidal(T)[0]
    True
    >>> T = to_regina('hvLAQkcdfegfggjwajpmpw')  # m004(0,1)
    >>> is_toroidal(T)[0]
    True
    >>> T = to_regina('nLLLLMLPQkcdgfihjlmmlkmlhshnrvaqtpsfnf')  # 5_2(10,1)
    >>> T.isHaken()
    True
    >>> is_toroidal(T)[0]
    False

    Note: currently checks all fundamental normal tori; possibly
    the theory lets one just check *vertex* normal tori.
    """
    T = regina_tri
    assert T.isZeroEfficient()
    surfaces = regina.NNormalSurfaceList.enumerate(T,
                          regina.NS_QUAD, regina.NS_FUNDAMENTAL)
    for i in range(surfaces.size()):
        S = surfaces.surface(i)
        if S.eulerChar() == 0:
            if not S.isOrientable():
                S = S.doubleSurface()
            assert S.isOrientable()
            X = S.cutAlong()
            X.intelligentSimplify()
            X.splitIntoComponents()
            pieces = list(children(X))
            if all(not C.hasCompressingDisc() for C in pieces):
                ids = [identify_with_torus_boundary(C) for C in pieces]
                return (True, sorted(ids))
                
    return (False, None)


def decompose_along_tori(regina_tri):
    """
    First, finds all essential normal tori in the manifold associated
    with fundamental normal surfaces.  Then takes a maximal disjoint
    collection of these tori, namely the one with the fewest tori
    involved, and cuts the manifold open along it.  It tries to
    identify the pieces, removing any (torus x I) components. 

    Returns: (has essential torus, list of pieces)

    Note: This may fail to be the true JSJ decomposition because there
    could be (torus x I)'s in the list of pieces and it might well be
    possible to amalgamate some of the pieces into a single SFS.
    """
    
    T = regina_tri
    assert T.isZeroEfficient()
    essential_tori = []
    surfaces = regina.NNormalSurfaceList.enumerate(T,
                          regina.NS_QUAD, regina.NS_FUNDAMENTAL)
    for i in range(surfaces.size()):
        S = surfaces.surface(i)
        if S.eulerChar() == 0:
            if not S.isOrientable():
                S = S.doubleSurface()
            assert S.isOrientable()
            X = S.cutAlong()
            X.intelligentSimplify()
            X.splitIntoComponents()
            pieces = list(children(X))
            if all(not C.hasCompressingDisc() for C in pieces):
                essential_tori.append(S)

    if len(essential_tori) == 0:
        return False, None
    
    D = nx.Graph()
    for a, A in enumerate(essential_tori):
        for b, B in enumerate(essential_tori):
            if a < b:
                if A.disjoint(B):
                    D.add_edge(a, b)

    cliques = list(nx.find_cliques(D))
    if len(cliques) == 0:
        clique = [0]
    else:
        clique = min(cliques, key=len)
    clique = [essential_tori[c] for c in clique]
    A = clique[0]
    for B in clique[1:]:
        A = haken_sum(A, B)

    X = A.cutAlong()
    X.intelligentSimplify()
    X.splitIntoComponents()
    ids = [identify_with_torus_boundary(C) for C in list(children(X))]
    # Remove products
    ids = [i for i in ids if i[1] not in ('SFS [A: (1,1)]', 'A x S1')]
    return (True, sorted(ids))

def regina_name(closed_snappy_manifold, trys=100):
    """
    >>> regina_name('m004(1,0)')
    'S3'
    >>> regina_name('s006(-2, 1)')
    'SFS [A: (5,1)] / [ 0,-1 | -1,0 ]'
    >>> regina_name('m010(-1, 1)')
    'L(3,1) # RP3'
    >>> regina_name('m022(-1,1)')
    'SFS [S2: (3,2) (3,2) (4,-3)]'
    >>> regina_name('v0004(0, 1)')
    'SFS [S2: (2,1) (4,1) (15,-13)]'
    >>> regina_name('m305(1, 0)')
    'L(3,1) # RP3'
    """
    M = snappy.Manifold(closed_snappy_manifold)
    isosigs = closed_isosigs(M, trys=trys, max_tets=25)
    if len(isosigs) == 0:
        return
    T = to_regina(isosigs[0])
    if T.isIrreducible():
        if T.countTetrahedra() <= 11:
            for i in range(3):
                T.simplifyExhaustive(i)
                name = census_lookup(T)
                if name is not None:
                    return name
            
        matches = [standard_lookup(to_regina(iso)) for iso in isosigs]
        match = best_match(matches)
        if match is not None:
            return str(match)
    else:
        T.connectedSumDecomposition()
        pieces = [regina_name(P) for P in children(T)]
        if None not in pieces:
            return ' # '.join(sorted(pieces))
        
def fill_triangulation(M):
    '''
    Fills all cusps but one.
    '''
    if M.num_cusps()==1:
        return M
    M=M.filled_triangulation([0])
    M=fill_triangulation(M)
    return M


def smoothings(link):
    '''
    Given a link L with a diagram D, this function returns a list of SnaPy link diagrams: For any crossing c in D
    we get the orientable smoothing of D together with the original crossing in the PD code marked.
    '''
    L=snappy.Manifold(link)
    PD=L.link().PD_code()
    SMOOTHINGS=[]
    for cros in PD:
        (a,b,c,d)=cros
        PDcopy=PD.copy()
        PDcopy.remove(cros) #Remove the crossing
        PD_new=[]
        if b<d:
            dic = {d:a, c:b,} #Replace d by a and c by b.
            for x in PDcopy:
                x=[dic.get(n, n) for n in x]
                [v,w,y,z]=x
                PD_new.append((v,w,y,z))
        if b>d:
            dic = {b:a, d:c,} #Replace b by a and c by d.
            for x in PDcopy:
                x=[dic.get(n, n) for n in x]
                [v,w,y,z]=x
                PD_new.append((v,w,y,z))
        K=snappy.Link(PD_new).exterior()
        SMOOTHINGS.append([K,cros])  
    return SMOOTHINGS

def change_crossing(link,crossing):
    '''
    Given a SnapPy link with a given diagram D and a crossing marked in its PD code. Then it returns the 
    SnapPy diagram that we get by changing that crossing together with the new crossing marked.
    '''
    (a,b,c,d)=crossing
    L=snappy.Manifold(link)
    PD=L.link().PD_code()
    for cros in PD:
        if cros==crossing:
            PD.remove(cros)
            if b<d:
                PD.append((b,c,d,a)) #Change the crossing
                return [snappy.Link(PD),(b,c,d,a)]
            if d<b:
                PD.append((d,a,b,c)) #Change the crossing
                return [snappy.Link(PD),(d,a,b,c)]
    return False

def is_unknot(link,index=100):
    '''
    Returns True if the given link is the unknot and False if it is not clear or False.
    '''
    L=link.exterior()
    if L.num_cusps()!=1:
        return False
    RandomizeCount=0
    while L.fundamental_group().num_generators()!=1 and RandomizeCount<index:
        L.randomize
        RandomizeCount=RandomizeCount+1
    if L.fundamental_group().num_generators()!=1:
        return False
    if L.fundamental_group().num_relators()!=0:
        return False
    return True

def get_tangle_exterior_of_crossing(link,crossing,index=100):
    '''
    Given a SnapPy link diagram D and a crossing marked in its PD code. Then it returns the 
    tangle complement of that crossing as a SnapPy manifold.
    '''
    (a,b,c,d)=crossing
    B=link.copy()
    PD=B.PD_code()
    X=[]
    for cros in PD:
        X=X+[x for x in cros]
    m=max(X) #The maximal value of the PD code.
    PD_new=PD.copy()
    PD_new.remove(crossing)
    #We introduce a circle C around that crossing.
    PD_new=PD_new+[(a,m+5,m+1,m+8),(m+5,b,m+6,m+2),(m+6,c,m+7,m+3),(m+4,m+7,d,m+8),(m+1,m+2,m+3,m+4)]
    E=snappy.Link(PD_new).exterior() #The exterior of that new link.
    for i in range(E.num_cusps()-1):
        E.dehn_fill((2,0),i)
    #We take the DBC along B. The Conway sphere around the crossing will lift to an torus T. 
    #The torus T will bound on one side the tangle exterior and on the other side a pair of pants times S1,
    #where the other two tori boundary components are given by the lifts of C.
    for C in E.covers(2): 
        if C.cusp_info('filling').count((0,0))==2:
            if (2,0) not in C.cusp_info('filling'):
                #Next we cut C along T to get the tangle exterior. 
                #The code looks a bit akward since the SnapPy function .split() is not deterministic.
                RandomizeCount=0
                while RandomizeCount<index:
                    try:
                        for S in C.splitting_surfaces():
                            for T in C.split(S):
                                if T.num_cusps()==1:
                                    return T
                        C.randomize()
                        RandomizeCount=RandomizeCount+1
                    except ValueError:
                        C.randomize()
                        RandomizeCount=RandomizeCount+1
    return False

def tangle_exterior(knot,half,int1,int2,index=100):
    '''
    Takes a strongly invertible knot K together with a half-integer  
    branching set and the nearby integer branching sets and builds the
    diagram D of the unknot together with a crossing c.
    Performing tangle replacement of that crossing gives the branching sets of the surgeries of K.
    The fourth and fifth return values are True, if D is verified to represent an unknot and if
    the exterior of the crossing c lifts to the exterior of K.
    '''
    B1=snappy.Manifold(int1)
    B2=snappy.Manifold(int2)
    for S in smoothings(half):
        if better_is_isometric_to(S[0],B1,index)==True or better_is_isometric_to(S[0],B2,index)==True:
            [U,cros]=change_crossing(half,S[1])
            w=False
            Count=0
            while w!=True and Count<index:
                T=get_tangle_exterior_of_crossing(U,cros)
                w=better_is_isometric_to(T,snappy.Manifold(knot),index)
            return [knot,U,cros,is_unknot(U,index),w]
        
def find_tangle_exterior(knot,slopes=[(0,1),(1,2),(1,1)]):
    '''
    Takes a strongly invertible census knot and tries to find its tangle exterior.
    Returns [name_of_knot,diagram of the unkot,crossing in its PD code whose exterior is the tangle exterior,True if the diagram is confirmed to be the unknot, True if the tangle exterior is confirmed to be correct]
    Slopes are the ones we test for finding branching sets, that can be changed to any triple of slopes.
    '''
    BR_SETS=[]
    for slope in slopes:
        if is_exceptional(knot,slope)==True:
            K=snappy.Manifold(knot)
            K.dehn_fill(slope)
            K_reg=regina_name(K)
            DBC=possible_DBC([K.homology().order()],max_crossings=15)
            for D in DBC:
                DF=fill_triangulation(D[0])
                if regina_name(DF)==K_reg:
                    BR_SETS.append([knot,slope,D[1]])
                    break
        else:
            BR=find_br_set(knot,slope,index=3,max_cro=13)
            if BR==False:
                BR=find_br_set(knot,slope,index=3,max_cro=15)
            if BR==False:
                BR=find_br_set(knot,slope,index=10,max_cro=15)
            if BR!=False:
                BR_SETS.append(BR[0])
    if len(BR_SETS)!=3:
        return [knot,False]
    return tangle_exterior(knot,BR_SETS[1][2],BR_SETS[0][2],BR_SETS[2][2])
        
def replace_strands(PD,sold,snew):
    '''Replaces the strand sold by the strand snew in the PD code.'''
    PD_new=PD.copy()
    PD_new=[tuple([snew if x==sold else x for x in c]) for c in PD]
    return PD_new 

def tangle_filling(PD,cros,slope):
    '''A tangle is presented by a PD code of a knot diagram together with a crossing that needs
    to be removed to get the tangle.
    This function returns a diagram of the (p,q)-tangle filling.
    Here the 0-filling corresponds to filling cros=(S,E,N,W) with S=W and N=E
    and the 1/0 filling is N=W and S=E.'''
    (p,q)=slope
    PD_fill=PD.copy()
    #ends of tangle
    (S,E,N,W)=cros
    PD_fill.remove(cros)
    #Return if tangle is trivial
    if (p,q)==(1,0):
        PD_fill=replace_strands(PD_fill,S,E)
        return replace_strands(PD_fill,N,W)
    if (p,q)==(-1,0):
        PD_fill=replace_strands(PD_fill,S,E)
        return replace_strands(PD_fill,N,W)
    if (p,q)==(0,1):
        PD_fill=replace_strands(PD_fill,S,W)
        return replace_strands(PD_fill,N,E)
    if (p,q)==(0,-1):
        PD_fill=replace_strands(PD_fill,S,W)
        return replace_strands(PD_fill,N,E)
    #Take inverse if |p/q|<1
    if abs(p/q)<1:
        PD_fill.append((E,N,W,S))
        return tangle_filling(PD_fill,(E,N,W,S),(q,p))
    m=max([max(c) for c in PD])+1 #maximal value in PD
    if p/q>=1:
        PD_fill.append((m,m+1,N,W))
        PD_fill.append((E,m+1,m,S))
        return tangle_filling(PD_fill,(m,m+1,N,W),(p-q,q))
    if p/q<=1:
        PD_fill.append((m,m+1,N,W))
        PD_fill.append((S,E,m+1,m))
        return tangle_filling(PD_fill,(m,m+1,N,W),(p+q,q))       
    

    
def find_braid_word_of_br_set_of_slope(knot,PD,cros,slope):
    '''
    Creates braid word of the branching set of the filling.
    '''
    K=snappy.Manifold(knot)
    K.dehn_fill(slope)
    K_hom=K.homology()
    
    B=snappy.Link(tangle_filling(PD,cros,(1,0)))
    C=double_branched_cover(B.exterior())
    if C.homology()==K_hom:
        B.simplify('global')
        return B.braid_word()

    B=snappy.Link(tangle_filling(PD,cros,(0,1)))
    C=double_branched_cover(B.exterior())
    if C.homology()==K_hom:
        B.simplify('global')
        return B.braid_word()  

    B=snappy.Link(tangle_filling(PD,cros,(1,2)))
    C=double_branched_cover(B.exterior())
    if C.homology()==K_hom:
        B.simplify('global')
        return B.braid_word()

    for p in range(-20,-1):
        B=snappy.Link(tangle_filling(PD,cros,(p,p-1)))
        C=double_branched_cover(B.exterior())
        if C.homology()==K_hom:
            B.simplify('global')
            return B.braid_word()

    for p in range(2,21):
        B=snappy.Link(tangle_filling(PD,cros,(p+2,-p-1)))
        C=double_branched_cover(B.exterior())
        if C.homology()==K_hom:
            B.simplify('global')
            return B.braid_word()

    return False 

def create_braid_words_of_branching_sets(knot,PD,cros,slopes=[(-5,1),(-4,1),(-3,1),(-2,1),(-1,1),(0,1),(1,1),(2,1),(3,1),(4,1),(5,1)]):
    '''
    Creates PD codes of small branching sets.
    '''
    braid_words=[]
    for s in slopes:
        word=find_braid_word_of_br_set_of_slope(knot,PD,cros,s)
        if word==False:
            return False
        braid_words.append(word)
    return [knot,braid_words]    
    
def braid_word_to_Mathematica(braid):
    '''
    Takes a SnapPy braid and returns the same braid word in the Mathematica form.
    '''
    string=str(braid)
    string=string.replace(']','}')
    string=string.replace('[','{')
    return string

def braid_index(braid):
    return max([abs(x) for x in braid])+1

def is_S3(manifold,index=100):
    '''Returns True if the Manifold is S3.'''
    for i in range(index):
        if manifold.fundamental_group().num_generators()==0:
            return True
        manifold.randomize()
        i=i+1
    return False

def find_S3_filling(knot):
    '''Returns the S3-filling.'''
    K=snappy.Manifold(knot)
    for s in K.short_slopes()[0]:
        K.dehn_fill(s)
        if is_S3(K):
            return s
    return False

<>:229: DeprecationWarning: invalid escape sequence '\d'
<>:229: DeprecationWarning: invalid escape sequence '\d'
/tmp/ipykernel_161529/3459475645.py:229: DeprecationWarning: invalid escape sequence '\d'
  match = re.search('(.*) : #\d+$', name)


We load the existing data.

In [3]:
Lspace_census=['m016', 'm071', 'm082', 'm103', 'm118', 'm144', 'm194', 'm198', 'm211', 'm223', 'm239', 'm240', 'm270', 'm276', 'm281', 's042', 's068', 's086', 's104', 's114', 's294', 's301', 's308', 's336', 's344', 's346', 's367', 's369', 's384', 's407', 's560', 's582', 's652', 's665', 's682', 's684', 's769', 's800', 's849', 'v0082', 'v0114', 'v0165', 'v0220', 'v0223', 'v0249', 'v0319', 'v0330', 'v0398', 'v0407', 'v0424', 'v0434', 'v0497', 'v0545', 'v0554', 'v0570', 'v0573', 'v0707', 'v0709', 'v0715', 'v0740', 'v0741', 'v0759', 'v0765', 'v0830', 'v0847', 'v0912', 'v0939', 'v0945', 'v0959', 'v1077', 'v1109', 'v1269', 'v1300', 'v1359', 'v1392', 'v1423', 'v1425', 'v1547', 'v1565', 'v1620', 'v1628', 'v1690', 'v1709', 'v1716', 'v1718', 'v1728', 'v1810', 'v1832', 'v1839', 'v1915', 'v1921', 'v1940', 'v1966', 'v1980', 'v1986', 'v2024', 'v2090', 'v2215', 'v2217', 'v2290', 'v2325', 'v2384', 'v2759', 'v2871', 'v2900', 'v2925', 'v2930', 'v3070', 'v3105', 'v3234', 'v3335', 'v3354', 'v3482', 't00110', 't00146', 't00324', 't00423', 't00434', 't00550', 't00621', 't00729', 't00787', 't00826', 't00855', 't00873', 't00932', 't01033', 't01037', 't01125', 't01216', 't01268', 't01292', 't01318', 't01368', 't01409', 't01422', 't01424', 't01440', 't01598', 't01636', 't01646', 't01690', 't01757', 't01815', 't01834', 't01850', 't01863', 't01949', 't01966', 't02099', 't02104', 't02238', 't02276', 't02378', 't02398', 't02404', 't02470', 't02537', 't02567', 't02639', 't03106', 't03566', 't03607', 't03709', 't03710', 't03713', 't03781', 't03843', 't03864', 't03956', 't03979', 't04003', 't04019', 't04102', 't04180', 't04228', 't04244', 't04382', 't04449', 't04557', 't04721', 't04756', 't04927', 't05118', 't05239', 't05390', 't05425', 't05426', 't05538', 't05564', 't05578', 't05658', 't05663', 't05674', 't05695', 't06001', 't06246', 't06440', 't06463', 't06525', 't06570', 't06573', 't06605', 't06637', 't06715', 't06957', 't07070', 't07104', 't07348', 't07355', 't07412', 't07670', 't08111', 't08114', 't08184', 't08201', 't08267', 't08273', 't08403', 't08532', 't08576', 't08936', 't09016', 't09126', 't09267', 't09284', 't09313', 't09450', 't09455', 't09500', 't09580', 't09633', 't09690', 't09704', 't09847', 't09852', 't09882', 't09912', 't09954', 't10177', 't10188', 't10215', 't10224', 't10230', 't10262', 't10292', 't10462', 't10496', 't10643', 't10681', 't10832', 't10985', 't11198', 't11376', 't11548', 't11556', 't11852', 't11887', 't11909', 't12288', 't12533', 't12681', 't12753', 'o9_00133', 'o9_00168', 'o9_00644', 'o9_00797', 'o9_00815', 'o9_01079', 'o9_01175', 'o9_01436', 'o9_01496', 'o9_01584', 'o9_01621', 'o9_01680', 'o9_01765', 'o9_01936', 'o9_01953', 'o9_01955', 'o9_02255', 'o9_02340', 'o9_02350', 'o9_02383', 'o9_02386', 'o9_02655', 'o9_02696', 'o9_02706', 'o9_02735', 'o9_02772', 'o9_02786', 'o9_02794', 'o9_02909', 'o9_03032', 'o9_03108', 'o9_03118', 'o9_03133', 'o9_03149', 'o9_03162', 'o9_03188', 'o9_03288', 'o9_03313', 'o9_03412', 'o9_03526', 'o9_03586', 'o9_03622', 'o9_03802', 'o9_03833', 'o9_03932', 'o9_04054', 'o9_04060', 'o9_04106', 'o9_04205', 'o9_04245', 'o9_04269', 'o9_04313', 'o9_04431', 'o9_04435', 'o9_04438', 'o9_04938', 'o9_05021', 'o9_05177', 'o9_05229', 'o9_05287', 'o9_05357', 'o9_05426', 'o9_05483', 'o9_05562', 'o9_05618', 'o9_05860', 'o9_05970', 'o9_06060', 'o9_06128', 'o9_06154', 'o9_06248', 'o9_06301', 'o9_06956', 'o9_07044', 'o9_07152', 'o9_07167', 'o9_07195', 'o9_07401', 'o9_07790', 'o9_07893', 'o9_07943', 'o9_07945', 'o9_08006', 'o9_08042', 'o9_08224', 'o9_08302', 'o9_08402', 'o9_08477', 'o9_08497', 'o9_08647', 'o9_08765', 'o9_08771', 'o9_08776', 'o9_08828', 'o9_08831', 'o9_08852', 'o9_08875', 'o9_09052', 'o9_09213', 'o9_09271', 'o9_09372', 'o9_09465', 'o9_09731', 'o9_09808', 'o9_10020', 'o9_10192', 'o9_10213', 'o9_10696', 'o9_11002', 'o9_11100', 'o9_11248', 'o9_11467', 'o9_11537', 'o9_11541', 'o9_11556', 'o9_11560', 'o9_11570', 'o9_11658', 'o9_11685', 'o9_11795', 'o9_11845', 'o9_11999', 'o9_12079', 'o9_12144', 'o9_12230', 'o9_12253', 'o9_12412', 'o9_12459', 'o9_12477', 'o9_12519', 'o9_12693', 'o9_12736', 'o9_12757', 'o9_12873', 'o9_12892', 'o9_12919', 'o9_12971', 'o9_13052', 'o9_13054', 'o9_13056', 'o9_13125', 'o9_13182', 'o9_13188', 'o9_13400', 'o9_13403', 'o9_13433', 'o9_13508', 'o9_13537', 'o9_13604', 'o9_13639', 'o9_13649', 'o9_13666', 'o9_13720', 'o9_13952', 'o9_14018', 'o9_14079', 'o9_14108', 'o9_14136', 'o9_14359', 'o9_14364', 'o9_14376', 'o9_14495', 'o9_14599', 'o9_14716', 'o9_14831', 'o9_14974', 'o9_15506', 'o9_15633', 'o9_15808', 'o9_15997', 'o9_16065', 'o9_16141', 'o9_16157', 'o9_16181', 'o9_16319', 'o9_16356', 'o9_16431', 'o9_16514', 'o9_16527', 'o9_16642', 'o9_16685', 'o9_16748', 'o9_16920', 'o9_17382', 'o9_17450', 'o9_17646', 'o9_18007', 'o9_18209', 'o9_18341', 'o9_18633', 'o9_18646', 'o9_18813', 'o9_19130', 'o9_19247', 'o9_19364', 'o9_19396', 'o9_19645', 'o9_19724', 'o9_20029', 'o9_20219', 'o9_20305', 'o9_20364', 'o9_20472', 'o9_21195', 'o9_21496', 'o9_21513', 'o9_21620', 'o9_21893', 'o9_21918', 'o9_22129', 'o9_22252', 'o9_22477', 'o9_22607', 'o9_22663', 'o9_22698', 'o9_22925', 'o9_23023', 'o9_23032', 'o9_23179', 'o9_23263', 'o9_23461', 'o9_23660', 'o9_23723', 'o9_23955', 'o9_23961', 'o9_23971', 'o9_23977', 'o9_24069', 'o9_24126', 'o9_24149', 'o9_24183', 'o9_24290', 'o9_24401', 'o9_24407', 'o9_24534', 'o9_24592', 'o9_24779', 'o9_24886', 'o9_24889', 'o9_24946', 'o9_25110', 'o9_25199', 'o9_25341', 'o9_25444', 'o9_25595', 'o9_25709', 'o9_25832', 'o9_26141', 'o9_26471', 'o9_26570', 'o9_26604', 'o9_26767', 'o9_26791', 'o9_27107', 'o9_27155', 'o9_27261', 'o9_27371', 'o9_27392', 'o9_27429', 'o9_27480', 'o9_27737', 'o9_27767', 'o9_28113', 'o9_28153', 'o9_28284', 'o9_28529', 'o9_28592', 'o9_28746', 'o9_28751', 'o9_28810', 'o9_29048', 'o9_29246', 'o9_29436', 'o9_29529', 'o9_29551', 'o9_29648', 'o9_29751', 'o9_29766', 'o9_30142', 'o9_30150', 'o9_30375', 'o9_30634', 'o9_30650', 'o9_30721', 'o9_30790', 'o9_31165', 'o9_31267', 'o9_31321', 'o9_31440', 'o9_31481', 'o9_32044', 'o9_32065', 'o9_32132', 'o9_32150', 'o9_32257', 'o9_32314', 'o9_32471', 'o9_32588', 'o9_32964', 'o9_33189', 'o9_33284', 'o9_33380', 'o9_33430', 'o9_33486', 'o9_33526', 'o9_33585', 'o9_33801', 'o9_33944', 'o9_33959', 'o9_34000', 'o9_34403', 'o9_34409', 'o9_34689', 'o9_35320', 'o9_35549', 'o9_35666', 'o9_35682', 'o9_35720', 'o9_35736', 'o9_35772', 'o9_35928', 'o9_36114', 'o9_36250', 'o9_36380', 'o9_36544', 'o9_36809', 'o9_36958', 'o9_37050', 'o9_37291', 'o9_37482', 'o9_37551', 'o9_37685', 'o9_37751', 'o9_37754', 'o9_37851', 'o9_37941', 'o9_38287', 'o9_38679', 'o9_38811', 'o9_38928', 'o9_38989', 'o9_39162', 'o9_39394', 'o9_39451', 'o9_39521', 'o9_39606', 'o9_39608', 'o9_39859', 'o9_39879', 'o9_39981', 'o9_40026', 'o9_40052', 'o9_40075', 'o9_40179', 'o9_40363', 'o9_40487', 'o9_40504', 'o9_40582', 'o9_41372', 'o9_42224', 'o9_42493', 'o9_42675', 'o9_42961', 'o9_43001', 'o9_43679', 'o9_43750', 'o9_43857', 'o9_43953', 'o9_44054']
len(Lspace_census)

632

In [17]:
asymmetric=['t12533', 't12681', 'o9_38928', 'o9_39162', 'o9_40363','o9_40487', 'o9_40504', 'o9_40582', 'o9_42675']

We start with the 6 knots for which we have not found any quasi-alternating surgery by brute force.

In [4]:
pos_no_QA_surgery=['v2871','t09847','t09882','o9_30634','o9_32044','o9_32964']

We load the tangle exteriors of the knots with alternating slopes. We first create tangle exteriors of the others.

In [5]:
PD_codes_of_tangle_exteriors=[['m016', [(5, 1, 6, 0), (1, 12, 2, 13), (7, 2, 8, 3), (3, 8, 4, 9), (13, 7, 14, 6), (9, 15, 10, 14), (15, 11, 0, 10), (4, 12, 5, 11)], (4, 12, 5, 11)], ['m071', [(5, 1, 6, 0), (1, 14, 2, 15), (7, 2, 8, 3), (3, 8, 4, 9), (13, 4, 14, 5), (9, 17, 10, 16), (17, 11, 18, 10), (11, 19, 12, 18), (19, 13, 0, 12), (6, 15, 7, 16)], (6, 15, 7, 16)], ['m103', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (15, 4, 16, 5), (5, 18, 6, 19), (7, 11, 8, 10), (11, 17, 12, 16), (17, 13, 18, 12), (19, 14, 0, 15), (6, 13, 7, 14)], (6, 13, 7, 14)], ['m118', [(3, 1, 4, 0), (1, 8, 2, 9), (7, 2, 8, 3), (5, 16, 6, 17), (11, 7, 12, 6), (9, 15, 10, 14), (15, 11, 16, 10), (17, 12, 0, 13), (4, 14, 5, 13)], (4, 14, 5, 13)], ['m240', [(5, 1, 6, 0), (9, 2, 10, 3), (3, 12, 4, 13), (19, 5, 0, 4), (13, 6, 14, 7), (7, 14, 8, 15), (15, 8, 16, 9), (17, 11, 18, 10), (11, 19, 12, 18), (16, 1, 17, 2)], (16, 1, 17, 2)], ['m270', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (13, 4, 14, 5), (5, 18, 6, 19), (17, 6, 18, 7), (7, 14, 8, 15), (15, 10, 16, 11), (11, 16, 12, 17), (12, 0, 13, 19)], (12, 0, 13, 19)], ['m276', [(3, 1, 4, 0), (1, 8, 2, 9), (7, 2, 8, 3), (5, 18, 6, 19), (13, 6, 14, 7), (9, 17, 10, 16), (15, 11, 16, 10), (19, 13, 0, 12), (17, 14, 18, 15), (4, 11, 5, 12)], (4, 11, 5, 12)], ['o9_00133', [(5, 1, 6, 0), (1, 18, 2, 19), (7, 2, 8, 3), (3, 8, 4, 9), (17, 4, 18, 5), (9, 21, 10, 20), (21, 11, 22, 10), (11, 27, 12, 26), (25, 13, 26, 12), (13, 25, 14, 24), (23, 15, 24, 14), (15, 23, 16, 22), (27, 17, 0, 16), (6, 19, 7, 20)], (6, 19, 7, 20)], ['o9_00168', [(5, 1, 6, 0), (1, 17, 2, 16), (7, 2, 8, 3), (3, 8, 4, 9), (17, 5, 18, 4), (9, 24, 10, 25), (23, 10, 24, 11), (11, 22, 12, 23), (21, 12, 22, 13), (13, 20, 14, 21), (25, 14, 26, 15), (27, 18, 0, 19), (19, 26, 20, 27), (6, 16, 7, 15)], (6, 16, 7, 15)], ['o9_00797', [(5, 1, 6, 0), (1, 16, 2, 17), (7, 2, 8, 3), (3, 8, 4, 9), (17, 7, 18, 6), (9, 25, 10, 24), (23, 11, 24, 10), (11, 23, 12, 22), (21, 13, 22, 12), (13, 19, 14, 18), (27, 15, 0, 14), (19, 26, 20, 27), (25, 20, 26, 21), (4, 16, 5, 15)], (4, 16, 5, 15)], ['o9_00815', [(5, 1, 6, 0), (1, 17, 2, 16), (7, 2, 8, 3), (3, 8, 4, 9), (15, 6, 16, 7), (9, 27, 10, 26), (19, 11, 20, 10), (11, 24, 12, 25), (23, 12, 24, 13), (13, 22, 14, 23), (21, 14, 22, 15), (27, 18, 0, 19), (25, 21, 26, 20), (4, 17, 5, 18)], (4, 17, 5, 18)], ['o9_01680', [(5, 1, 6, 0), (1, 7, 2, 6), (3, 17, 4, 16), (19, 5, 20, 4), (7, 19, 8, 18), (17, 9, 18, 8), (21, 10, 22, 11), (11, 22, 12, 23), (23, 12, 24, 13), (13, 26, 14, 27), (25, 14, 26, 15), (15, 24, 16, 25), (27, 20, 0, 21), (2, 9, 3, 10)], (2, 9, 3, 10)], ['o9_01765', [(5, 1, 6, 0), (1, 7, 2, 6), (3, 18, 4, 19), (15, 4, 16, 5), (7, 16, 8, 17), (17, 8, 18, 9), (19, 11, 20, 10), (11, 25, 12, 24), (23, 13, 24, 12), (13, 23, 14, 22), (25, 15, 26, 14), (27, 21, 0, 20), (21, 27, 22, 26), (2, 9, 3, 10)], (2, 9, 3, 10)], ['o9_02340', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (15, 4, 16, 5), (5, 22, 6, 23), (17, 7, 18, 6), (7, 19, 8, 18), (23, 8, 24, 9), (9, 26, 10, 27), (13, 11, 14, 10), (27, 14, 0, 15), (19, 25, 20, 24), (25, 21, 26, 20), (16, 21, 17, 22)], (16, 21, 17, 22)], ['o9_02386', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (13, 5, 14, 4), (5, 24, 6, 25), (19, 7, 20, 6), (7, 19, 8, 18), (23, 8, 24, 9), (9, 16, 10, 17), (15, 10, 16, 11), (27, 15, 0, 14), (25, 20, 26, 21), (21, 26, 22, 27), (22, 18, 23, 17)], (22, 18, 23, 17)], ['o9_02706', [(3, 1, 4, 0), (1, 8, 2, 9), (7, 2, 8, 3), (5, 26, 6, 27), (17, 6, 18, 7), (9, 25, 10, 24), (19, 11, 20, 10), (11, 23, 12, 22), (21, 13, 22, 12), (13, 21, 14, 20), (23, 15, 24, 14), (27, 17, 0, 16), (25, 18, 26, 19), (4, 15, 5, 16)], (4, 15, 5, 16)], ['o9_02735', [(3, 1, 4, 0), (1, 8, 2, 9), (7, 2, 8, 3), (5, 26, 6, 27), (15, 7, 16, 6), (9, 25, 10, 24), (19, 10, 20, 11), (11, 22, 12, 23), (21, 12, 22, 13), (13, 20, 14, 21), (25, 15, 26, 14), (27, 16, 0, 17), (23, 18, 24, 19), (4, 18, 5, 17)], (4, 18, 5, 17)], ['o9_03118', [(3, 1, 4, 0), (1, 8, 2, 9), (7, 2, 8, 3), (13, 5, 14, 4), (5, 26, 6, 27), (9, 23, 10, 22), (21, 11, 22, 10), (11, 19, 12, 18), (17, 13, 18, 12), (27, 15, 0, 14), (25, 16, 26, 17), (19, 24, 20, 25), (23, 20, 24, 21), (6, 16, 7, 15)], (6, 16, 7, 15)], ['o9_03149', [(3, 1, 4, 0), (1, 8, 2, 9), (7, 2, 8, 3), (17, 4, 18, 5), (5, 26, 6, 27), (9, 25, 10, 24), (19, 11, 20, 10), (11, 22, 12, 23), (21, 12, 22, 13), (13, 19, 14, 18), (25, 15, 26, 14), (27, 16, 0, 17), (23, 21, 24, 20), (6, 15, 7, 16)], (6, 15, 7, 16)], ['o9_03288', [(5, 1, 6, 0), (9, 2, 10, 3), (3, 19, 4, 18), (27, 5, 0, 4), (17, 7, 18, 6), (7, 17, 8, 16), (15, 9, 16, 8), (21, 10, 22, 11), (11, 22, 12, 23), (23, 12, 24, 13), (13, 24, 14, 25), (19, 26, 20, 27), (25, 20, 26, 21), (14, 2, 15, 1)], (14, 2, 15, 1)], ['o9_03412', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (15, 5, 16, 4), (17, 6, 18, 7), (7, 18, 8, 19), (19, 8, 20, 9), (9, 22, 10, 23), (27, 11, 0, 10), (11, 15, 12, 14), (23, 16, 24, 17), (25, 21, 26, 20), (21, 27, 22, 26), (24, 5, 25, 6)], (24, 5, 25, 6)], ['o9_03526', [(5, 1, 6, 0), (11, 2, 12, 3), (3, 16, 4, 17), (27, 5, 0, 4), (17, 6, 18, 7), (7, 18, 8, 19), (19, 8, 20, 9), (9, 20, 10, 21), (21, 10, 22, 11), (23, 13, 24, 12), (13, 25, 14, 24), (25, 15, 26, 14), (15, 27, 16, 26), (22, 1, 23, 2)], (22, 1, 23, 2)], ['o9_03932', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (19, 4, 20, 5), (5, 26, 6, 27), (25, 6, 26, 7), (7, 24, 8, 25), (23, 8, 24, 9), (9, 20, 10, 21), (15, 11, 16, 10), (11, 15, 12, 14), (21, 16, 22, 17), (17, 22, 18, 23), (18, 0, 19, 27)], (18, 0, 19, 27)], ['o9_04313', [(5, 1, 6, 0), (9, 2, 10, 3), (3, 19, 4, 18), (25, 5, 26, 4), (27, 7, 0, 6), (7, 27, 8, 26), (17, 9, 18, 8), (19, 10, 20, 11), (11, 24, 12, 25), (21, 12, 22, 13), (13, 22, 14, 23), (23, 14, 24, 15), (15, 20, 16, 21), (16, 2, 17, 1)], (16, 2, 17, 1)], ['o9_04431', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (19, 4, 20, 5), (5, 24, 6, 25), (7, 13, 8, 12), (27, 10, 0, 11), (11, 26, 12, 27), (13, 23, 14, 22), (21, 15, 22, 14), (15, 21, 16, 20), (23, 17, 24, 16), (25, 18, 26, 19), (6, 17, 7, 18)], (6, 17, 7, 18)], ['o9_04435', [(5, 1, 6, 0), (1, 12, 2, 13), (11, 2, 12, 3), (3, 10, 4, 11), (9, 4, 10, 5), (7, 26, 8, 27), (19, 8, 20, 9), (13, 25, 14, 24), (23, 15, 24, 14), (15, 23, 16, 22), (21, 17, 22, 16), (27, 19, 0, 18), (25, 20, 26, 21), (6, 17, 7, 18)], (6, 17, 7, 18)], ['o9_07943', [(7, 1, 8, 0), (1, 9, 2, 8), (9, 3, 10, 2), (3, 21, 4, 20), (11, 4, 12, 5), (5, 12, 6, 13), (21, 7, 22, 6), (13, 22, 14, 23), (23, 14, 24, 15), (15, 26, 16, 27), (25, 16, 26, 17), (17, 24, 18, 25), (27, 18, 0, 19), (10, 20, 11, 19)], (10, 20, 11, 19)], ['o9_08042', [(5, 1, 6, 0), (1, 13, 2, 12), (11, 3, 12, 2), (3, 11, 4, 10), (13, 5, 14, 4), (7, 24, 8, 25), (21, 8, 22, 9), (9, 15, 10, 14), (15, 22, 16, 23), (23, 16, 24, 17), (25, 19, 26, 18), (19, 27, 20, 26), (27, 21, 0, 20), (6, 17, 7, 18)], (6, 17, 7, 18)], ['o9_08776', [(3, 1, 4, 0), (1, 8, 2, 9), (7, 2, 8, 3), (17, 4, 18, 5), (5, 20, 6, 21), (9, 19, 10, 18), (19, 11, 20, 10), (21, 12, 22, 13), (13, 27, 14, 26), (23, 15, 24, 14), (15, 25, 16, 24), (25, 17, 26, 16), (27, 22, 0, 23), (6, 11, 7, 12)], (6, 11, 7, 12)], ['o9_08852', [(3, 1, 4, 0), (1, 8, 2, 9), (7, 2, 8, 3), (19, 5, 20, 4), (5, 17, 6, 16), (9, 18, 10, 19), (17, 10, 18, 11), (21, 13, 22, 12), (13, 27, 14, 26), (23, 14, 24, 15), (15, 24, 16, 25), (27, 21, 0, 20), (25, 22, 26, 23), (6, 11, 7, 12)], (6, 11, 7, 12)], ['o9_10696', [(7, 1, 8, 0), (1, 9, 2, 8), (9, 3, 10, 2), (3, 18, 4, 19), (13, 4, 14, 5), (27, 7, 0, 6), (19, 11, 20, 10), (11, 21, 12, 20), (21, 13, 22, 12), (23, 14, 24, 15), (15, 24, 16, 25), (25, 16, 26, 17), (17, 26, 18, 27), (22, 5, 23, 6)], (22, 5, 23, 6)], ['o9_11467', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (13, 4, 14, 5), (19, 6, 20, 7), (7, 22, 8, 23), (11, 9, 12, 8), (27, 12, 0, 13), (21, 14, 22, 15), (15, 20, 16, 21), (17, 25, 18, 24), (25, 19, 26, 18), (23, 27, 24, 26), (16, 6, 17, 5)], (16, 6, 17, 5)], ['o9_11560', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (11, 5, 12, 4), (5, 22, 6, 23), (19, 6, 20, 7), (13, 8, 14, 9), (27, 13, 0, 12), (21, 15, 22, 14), (15, 21, 16, 20), (17, 24, 18, 25), (25, 18, 26, 19), (23, 26, 24, 27), (16, 8, 17, 7)], (16, 8, 17, 7)], ['o9_12144', [(5, 1, 6, 0), (13, 2, 14, 3), (3, 16, 4, 17), (25, 5, 26, 4), (27, 7, 0, 6), (7, 27, 8, 26), (17, 8, 18, 9), (9, 20, 10, 21), (19, 10, 20, 11), (11, 18, 12, 19), (21, 12, 22, 13), (23, 15, 24, 14), (15, 25, 16, 24), (22, 1, 23, 2)], (22, 1, 23, 2)], ['o9_12230', [(5, 1, 6, 0), (1, 13, 2, 12), (11, 3, 12, 2), (3, 11, 4, 10), (13, 5, 14, 4), (19, 6, 20, 7), (7, 26, 8, 27), (25, 8, 26, 9), (9, 20, 10, 21), (21, 14, 22, 15), (15, 24, 16, 25), (23, 16, 24, 17), (17, 22, 18, 23), (18, 0, 19, 27)], (18, 0, 19, 27)], ['o9_12736', [(3, 1, 4, 0), (1, 8, 2, 9), (7, 2, 8, 3), (11, 5, 12, 4), (5, 24, 6, 25), (9, 21, 10, 20), (19, 11, 20, 10), (27, 13, 0, 12), (13, 27, 14, 26), (25, 15, 26, 14), (23, 16, 24, 17), (17, 22, 18, 23), (21, 18, 22, 19), (6, 16, 7, 15)], (6, 16, 7, 15)], ['o9_13056', [(3, 1, 4, 0), (1, 8, 2, 9), (7, 2, 8, 3), (5, 24, 6, 25), (17, 6, 18, 7), (9, 18, 10, 19), (19, 10, 20, 11), (11, 23, 12, 22), (21, 13, 22, 12), (25, 15, 26, 14), (15, 27, 16, 26), (27, 17, 0, 16), (23, 20, 24, 21), (4, 13, 5, 14)], (4, 13, 5, 14)], ['o9_13508', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (17, 4, 18, 5), (5, 26, 6, 27), (25, 6, 26, 7), (7, 18, 8, 19), (11, 9, 12, 8), (21, 13, 22, 12), (13, 23, 14, 22), (23, 15, 24, 14), (15, 21, 16, 20), (19, 24, 20, 25), (16, 0, 17, 27)], (16, 0, 17, 27)], ['o9_14018', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (11, 4, 12, 5), (5, 20, 6, 21), (17, 6, 18, 7), (27, 10, 0, 11), (19, 13, 20, 12), (13, 19, 14, 18), (15, 25, 16, 24), (23, 17, 24, 16), (21, 26, 22, 27), (25, 22, 26, 23), (14, 8, 15, 7)], (14, 8, 15, 7)], ['o9_14136', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (19, 4, 20, 5), (5, 24, 6, 25), (7, 11, 8, 10), (11, 21, 12, 20), (21, 13, 22, 12), (13, 23, 14, 22), (23, 15, 24, 14), (25, 16, 26, 17), (17, 26, 18, 27), (27, 18, 0, 19), (6, 15, 7, 16)], (6, 15, 7, 16)], ['s042', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (17, 4, 18, 5), (5, 20, 6, 21), (7, 13, 8, 12), (11, 9, 12, 8), (13, 19, 14, 18), (19, 15, 20, 14), (21, 16, 0, 17), (6, 15, 7, 16)], (6, 15, 7, 16)], ['s068', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (11, 5, 12, 4), (5, 16, 6, 17), (13, 6, 14, 7), (21, 9, 0, 8), (17, 13, 18, 12), (19, 14, 20, 15), (15, 20, 16, 21), (18, 7, 19, 8)], (18, 7, 19, 8)], ['s104', [(5, 1, 6, 0), (1, 15, 2, 14), (7, 2, 8, 3), (3, 8, 4, 9), (13, 6, 14, 7), (9, 21, 10, 20), (17, 11, 18, 10), (11, 19, 12, 18), (19, 13, 20, 12), (21, 16, 0, 17), (4, 15, 5, 16)], (4, 15, 5, 16)], ['s114', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (11, 4, 12, 5), (13, 7, 14, 6), (7, 18, 8, 19), (21, 10, 0, 11), (17, 12, 18, 13), (19, 15, 20, 14), (15, 21, 16, 20), (16, 5, 17, 6)], (16, 5, 17, 6)], ['s344', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (15, 5, 16, 4), (5, 20, 6, 21), (19, 6, 20, 7), (7, 15, 8, 14), (11, 9, 12, 8), (17, 13, 18, 12), (13, 19, 14, 18), (16, 21, 17, 0)], (16, 21, 17, 0)], ['s369', [(3, 1, 4, 0), (1, 8, 2, 9), (7, 2, 8, 3), (15, 4, 16, 5), (5, 20, 6, 21), (9, 19, 10, 18), (17, 11, 18, 10), (11, 17, 12, 16), (19, 13, 20, 12), (21, 14, 0, 15), (6, 13, 7, 14)], (6, 13, 7, 14)], ['s769', [(3, 1, 4, 0), (1, 8, 2, 9), (7, 2, 8, 3), (17, 5, 18, 4), (5, 15, 6, 14), (9, 16, 10, 17), (15, 10, 16, 11), (19, 13, 20, 12), (13, 21, 14, 20), (21, 19, 0, 18), (6, 11, 7, 12)], (6, 11, 7, 12)], ['t00110', [(5, 1, 6, 0), (1, 15, 2, 14), (7, 2, 8, 3), (3, 8, 4, 9), (15, 5, 16, 4), (9, 23, 10, 22), (21, 11, 22, 10), (11, 21, 12, 20), (19, 13, 20, 12), (25, 16, 0, 17), (17, 24, 18, 25), (23, 18, 24, 19), (6, 14, 7, 13)], (6, 14, 7, 13)], ['t00146', [(5, 1, 6, 0), (1, 16, 2, 17), (7, 2, 8, 3), (3, 8, 4, 9), (15, 4, 16, 5), (9, 19, 10, 18), (19, 11, 20, 10), (11, 24, 12, 25), (23, 12, 24, 13), (13, 22, 14, 23), (21, 14, 22, 15), (25, 21, 0, 20), (6, 17, 7, 18)], (6, 17, 7, 18)], ['t00423', [(5, 1, 6, 0), (1, 17, 2, 16), (7, 2, 8, 3), (3, 8, 4, 9), (15, 6, 16, 7), (9, 25, 10, 24), (19, 11, 20, 10), (11, 23, 12, 22), (21, 13, 22, 12), (13, 21, 14, 20), (23, 15, 24, 14), (25, 18, 0, 19), (4, 17, 5, 18)], (4, 17, 5, 18)], ['t00434', [(5, 1, 6, 0), (1, 16, 2, 17), (7, 2, 8, 3), (3, 8, 4, 9), (17, 7, 18, 6), (9, 25, 10, 24), (19, 10, 20, 11), (11, 22, 12, 23), (21, 12, 22, 13), (13, 20, 14, 21), (25, 15, 0, 14), (23, 18, 24, 19), (4, 16, 5, 15)], (4, 16, 5, 15)], ['t00873', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (15, 5, 16, 4), (5, 22, 6, 23), (17, 6, 18, 7), (7, 18, 8, 19), (21, 8, 22, 9), (9, 15, 10, 14), (13, 11, 14, 10), (19, 25, 20, 24), (25, 21, 0, 20), (16, 23, 17, 24)], (16, 23, 17, 24)], ['t00932', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (13, 5, 14, 4), (5, 20, 6, 21), (17, 6, 18, 7), (7, 16, 8, 17), (21, 8, 22, 9), (9, 15, 10, 14), (25, 11, 0, 10), (23, 18, 24, 19), (19, 24, 20, 25), (22, 16, 23, 15)], (22, 16, 23, 15)], ['t01216', [(5, 1, 6, 0), (1, 14, 2, 15), (11, 2, 12, 3), (3, 12, 4, 13), (13, 4, 14, 5), (19, 7, 20, 6), (7, 24, 8, 25), (23, 8, 24, 9), (9, 19, 10, 18), (15, 11, 16, 10), (21, 17, 22, 16), (17, 23, 18, 22), (20, 25, 21, 0)], (20, 25, 21, 0)], ['t01292', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (13, 4, 14, 5), (5, 20, 6, 21), (15, 7, 16, 6), (7, 17, 8, 16), (21, 8, 22, 9), (9, 24, 10, 25), (25, 12, 0, 13), (17, 23, 18, 22), (23, 19, 24, 18), (14, 19, 15, 20)], (14, 19, 15, 20)], ['t01409', [(3, 1, 4, 0), (1, 8, 2, 9), (7, 2, 8, 3), (5, 24, 6, 25), (13, 7, 14, 6), (9, 21, 10, 20), (19, 11, 20, 10), (11, 17, 12, 16), (23, 13, 24, 12), (25, 14, 0, 15), (17, 22, 18, 23), (21, 18, 22, 19), (4, 16, 5, 15)], (4, 16, 5, 15)], ['t01424', [(3, 1, 4, 0), (1, 8, 2, 9), (7, 2, 8, 3), (5, 24, 6, 25), (15, 6, 16, 7), (9, 23, 10, 22), (17, 11, 18, 10), (11, 20, 12, 21), (19, 12, 20, 13), (25, 15, 0, 14), (23, 16, 24, 17), (21, 19, 22, 18), (4, 13, 5, 14)], (4, 13, 5, 14)], ['t01598', [(3, 1, 4, 0), (1, 8, 2, 9), (7, 2, 8, 3), (17, 4, 18, 5), (5, 24, 6, 25), (9, 23, 10, 22), (19, 11, 20, 10), (11, 21, 12, 20), (21, 13, 22, 12), (13, 19, 14, 18), (23, 15, 24, 14), (25, 16, 0, 17), (6, 15, 7, 16)], (6, 15, 7, 16)], ['t01646', [(3, 1, 4, 0), (1, 8, 2, 9), (7, 2, 8, 3), (13, 5, 14, 4), (5, 24, 6, 25), (9, 23, 10, 22), (19, 10, 20, 11), (11, 20, 12, 21), (17, 13, 18, 12), (25, 15, 0, 14), (23, 16, 24, 17), (21, 18, 22, 19), (6, 16, 7, 15)], (6, 16, 7, 15)], ['t01850', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (5, 24, 6, 25), (23, 6, 24, 7), (7, 22, 8, 23), (17, 9, 18, 8), (9, 15, 10, 14), (13, 11, 14, 10), (15, 21, 16, 20), (21, 17, 22, 16), (25, 18, 0, 19), (4, 20, 5, 19)], (4, 20, 5, 19)], ['t01949', [(5, 1, 6, 0), (1, 17, 2, 16), (9, 2, 10, 3), (23, 5, 24, 4), (25, 7, 0, 6), (7, 25, 8, 24), (15, 8, 16, 9), (21, 10, 22, 11), (11, 20, 12, 21), (19, 12, 20, 13), (13, 18, 14, 19), (17, 23, 18, 22), (14, 4, 15, 3)], (14, 4, 15, 3)], ['t03566', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (5, 22, 6, 23), (19, 6, 20, 7), (7, 13, 8, 12), (11, 9, 12, 8), (13, 20, 14, 21), (21, 14, 22, 15), (23, 17, 24, 16), (17, 25, 18, 24), (25, 19, 0, 18), (4, 15, 5, 16)], (4, 15, 5, 16)], ['t03709', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (11, 5, 12, 4), (15, 6, 16, 7), (7, 18, 8, 19), (25, 9, 0, 8), (19, 12, 20, 13), (13, 20, 14, 21), (21, 14, 22, 15), (23, 17, 24, 16), (17, 25, 18, 24), (22, 5, 23, 6)], (22, 5, 23, 6)], ['t04228', [(3, 1, 4, 0), (1, 8, 2, 9), (7, 2, 8, 3), (5, 20, 6, 21), (17, 6, 18, 7), (9, 18, 10, 19), (19, 10, 20, 11), (21, 13, 22, 12), (13, 25, 14, 24), (23, 15, 24, 14), (15, 23, 16, 22), (25, 17, 0, 16), (4, 11, 5, 12)], (4, 11, 5, 12)], ['t05239', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (15, 5, 16, 4), (5, 24, 6, 25), (23, 6, 24, 7), (7, 15, 8, 14), (19, 10, 20, 11), (11, 20, 12, 21), (21, 12, 22, 13), (13, 18, 14, 19), (17, 23, 18, 22), (16, 25, 17, 0)], (16, 25, 17, 0)], ['t05426', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (11, 4, 12, 5), (17, 6, 18, 7), (7, 20, 8, 21), (25, 10, 0, 11), (19, 12, 20, 13), (13, 18, 14, 19), (15, 23, 16, 22), (23, 17, 24, 16), (21, 25, 22, 24), (14, 6, 15, 5)], (14, 6, 15, 5)], ['t05578', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (11, 5, 12, 4), (5, 20, 6, 21), (13, 6, 14, 7), (7, 14, 8, 15), (19, 8, 20, 9), (15, 25, 16, 24), (23, 17, 24, 16), (17, 23, 18, 22), (25, 19, 0, 18), (12, 21, 13, 22)], (12, 21, 13, 22)], ['t05663', [(3, 1, 4, 0), (1, 8, 2, 9), (7, 2, 8, 3), (5, 22, 6, 23), (11, 7, 12, 6), (9, 19, 10, 18), (19, 11, 20, 10), (25, 12, 0, 13), (13, 24, 14, 25), (23, 14, 24, 15), (21, 17, 22, 16), (17, 21, 18, 20), (4, 16, 5, 15)], (4, 16, 5, 15)], ['v0082', [(5, 1, 6, 0), (1, 16, 2, 17), (7, 2, 8, 3), (3, 8, 4, 9), (15, 4, 16, 5), (9, 19, 10, 18), (19, 11, 20, 10), (11, 23, 12, 22), (21, 13, 22, 12), (13, 21, 14, 20), (23, 15, 0, 14), (6, 17, 7, 18)], (6, 17, 7, 18)], ['v0114', [(5, 1, 6, 0), (1, 15, 2, 14), (7, 2, 8, 3), (3, 8, 4, 9), (15, 5, 16, 4), (9, 20, 10, 21), (19, 10, 20, 11), (11, 18, 12, 19), (21, 12, 22, 13), (23, 16, 0, 17), (17, 22, 18, 23), (6, 14, 7, 13)], (6, 14, 7, 13)], ['v0220', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (13, 4, 14, 5), (15, 7, 16, 6), (7, 20, 8, 21), (11, 9, 12, 8), (23, 12, 0, 13), (19, 14, 20, 15), (21, 17, 22, 16), (17, 23, 18, 22), (18, 5, 19, 6)], (18, 5, 19, 6)], ['v0223', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (11, 5, 12, 4), (5, 20, 6, 21), (15, 7, 16, 6), (13, 8, 14, 9), (23, 13, 0, 12), (19, 15, 20, 14), (21, 16, 22, 17), (17, 22, 18, 23), (18, 7, 19, 8)], (18, 7, 19, 8)], ['v0424', [(5, 1, 6, 0), (1, 7, 2, 6), (3, 15, 4, 14), (17, 5, 18, 4), (7, 17, 8, 16), (15, 9, 16, 8), (19, 10, 20, 11), (11, 20, 12, 21), (21, 12, 22, 13), (13, 22, 14, 23), (23, 18, 0, 19), (2, 9, 3, 10)], (2, 9, 3, 10)], ['v0497', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (13, 5, 14, 4), (5, 20, 6, 21), (15, 6, 16, 7), (7, 16, 8, 17), (19, 8, 20, 9), (9, 13, 10, 12), (17, 23, 18, 22), (23, 19, 0, 18), (14, 21, 15, 22)], (14, 21, 15, 22)], ['v0709', [(5, 1, 6, 0), (1, 14, 2, 15), (9, 2, 10, 3), (23, 5, 0, 4), (15, 7, 16, 6), (7, 17, 8, 16), (17, 9, 18, 8), (19, 10, 20, 11), (11, 20, 12, 21), (21, 12, 22, 13), (13, 22, 14, 23), (18, 3, 19, 4)], (18, 3, 19, 4)], ['v0715', [(3, 1, 4, 0), (1, 8, 2, 9), (7, 2, 8, 3), (5, 22, 6, 23), (15, 6, 16, 7), (9, 21, 10, 20), (17, 11, 18, 10), (11, 19, 12, 18), (19, 13, 20, 12), (23, 15, 0, 14), (21, 16, 22, 17), (4, 13, 5, 14)], (4, 13, 5, 14)], ['v0741', [(3, 1, 4, 0), (1, 8, 2, 9), (7, 2, 8, 3), (5, 22, 6, 23), (13, 7, 14, 6), (9, 21, 10, 20), (17, 10, 18, 11), (11, 18, 12, 19), (21, 13, 22, 12), (23, 14, 0, 15), (19, 16, 20, 17), (4, 16, 5, 15)], (4, 16, 5, 15)], ['v0847', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (5, 22, 6, 23), (21, 6, 22, 7), (7, 20, 8, 21), (17, 8, 18, 9), (9, 13, 10, 12), (13, 18, 14, 19), (19, 14, 20, 15), (23, 17, 0, 16), (4, 15, 5, 16)], (4, 15, 5, 16)], ['v0912', [(5, 1, 6, 0), (9, 2, 10, 3), (3, 17, 4, 16), (21, 5, 22, 4), (23, 7, 0, 6), (7, 23, 8, 22), (15, 9, 16, 8), (17, 10, 18, 11), (11, 20, 12, 21), (19, 12, 20, 13), (13, 18, 14, 19), (14, 2, 15, 1)], (14, 2, 15, 1)], ['v1300', [(7, 1, 8, 0), (1, 9, 2, 8), (9, 3, 10, 2), (3, 19, 4, 18), (11, 4, 12, 5), (5, 12, 6, 13), (19, 7, 20, 6), (13, 20, 14, 21), (21, 14, 22, 15), (15, 22, 16, 23), (23, 16, 0, 17), (10, 18, 11, 17)], (10, 18, 11, 17)], ['v1628', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (5, 20, 6, 21), (17, 6, 18, 7), (7, 11, 8, 10), (11, 18, 12, 19), (19, 12, 20, 13), (21, 15, 22, 14), (15, 23, 16, 22), (23, 17, 0, 16), (4, 13, 5, 14)], (4, 13, 5, 14)], ['v1940', [(3, 1, 4, 0), (1, 8, 2, 9), (7, 2, 8, 3), (5, 15, 6, 14), (17, 7, 18, 6), (9, 17, 10, 16), (15, 11, 16, 10), (21, 13, 22, 12), (13, 21, 14, 20), (23, 18, 0, 19), (19, 22, 20, 23), (4, 11, 5, 12)], (4, 11, 5, 12)], ['v1966', [(3, 1, 4, 0), (1, 8, 2, 9), (7, 2, 8, 3), (15, 4, 16, 5), (5, 18, 6, 19), (9, 17, 10, 16), (17, 11, 18, 10), (19, 12, 20, 13), (13, 23, 14, 22), (21, 15, 22, 14), (23, 20, 0, 21), (6, 11, 7, 12)], (6, 11, 7, 12)], ['m082', [(5, 1, 6, 0), (1, 10, 2, 11), (9, 2, 10, 3), (3, 14, 4, 15), (13, 4, 14, 5), (17, 7, 0, 6), (15, 8, 16, 9), (11, 16, 12, 17), (12, 8, 13, 7)], (12, 8, 13, 7)], ['m144', [(5, 1, 6, 0), (1, 14, 2, 15), (9, 2, 10, 3), (3, 10, 4, 11), (17, 7, 18, 6), (7, 17, 8, 16), (15, 9, 16, 8), (11, 19, 12, 18), (19, 13, 0, 12), (4, 14, 5, 13)], (4, 14, 5, 13)], ['m194', [(1, 14, 2, 15), (9, 2, 10, 3), (3, 10, 4, 11), (13, 4, 14, 5), (17, 7, 18, 6), (7, 17, 8, 16), (15, 9, 16, 8), (11, 19, 12, 18), (19, 13, 0, 12), (0, 5, 1, 6)], (0, 5, 1, 6)], ['m198', [(5, 1, 6, 0), (1, 15, 2, 14), (15, 3, 16, 2), (3, 11, 4, 10), (11, 5, 12, 4), (7, 16, 8, 17), (17, 8, 18, 9), (9, 18, 10, 19), (19, 12, 0, 13), (6, 14, 7, 13)], (6, 14, 7, 13)], ['m239', [(5, 1, 6, 0), (1, 14, 2, 15), (7, 2, 8, 3), (9, 5, 10, 4), (15, 7, 16, 6), (13, 8, 14, 9), (17, 10, 0, 11), (11, 16, 12, 17), (12, 3, 13, 4)], (12, 3, 13, 4)], ['m281', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (5, 18, 6, 19), (13, 7, 14, 6), (7, 13, 8, 12), (17, 8, 18, 9), (9, 16, 10, 17), (19, 14, 0, 15), (4, 16, 5, 15)], (4, 16, 5, 15)], ['o9_00644', [(5, 1, 6, 0), (1, 18, 2, 19), (13, 2, 14, 3), (3, 14, 4, 15), (25, 7, 26, 6), (7, 25, 8, 24), (23, 9, 24, 8), (9, 23, 10, 22), (21, 11, 22, 10), (11, 21, 12, 20), (19, 13, 20, 12), (15, 27, 16, 26), (27, 17, 0, 16), (4, 18, 5, 17)], (4, 18, 5, 17)], ['o9_01436', [(5, 1, 6, 0), (1, 19, 2, 18), (19, 3, 20, 2), (3, 15, 4, 14), (15, 5, 16, 4), (7, 20, 8, 21), (21, 8, 22, 9), (9, 22, 10, 23), (23, 10, 24, 11), (11, 26, 12, 27), (25, 12, 26, 13), (13, 24, 14, 25), (27, 16, 0, 17), (6, 18, 7, 17)], (6, 18, 7, 17)], ['o9_01496', [(5, 1, 6, 0), (1, 12, 2, 13), (11, 2, 12, 3), (3, 16, 4, 17), (15, 4, 16, 5), (7, 23, 8, 22), (21, 9, 22, 8), (9, 21, 10, 20), (23, 11, 24, 10), (27, 15, 0, 14), (17, 27, 18, 26), (25, 19, 26, 18), (19, 25, 20, 24), (6, 13, 7, 14)], (6, 13, 7, 14)], ['o9_01584', [(1, 18, 2, 19), (13, 2, 14, 3), (3, 14, 4, 15), (17, 4, 18, 5), (21, 7, 22, 6), (7, 25, 8, 24), (23, 9, 24, 8), (9, 23, 10, 22), (25, 11, 26, 10), (11, 21, 12, 20), (19, 13, 20, 12), (15, 27, 16, 26), (27, 17, 0, 16), (0, 5, 1, 6)], (0, 5, 1, 6)], ['o9_01621', [(5, 1, 6, 0), (1, 15, 2, 14), (13, 3, 14, 2), (3, 13, 4, 12), (15, 5, 16, 4), (21, 6, 22, 7), (7, 24, 8, 25), (19, 9, 20, 8), (9, 19, 10, 18), (23, 10, 24, 11), (11, 22, 12, 23), (27, 16, 0, 17), (17, 26, 18, 27), (20, 26, 21, 25)], (20, 26, 21, 25)], ['o9_01953', [(7, 1, 8, 0), (9, 2, 10, 3), (3, 10, 4, 11), (19, 4, 20, 5), (5, 18, 6, 19), (17, 6, 18, 7), (21, 9, 22, 8), (11, 27, 12, 26), (25, 13, 26, 12), (13, 25, 14, 24), (23, 15, 24, 14), (15, 23, 16, 22), (27, 17, 0, 16), (20, 2, 21, 1)], (20, 2, 21, 1)], ['o9_01955', [(7, 1, 8, 0), (1, 21, 2, 20), (9, 2, 10, 3), (3, 16, 4, 17), (17, 4, 18, 5), (5, 18, 6, 19), (27, 7, 0, 6), (21, 11, 22, 10), (11, 27, 12, 26), (25, 13, 26, 12), (13, 25, 14, 24), (23, 15, 24, 14), (15, 23, 16, 22), (8, 20, 9, 19)], (8, 20, 9, 19)], ['o9_02255', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (15, 5, 16, 4), (5, 20, 6, 21), (19, 6, 20, 7), (7, 24, 8, 25), (23, 8, 24, 9), (9, 15, 10, 14), (13, 11, 14, 10), (27, 17, 0, 16), (25, 18, 26, 19), (21, 26, 22, 27), (22, 18, 23, 17)], (22, 18, 23, 17)], ['o9_02350', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (13, 5, 14, 4), (5, 27, 6, 26), (21, 6, 22, 7), (7, 22, 8, 23), (17, 8, 18, 9), (9, 18, 10, 19), (27, 11, 0, 10), (25, 14, 26, 15), (23, 17, 24, 16), (19, 25, 20, 24), (20, 15, 21, 16)], (20, 15, 21, 16)], ['o9_02655', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (5, 22, 6, 23), (15, 6, 16, 7), (7, 21, 8, 20), (11, 9, 12, 8), (25, 12, 0, 13), (13, 24, 14, 25), (19, 15, 20, 14), (21, 16, 22, 17), (23, 19, 24, 18), (4, 17, 5, 18)], (4, 17, 5, 18)], ['o9_02696', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (11, 5, 12, 4), (15, 6, 16, 7), (7, 22, 8, 23), (17, 9, 18, 8), (25, 13, 0, 12), (13, 25, 14, 24), (19, 14, 20, 15), (21, 17, 22, 16), (23, 18, 24, 19), (20, 5, 21, 6)], (20, 5, 21, 6)], ['o9_02772', [(7, 1, 8, 0), (1, 20, 2, 21), (9, 2, 10, 3), (3, 10, 4, 11), (17, 4, 18, 5), (5, 18, 6, 19), (19, 6, 20, 7), (11, 23, 12, 22), (23, 13, 24, 12), (13, 25, 14, 24), (25, 15, 26, 14), (15, 27, 16, 26), (27, 17, 0, 16), (8, 21, 9, 22)], (8, 21, 9, 22)], ['o9_02786', [(5, 1, 6, 0), (1, 22, 2, 23), (7, 2, 8, 3), (3, 17, 4, 16), (13, 5, 14, 4), (21, 8, 22, 9), (9, 20, 10, 21), (19, 10, 20, 11), (11, 18, 12, 19), (17, 12, 18, 13), (25, 14, 0, 15), (15, 24, 16, 25), (6, 23, 7, 24)], (6, 23, 7, 24)], ['o9_02794', [(3, 1, 4, 0), (1, 15, 2, 14), (15, 3, 16, 2), (19, 4, 20, 5), (5, 18, 6, 19), (27, 7, 0, 6), (7, 27, 8, 26), (25, 9, 26, 8), (9, 25, 10, 24), (23, 11, 24, 10), (11, 23, 12, 22), (17, 12, 18, 13), (21, 16, 22, 17), (20, 14, 21, 13)], (20, 14, 21, 13)], ['o9_03032', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (23, 4, 24, 5), (5, 22, 6, 23), (21, 6, 22, 7), (7, 26, 8, 27), (9, 17, 10, 16), (15, 11, 16, 10), (11, 15, 12, 14), (17, 25, 18, 24), (25, 19, 26, 18), (27, 20, 0, 21), (8, 19, 9, 20)], (8, 19, 9, 20)], ['o9_03108', [(7, 1, 8, 0), (1, 19, 2, 18), (19, 3, 20, 2), (3, 21, 4, 20), (21, 5, 22, 4), (5, 15, 6, 14), (15, 7, 16, 6), (17, 8, 18, 9), (9, 22, 10, 23), (23, 10, 24, 11), (11, 24, 12, 25), (25, 12, 26, 13), (13, 26, 14, 27), (16, 0, 17, 27)], (16, 0, 17, 27)], ['o9_03133', [(5, 1, 6, 0), (1, 13, 2, 12), (13, 3, 14, 2), (3, 15, 4, 14), (15, 5, 16, 4), (21, 6, 22, 7), (7, 22, 8, 23), (23, 8, 24, 9), (9, 26, 10, 27), (11, 17, 12, 16), (17, 25, 18, 24), (25, 19, 26, 18), (27, 20, 0, 21), (10, 19, 11, 20)], (10, 19, 11, 20)], ['o9_03162', [(3, 1, 4, 0), (1, 14, 2, 15), (13, 2, 14, 3), (17, 4, 18, 5), (5, 26, 6, 27), (7, 25, 8, 24), (19, 8, 20, 9), (9, 18, 10, 19), (23, 11, 24, 10), (11, 23, 12, 22), (15, 13, 16, 12), (27, 16, 0, 17), (25, 21, 26, 20), (6, 21, 7, 22)], (6, 21, 7, 22)], ['o9_03188', [(3, 1, 4, 0), (1, 14, 2, 15), (13, 2, 14, 3), (15, 5, 16, 4), (5, 26, 6, 27), (17, 6, 18, 7), (19, 9, 20, 8), (9, 24, 10, 25), (25, 10, 26, 11), (11, 21, 12, 20), (21, 13, 22, 12), (27, 17, 0, 16), (23, 18, 24, 19), (22, 7, 23, 8)], (22, 7, 23, 8)], ['o9_03313', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (15, 5, 16, 4), (5, 17, 6, 16), (7, 26, 8, 27), (23, 8, 24, 9), (9, 22, 10, 23), (21, 10, 22, 11), (11, 15, 12, 14), (17, 24, 18, 25), (25, 18, 26, 19), (27, 21, 0, 20), (6, 19, 7, 20)], (6, 19, 7, 20)], ['o9_03586', [(7, 1, 8, 0), (1, 20, 2, 21), (13, 2, 14, 3), (3, 14, 4, 15), (15, 4, 16, 5), (5, 16, 6, 17), (21, 9, 22, 8), (9, 25, 10, 24), (23, 11, 24, 10), (11, 23, 12, 22), (25, 13, 26, 12), (17, 27, 18, 26), (27, 19, 0, 18), (6, 20, 7, 19)], (6, 20, 7, 19)], ['o9_03622', [(5, 1, 6, 0), (1, 17, 2, 16), (15, 3, 16, 2), (3, 15, 4, 14), (17, 5, 18, 4), (21, 6, 22, 7), (19, 9, 20, 8), (9, 19, 10, 18), (25, 10, 26, 11), (11, 24, 12, 25), (23, 12, 24, 13), (13, 22, 14, 23), (27, 20, 0, 21), (26, 8, 27, 7)], (26, 8, 27, 7)], ['o9_03802', [(1, 20, 2, 21), (13, 2, 14, 3), (3, 14, 4, 15), (15, 4, 16, 5), (5, 16, 6, 17), (19, 6, 20, 7), (23, 9, 24, 8), (9, 25, 10, 24), (25, 11, 26, 10), (11, 23, 12, 22), (21, 13, 22, 12), (17, 27, 18, 26), (27, 19, 0, 18), (0, 7, 1, 8)], (0, 7, 1, 8)], ['o9_03833', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (17, 5, 18, 4), (23, 6, 24, 7), (7, 22, 8, 23), (21, 8, 22, 9), (9, 20, 10, 21), (19, 10, 20, 11), (11, 15, 12, 14), (15, 24, 16, 25), (25, 16, 26, 17), (27, 19, 0, 18), (26, 6, 27, 5)], (26, 6, 27, 5)], ['o9_04106', [(3, 1, 4, 0), (1, 15, 2, 14), (15, 3, 16, 2), (19, 4, 20, 5), (17, 7, 18, 6), (7, 17, 8, 16), (23, 8, 24, 9), (9, 24, 10, 25), (25, 10, 26, 11), (11, 22, 12, 23), (21, 12, 22, 13), (13, 20, 14, 21), (27, 18, 0, 19), (26, 6, 27, 5)], (26, 6, 27, 5)], ['o9_04205', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (15, 5, 16, 4), (21, 7, 22, 6), (7, 23, 8, 22), (19, 8, 20, 9), (9, 18, 10, 19), (17, 10, 18, 11), (13, 24, 14, 25), (25, 14, 26, 15), (27, 17, 0, 16), (23, 21, 24, 20), (26, 6, 27, 5)], (26, 6, 27, 5)], ['o9_04245', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (17, 4, 18, 5), (5, 16, 6, 17), (27, 7, 0, 6), (7, 15, 8, 14), (21, 8, 22, 9), (9, 22, 10, 23), (23, 10, 24, 11), (11, 20, 12, 21), (25, 19, 26, 18), (19, 25, 20, 24), (26, 15, 27, 16)], (26, 15, 27, 16)], ['o9_04269', [(5, 1, 6, 0), (7, 2, 8, 3), (3, 17, 4, 16), (11, 5, 12, 4), (21, 7, 22, 6), (17, 8, 18, 9), (9, 18, 10, 19), (19, 10, 20, 11), (25, 12, 0, 13), (13, 24, 14, 25), (23, 14, 24, 15), (15, 22, 16, 23), (20, 2, 21, 1)], (20, 2, 21, 1)], ['o9_04438', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (17, 4, 18, 5), (5, 22, 6, 23), (19, 7, 20, 6), (7, 21, 8, 20), (23, 8, 24, 9), (15, 11, 16, 10), (13, 25, 14, 24), (25, 15, 26, 14), (27, 16, 0, 17), (21, 19, 22, 18), (26, 10, 27, 9)], (26, 10, 27, 9)], ['o9_05021', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (19, 4, 20, 5), (5, 22, 6, 23), (21, 6, 22, 7), (7, 20, 8, 21), (23, 8, 24, 9), (9, 26, 10, 27), (11, 15, 12, 14), (15, 25, 16, 24), (25, 17, 26, 16), (27, 18, 0, 19), (10, 17, 11, 18)], (10, 17, 11, 18)], ['o9_05177', [(3, 1, 4, 0), (1, 14, 2, 15), (13, 2, 14, 3), (19, 5, 20, 4), (5, 23, 6, 22), (21, 7, 22, 6), (7, 21, 8, 20), (27, 9, 0, 8), (25, 10, 26, 11), (11, 24, 12, 25), (15, 13, 16, 12), (17, 27, 18, 26), (23, 19, 24, 18), (16, 9, 17, 10)], (16, 9, 17, 10)], ['o9_05229', [(3, 1, 4, 0), (1, 15, 2, 14), (15, 3, 16, 2), (21, 4, 22, 5), (5, 22, 6, 23), (23, 6, 24, 7), (7, 20, 8, 21), (27, 9, 0, 8), (25, 10, 26, 11), (11, 24, 12, 25), (17, 13, 18, 12), (13, 17, 14, 16), (19, 27, 20, 26), (18, 9, 19, 10)], (18, 9, 19, 10)], ['o9_05357', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (19, 4, 20, 5), (5, 22, 6, 23), (21, 6, 22, 7), (7, 20, 8, 21), (23, 8, 24, 9), (17, 11, 18, 10), (11, 15, 12, 14), (15, 25, 16, 24), (25, 17, 26, 16), (27, 18, 0, 19), (26, 10, 27, 9)], (26, 10, 27, 9)], ['o9_05426', [(3, 1, 4, 0), (1, 14, 2, 15), (13, 2, 14, 3), (15, 5, 16, 4), (17, 6, 18, 7), (7, 26, 8, 27), (25, 8, 26, 9), (9, 24, 10, 25), (23, 10, 24, 11), (11, 18, 12, 19), (19, 12, 20, 13), (21, 17, 22, 16), (27, 23, 0, 22), (20, 6, 21, 5)], (20, 6, 21, 5)], ['o9_05483', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (23, 4, 24, 5), (5, 24, 6, 25), (19, 7, 20, 6), (7, 19, 8, 18), (17, 9, 18, 8), (9, 17, 10, 16), (27, 12, 0, 13), (25, 15, 26, 14), (15, 20, 16, 21), (21, 27, 22, 26), (22, 13, 23, 14)], (22, 13, 23, 14)], ['o9_05562', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (19, 4, 20, 5), (5, 24, 6, 25), (17, 7, 18, 6), (7, 17, 8, 16), (21, 8, 22, 9), (9, 22, 10, 23), (23, 10, 24, 11), (11, 20, 12, 21), (27, 14, 0, 15), (15, 26, 16, 27), (18, 26, 19, 25)], (18, 26, 19, 25)], ['o9_05618', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (21, 4, 22, 5), (5, 20, 6, 21), (13, 6, 14, 7), (7, 26, 8, 27), (15, 9, 16, 8), (9, 17, 10, 16), (27, 12, 0, 13), (17, 25, 18, 24), (23, 19, 24, 18), (19, 23, 20, 22), (14, 25, 15, 26)], (14, 25, 15, 26)], ['o9_05860', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (17, 4, 18, 5), (5, 20, 6, 21), (19, 6, 20, 7), (7, 18, 8, 19), (25, 9, 26, 8), (9, 25, 10, 24), (13, 11, 14, 10), (23, 15, 24, 14), (21, 16, 22, 17), (27, 23, 0, 22), (26, 15, 27, 16)], (26, 15, 27, 16)], ['o9_05970', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (13, 5, 14, 4), (5, 15, 6, 14), (7, 26, 8, 27), (21, 9, 22, 8), (9, 23, 10, 22), (19, 10, 20, 11), (15, 24, 16, 25), (25, 16, 26, 17), (27, 19, 0, 18), (23, 21, 24, 20), (6, 17, 7, 18)], (6, 17, 7, 18)], ['o9_06060', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (15, 4, 16, 5), (5, 22, 6, 23), (21, 6, 22, 7), (7, 16, 8, 17), (11, 9, 12, 8), (17, 12, 18, 13), (13, 25, 14, 24), (19, 15, 20, 14), (23, 21, 24, 20), (18, 0, 19, 25)], (18, 0, 19, 25)], ['o9_06128', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (13, 5, 14, 4), (5, 24, 6, 25), (21, 6, 22, 7), (7, 22, 8, 23), (23, 8, 24, 9), (9, 17, 10, 16), (17, 11, 18, 10), (19, 14, 20, 15), (27, 18, 0, 19), (25, 21, 26, 20), (26, 16, 27, 15)], (26, 16, 27, 15)], ['o9_06154', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (17, 5, 18, 4), (21, 6, 22, 7), (7, 22, 8, 23), (15, 8, 16, 9), (11, 16, 12, 17), (13, 19, 14, 18), (25, 15, 0, 14), (19, 24, 20, 25), (23, 20, 24, 21), (12, 6, 13, 5)], (12, 6, 13, 5)], ['o9_06248', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (11, 5, 12, 4), (5, 16, 6, 17), (15, 6, 16, 7), (7, 22, 8, 23), (21, 8, 22, 9), (27, 13, 0, 12), (13, 18, 14, 19), (25, 14, 26, 15), (19, 25, 20, 24), (23, 21, 24, 20), (26, 18, 27, 17)], (26, 18, 27, 17)], ['o9_06301', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (13, 4, 14, 5), (15, 7, 16, 6), (7, 23, 8, 22), (21, 9, 22, 8), (9, 24, 10, 25), (27, 12, 0, 13), (19, 14, 20, 15), (25, 17, 26, 16), (17, 27, 18, 26), (23, 20, 24, 21), (18, 5, 19, 6)], (18, 5, 19, 6)], ['o9_07790', [(5, 1, 6, 0), (1, 19, 2, 18), (19, 3, 20, 2), (3, 15, 4, 14), (15, 5, 16, 4), (27, 7, 0, 6), (21, 8, 22, 9), (9, 22, 10, 23), (23, 10, 24, 11), (11, 24, 12, 25), (25, 12, 26, 13), (13, 20, 14, 21), (17, 27, 18, 26), (16, 7, 17, 8)], (16, 7, 17, 8)], ['o9_07893', [(1, 17, 2, 16), (7, 2, 8, 3), (3, 18, 4, 19), (9, 5, 10, 4), (15, 6, 16, 7), (17, 9, 18, 8), (19, 10, 20, 11), (11, 25, 12, 24), (21, 13, 22, 12), (13, 23, 14, 22), (23, 15, 24, 14), (25, 20, 0, 21), (0, 5, 1, 6)], (0, 5, 1, 6)], ['o9_07945', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (13, 4, 14, 5), (5, 24, 6, 25), (17, 6, 18, 7), (7, 23, 8, 22), (15, 9, 16, 8), (9, 20, 10, 21), (25, 12, 0, 13), (21, 17, 22, 16), (23, 18, 24, 19), (14, 20, 15, 19)], (14, 20, 15, 19)], ['o9_08006', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (21, 4, 22, 5), (5, 20, 6, 21), (7, 26, 8, 27), (23, 8, 24, 9), (9, 22, 10, 23), (15, 11, 16, 10), (11, 15, 12, 14), (27, 16, 0, 17), (25, 19, 26, 18), (19, 25, 20, 24), (6, 18, 7, 17)], (6, 18, 7, 17)], ['o9_08224', [(5, 1, 6, 0), (1, 18, 2, 19), (9, 3, 10, 2), (3, 9, 4, 8), (15, 7, 16, 6), (7, 24, 8, 25), (19, 10, 20, 11), (11, 22, 12, 23), (21, 12, 22, 13), (13, 20, 14, 21), (23, 14, 24, 15), (25, 17, 0, 16), (4, 18, 5, 17)], (4, 18, 5, 17)], ['o9_08302', [(5, 1, 6, 0), (1, 15, 2, 14), (9, 3, 10, 2), (3, 9, 4, 8), (17, 6, 18, 7), (7, 24, 8, 25), (19, 11, 20, 10), (11, 23, 12, 22), (21, 13, 22, 12), (13, 21, 14, 20), (25, 16, 0, 17), (23, 19, 24, 18), (4, 15, 5, 16)], (4, 15, 5, 16)], ['o9_08477', [(1, 19, 2, 18), (19, 3, 20, 2), (3, 13, 4, 12), (13, 5, 14, 4), (17, 6, 18, 7), (7, 24, 8, 25), (23, 8, 24, 9), (9, 22, 10, 23), (21, 10, 22, 11), (11, 20, 12, 21), (25, 14, 26, 15), (15, 26, 16, 27), (27, 16, 0, 17), (0, 5, 1, 6)], (0, 5, 1, 6)], ['o9_08647', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (19, 5, 20, 4), (5, 24, 6, 25), (21, 6, 22, 7), (7, 22, 8, 23), (23, 8, 24, 9), (9, 14, 10, 15), (17, 10, 18, 11), (15, 27, 16, 26), (27, 17, 0, 16), (25, 21, 26, 20), (18, 14, 19, 13)], (18, 14, 19, 13)], ['o9_08765', [(5, 1, 6, 0), (1, 10, 2, 11), (9, 2, 10, 3), (3, 18, 4, 19), (17, 4, 18, 5), (21, 7, 22, 6), (7, 12, 8, 13), (19, 8, 20, 9), (13, 25, 14, 24), (23, 15, 24, 14), (15, 23, 16, 22), (25, 17, 0, 16), (20, 12, 21, 11)], (20, 12, 21, 11)], ['o9_08771', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (13, 4, 14, 5), (5, 26, 6, 27), (17, 6, 18, 7), (7, 16, 8, 17), (21, 9, 22, 8), (9, 23, 10, 22), (27, 12, 0, 13), (15, 24, 16, 25), (25, 18, 26, 19), (23, 21, 24, 20), (14, 20, 15, 19)], (14, 20, 15, 19)], ['o9_08828', [(5, 1, 6, 0), (1, 13, 2, 12), (13, 3, 14, 2), (3, 17, 4, 16), (17, 5, 18, 4), (19, 6, 20, 7), (7, 24, 8, 25), (21, 8, 22, 9), (9, 22, 10, 23), (23, 10, 24, 11), (11, 20, 12, 21), (25, 14, 26, 15), (15, 26, 16, 27), (18, 0, 19, 27)], (18, 0, 19, 27)], ['o9_08831', [(5, 1, 6, 0), (1, 10, 2, 11), (9, 2, 10, 3), (3, 21, 4, 20), (21, 5, 22, 4), (17, 6, 18, 7), (7, 12, 8, 13), (19, 9, 20, 8), (13, 24, 14, 25), (23, 14, 24, 15), (15, 22, 16, 23), (25, 16, 0, 17), (18, 11, 19, 12)], (18, 11, 19, 12)], ['o9_08875', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (5, 24, 6, 25), (23, 6, 24, 7), (7, 19, 8, 18), (17, 9, 18, 8), (9, 22, 10, 23), (21, 10, 22, 11), (11, 17, 12, 16), (27, 14, 0, 15), (15, 26, 16, 27), (25, 21, 26, 20), (4, 19, 5, 20)], (4, 19, 5, 20)], ['o9_09213', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (5, 26, 6, 27), (21, 6, 22, 7), (7, 20, 8, 21), (19, 8, 20, 9), (9, 18, 10, 19), (17, 10, 18, 11), (13, 25, 14, 24), (23, 15, 24, 14), (27, 17, 0, 16), (25, 22, 26, 23), (4, 15, 5, 16)], (4, 15, 5, 16)], ['o9_09465', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (15, 5, 16, 4), (5, 26, 6, 27), (19, 7, 20, 6), (7, 21, 8, 20), (21, 9, 22, 8), (9, 23, 10, 22), (17, 10, 18, 11), (13, 24, 14, 25), (25, 14, 26, 15), (23, 19, 24, 18), (16, 27, 17, 0)], (16, 27, 17, 0)], ['o9_09808', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (15, 5, 16, 4), (5, 20, 6, 21), (21, 6, 22, 7), (7, 26, 8, 27), (25, 8, 26, 9), (9, 19, 10, 18), (11, 15, 12, 14), (27, 17, 0, 16), (17, 25, 18, 24), (19, 22, 20, 23), (10, 23, 11, 24)], (10, 23, 11, 24)], ['o9_11248', [(5, 1, 6, 0), (1, 19, 2, 18), (19, 3, 20, 2), (3, 15, 4, 14), (15, 5, 16, 4), (27, 7, 0, 6), (7, 27, 8, 26), (9, 24, 10, 25), (21, 10, 22, 11), (11, 22, 12, 23), (23, 12, 24, 13), (13, 20, 14, 21), (25, 16, 26, 17), (8, 18, 9, 17)], (8, 18, 9, 17)], ['o9_11570', [(5, 1, 6, 0), (1, 13, 2, 12), (13, 3, 14, 2), (3, 15, 4, 14), (15, 5, 16, 4), (19, 6, 20, 7), (7, 26, 8, 27), (25, 8, 26, 9), (9, 20, 10, 21), (21, 10, 22, 11), (11, 22, 12, 23), (23, 16, 24, 17), (17, 24, 18, 25), (18, 0, 19, 27)], (18, 0, 19, 27)], ['o9_11685', [(5, 1, 6, 0), (1, 20, 2, 21), (11, 3, 12, 2), (3, 11, 4, 10), (15, 7, 16, 6), (7, 17, 8, 16), (17, 9, 18, 8), (9, 24, 10, 25), (21, 12, 22, 13), (13, 22, 14, 23), (23, 14, 24, 15), (25, 19, 0, 18), (4, 20, 5, 19)], (4, 20, 5, 19)], ['o9_11795', [(1, 22, 2, 23), (11, 2, 12, 3), (3, 12, 4, 13), (21, 4, 22, 5), (23, 7, 24, 6), (7, 17, 8, 16), (17, 9, 18, 8), (9, 19, 10, 18), (19, 11, 20, 10), (13, 21, 14, 20), (25, 14, 0, 15), (15, 24, 16, 25), (0, 5, 1, 6)], (0, 5, 1, 6)], ['o9_11845', [(5, 1, 6, 0), (1, 18, 2, 19), (9, 2, 10, 3), (11, 5, 12, 4), (25, 7, 0, 6), (7, 25, 8, 24), (19, 9, 20, 8), (17, 10, 18, 11), (23, 12, 24, 13), (13, 22, 14, 23), (21, 14, 22, 15), (15, 20, 16, 21), (16, 3, 17, 4)], (16, 3, 17, 4)], ['o9_11999', [(3, 1, 4, 0), (1, 14, 2, 15), (13, 2, 14, 3), (21, 5, 22, 4), (5, 19, 6, 18), (23, 6, 24, 7), (7, 26, 8, 27), (25, 8, 26, 9), (9, 24, 10, 25), (19, 11, 20, 10), (11, 21, 12, 20), (15, 13, 16, 12), (27, 16, 0, 17), (22, 18, 23, 17)], (22, 18, 23, 17)], ['o9_12412', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (17, 4, 18, 5), (5, 16, 6, 17), (15, 6, 16, 7), (7, 23, 8, 22), (23, 9, 24, 8), (9, 18, 10, 19), (19, 12, 20, 13), (21, 15, 22, 14), (25, 20, 0, 21), (24, 13, 25, 14)], (24, 13, 25, 14)], ['o9_12459', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (5, 24, 6, 25), (23, 6, 24, 7), (7, 22, 8, 23), (15, 8, 16, 9), (9, 21, 10, 20), (13, 11, 14, 10), (19, 15, 20, 14), (21, 16, 22, 17), (25, 19, 0, 18), (4, 17, 5, 18)], (4, 17, 5, 18)], ['o9_12519', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (5, 24, 6, 25), (15, 7, 16, 6), (7, 23, 8, 22), (21, 9, 22, 8), (9, 21, 10, 20), (19, 11, 20, 10), (13, 19, 14, 18), (23, 15, 24, 14), (25, 16, 0, 17), (4, 18, 5, 17)], (4, 18, 5, 17)], ['o9_12693', [(7, 1, 8, 0), (1, 9, 2, 8), (3, 24, 4, 25), (19, 4, 20, 5), (5, 18, 6, 19), (17, 6, 18, 7), (9, 20, 10, 21), (21, 10, 22, 11), (11, 22, 12, 23), (23, 12, 24, 13), (25, 15, 26, 14), (15, 27, 16, 26), (27, 17, 0, 16), (2, 13, 3, 14)], (2, 13, 3, 14)], ['o9_12757', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (17, 4, 18, 5), (5, 26, 6, 27), (25, 6, 26, 7), (7, 22, 8, 23), (21, 8, 22, 9), (9, 20, 10, 21), (19, 10, 20, 11), (11, 18, 12, 19), (23, 14, 24, 15), (15, 24, 16, 25), (16, 0, 17, 27)], (16, 0, 17, 27)], ['o9_12873', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (5, 26, 6, 27), (15, 7, 16, 6), (7, 17, 8, 16), (17, 9, 18, 8), (11, 25, 12, 24), (21, 12, 22, 13), (13, 22, 14, 23), (25, 15, 26, 14), (27, 18, 0, 19), (23, 20, 24, 21), (4, 20, 5, 19)], (4, 20, 5, 19)], ['o9_12892', [(5, 1, 6, 0), (13, 3, 14, 2), (3, 13, 4, 12), (19, 5, 20, 4), (17, 6, 18, 7), (7, 14, 8, 15), (21, 9, 22, 8), (9, 23, 10, 22), (23, 11, 24, 10), (11, 21, 12, 20), (15, 24, 16, 25), (25, 16, 0, 17), (18, 1, 19, 2)], (18, 1, 19, 2)], ['o9_12919', [(5, 1, 6, 0), (9, 2, 10, 3), (3, 17, 4, 16), (11, 5, 12, 4), (25, 7, 0, 6), (7, 25, 8, 24), (19, 9, 20, 8), (17, 10, 18, 11), (21, 12, 22, 13), (13, 22, 14, 23), (23, 14, 24, 15), (15, 20, 16, 21), (18, 2, 19, 1)], (18, 2, 19, 1)], ['o9_12971', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (17, 5, 18, 4), (5, 24, 6, 25), (23, 6, 24, 7), (7, 22, 8, 23), (15, 8, 16, 9), (9, 14, 10, 15), (19, 10, 20, 11), (21, 16, 22, 17), (25, 19, 0, 18), (20, 14, 21, 13)], (20, 14, 21, 13)], ['o9_13052', [(3, 1, 4, 0), (1, 15, 2, 14), (15, 3, 16, 2), (5, 26, 6, 27), (25, 6, 26, 7), (7, 24, 8, 25), (19, 9, 20, 8), (9, 19, 10, 18), (23, 10, 24, 11), (11, 22, 12, 23), (17, 13, 18, 12), (13, 17, 14, 16), (27, 20, 0, 21), (4, 22, 5, 21)], (4, 22, 5, 21)], ['o9_13125', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (13, 5, 14, 4), (5, 21, 6, 20), (15, 6, 16, 7), (7, 22, 8, 23), (23, 8, 24, 9), (25, 11, 0, 10), (19, 14, 20, 15), (21, 17, 22, 16), (17, 25, 18, 24), (18, 10, 19, 9)], (18, 10, 19, 9)], ['o9_13182', [(7, 1, 8, 0), (1, 20, 2, 21), (11, 2, 12, 3), (3, 19, 4, 18), (15, 5, 16, 4), (5, 15, 6, 14), (13, 7, 14, 6), (23, 9, 24, 8), (9, 23, 10, 22), (21, 11, 22, 10), (25, 16, 0, 17), (17, 24, 18, 25), (12, 20, 13, 19)], (12, 20, 13, 19)], ['o9_13188', [(3, 1, 4, 0), (1, 15, 2, 14), (15, 3, 16, 2), (21, 5, 22, 4), (5, 26, 6, 27), (25, 6, 26, 7), (7, 21, 8, 20), (19, 9, 20, 8), (9, 19, 10, 18), (17, 11, 18, 10), (11, 24, 12, 25), (23, 12, 24, 13), (13, 17, 14, 16), (22, 27, 23, 0)], (22, 27, 23, 0)], ['o9_13400', [(5, 1, 6, 0), (1, 15, 2, 14), (15, 3, 16, 2), (3, 17, 4, 16), (17, 5, 18, 4), (7, 26, 8, 27), (19, 9, 20, 8), (9, 19, 10, 18), (23, 10, 24, 11), (11, 24, 12, 25), (25, 12, 26, 13), (13, 22, 14, 23), (27, 20, 0, 21), (6, 22, 7, 21)], (6, 22, 7, 21)], ['o9_13403', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (19, 5, 20, 4), (5, 21, 6, 20), (15, 6, 16, 7), (7, 24, 8, 25), (23, 8, 24, 9), (9, 16, 10, 17), (11, 19, 12, 18), (25, 14, 0, 15), (17, 22, 18, 23), (10, 21, 11, 22)], (10, 21, 11, 22)], ['o9_13433', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (13, 4, 14, 5), (15, 7, 16, 6), (7, 22, 8, 23), (17, 8, 18, 9), (9, 18, 10, 19), (25, 12, 0, 13), (21, 14, 22, 15), (23, 17, 24, 16), (19, 25, 20, 24), (20, 5, 21, 6)], (20, 5, 21, 6)], ['o9_13537', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (19, 4, 20, 5), (5, 13, 6, 12), (7, 14, 8, 15), (25, 10, 0, 11), (11, 19, 12, 18), (13, 20, 14, 21), (15, 25, 16, 24), (23, 17, 24, 16), (17, 23, 18, 22), (6, 21, 7, 22)], (6, 21, 7, 22)], ['o9_13604', [(3, 1, 4, 0), (1, 15, 2, 14), (15, 3, 16, 2), (19, 4, 20, 5), (5, 20, 6, 21), (21, 6, 22, 7), (7, 18, 8, 19), (9, 17, 10, 16), (25, 10, 26, 11), (11, 24, 12, 25), (23, 12, 24, 13), (13, 22, 14, 23), (17, 27, 18, 26), (8, 27, 9, 0)], (8, 27, 9, 0)], ['o9_13639', [(5, 1, 6, 0), (1, 18, 2, 19), (13, 3, 14, 2), (3, 13, 4, 12), (17, 4, 18, 5), (23, 7, 24, 6), (7, 25, 8, 24), (25, 9, 0, 8), (9, 16, 10, 17), (21, 10, 22, 11), (11, 20, 12, 21), (19, 14, 20, 15), (22, 16, 23, 15)], (22, 16, 23, 15)], ['o9_13649', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (15, 5, 16, 4), (13, 6, 14, 7), (7, 20, 8, 21), (21, 8, 22, 9), (9, 22, 10, 23), (23, 12, 24, 13), (19, 15, 20, 14), (25, 17, 0, 16), (17, 25, 18, 24), (18, 5, 19, 6)], (18, 5, 19, 6)], ['o9_13666', [(1, 20, 2, 21), (13, 2, 14, 3), (3, 14, 4, 15), (17, 4, 18, 5), (5, 18, 6, 19), (19, 6, 20, 7), (21, 9, 22, 8), (9, 25, 10, 24), (23, 11, 24, 10), (11, 23, 12, 22), (25, 13, 26, 12), (15, 27, 16, 26), (27, 17, 0, 16), (0, 7, 1, 8)], (0, 7, 1, 8)], ['o9_13720', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (15, 5, 16, 4), (5, 18, 6, 19), (17, 6, 18, 7), (7, 21, 8, 20), (27, 9, 0, 8), (9, 22, 10, 23), (23, 10, 24, 11), (13, 24, 14, 25), (25, 14, 26, 15), (19, 17, 20, 16), (26, 22, 27, 21)], (26, 22, 27, 21)], ['o9_13952', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (15, 5, 16, 4), (5, 24, 6, 25), (17, 6, 18, 7), (7, 18, 8, 19), (21, 8, 22, 9), (9, 22, 10, 23), (23, 10, 24, 11), (11, 15, 12, 14), (19, 27, 20, 26), (27, 21, 0, 20), (16, 25, 17, 26)], (16, 25, 17, 26)], ['o9_14079', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (5, 26, 6, 27), (17, 6, 18, 7), (7, 18, 8, 19), (19, 8, 20, 9), (11, 25, 12, 24), (21, 13, 22, 12), (13, 23, 14, 22), (23, 15, 24, 14), (27, 17, 0, 16), (25, 20, 26, 21), (4, 15, 5, 16)], (4, 15, 5, 16)], ['o9_14364', [(5, 1, 6, 0), (1, 11, 2, 10), (11, 3, 12, 2), (3, 16, 4, 17), (15, 4, 16, 5), (19, 6, 20, 7), (7, 20, 8, 21), (21, 8, 22, 9), (9, 22, 10, 23), (23, 12, 24, 13), (25, 15, 0, 14), (17, 25, 18, 24), (18, 14, 19, 13)], (18, 14, 19, 13)], ['o9_14376', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (21, 5, 22, 4), (5, 21, 6, 20), (15, 6, 16, 7), (7, 16, 8, 17), (27, 10, 0, 11), (11, 26, 12, 27), (13, 25, 14, 24), (23, 15, 24, 14), (25, 18, 26, 19), (19, 22, 20, 23), (12, 18, 13, 17)], (12, 18, 13, 17)], ['o9_14495', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (17, 4, 18, 5), (5, 18, 6, 19), (13, 7, 14, 6), (7, 22, 8, 23), (21, 8, 22, 9), (9, 13, 10, 12), (23, 15, 24, 14), (15, 21, 16, 20), (25, 16, 0, 17), (24, 20, 25, 19)], (24, 20, 25, 19)], ['o9_14599', [(3, 1, 4, 0), (1, 15, 2, 14), (15, 3, 16, 2), (17, 5, 18, 4), (5, 24, 6, 25), (23, 6, 24, 7), (7, 22, 8, 23), (25, 8, 26, 9), (27, 11, 0, 10), (11, 20, 12, 21), (21, 12, 22, 13), (13, 17, 14, 16), (19, 26, 20, 27), (18, 9, 19, 10)], (18, 9, 19, 10)], ['o9_14716', [(7, 1, 8, 0), (1, 20, 2, 21), (11, 2, 12, 3), (3, 14, 4, 15), (13, 4, 14, 5), (5, 12, 6, 13), (23, 9, 24, 8), (9, 23, 10, 22), (21, 11, 22, 10), (15, 27, 16, 26), (25, 17, 26, 16), (17, 25, 18, 24), (27, 19, 0, 18), (6, 20, 7, 19)], (6, 20, 7, 19)], ['o9_14831', [(5, 1, 6, 0), (1, 17, 2, 16), (3, 10, 4, 11), (19, 5, 20, 4), (15, 6, 16, 7), (17, 9, 18, 8), (9, 19, 10, 18), (11, 20, 12, 21), (21, 12, 22, 13), (13, 22, 14, 23), (23, 14, 0, 15), (2, 8, 3, 7)], (2, 8, 3, 7)], ['o9_14974', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (21, 4, 22, 5), (5, 20, 6, 21), (19, 6, 20, 7), (7, 18, 8, 19), (27, 9, 0, 8), (9, 27, 10, 26), (11, 17, 12, 16), (25, 14, 26, 15), (15, 24, 16, 25), (17, 23, 18, 22), (10, 23, 11, 24)], (10, 23, 11, 24)], ['o9_15506', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (15, 4, 16, 5), (5, 16, 6, 17), (11, 7, 12, 6), (7, 21, 8, 20), (21, 9, 22, 8), (23, 13, 24, 12), (13, 19, 14, 18), (25, 14, 0, 15), (19, 23, 20, 22), (24, 18, 25, 17)], (24, 18, 25, 17)], ['o9_15633', [(3, 1, 4, 0), (1, 14, 2, 15), (13, 2, 14, 3), (9, 5, 10, 4), (5, 18, 6, 19), (21, 6, 22, 7), (7, 17, 8, 16), (19, 11, 20, 10), (11, 21, 12, 20), (25, 13, 0, 12), (15, 25, 16, 24), (17, 22, 18, 23), (8, 23, 9, 24)], (8, 23, 9, 24)], ['o9_15997', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (17, 5, 18, 4), (5, 19, 6, 18), (23, 6, 24, 7), (7, 26, 8, 27), (25, 8, 26, 9), (9, 24, 10, 25), (11, 20, 12, 21), (21, 14, 22, 15), (19, 17, 20, 16), (27, 22, 0, 23), (10, 16, 11, 15)], (10, 16, 11, 15)], ['o9_16065', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (13, 5, 14, 4), (5, 18, 6, 19), (19, 6, 20, 7), (7, 25, 8, 24), (25, 9, 26, 8), (9, 17, 10, 16), (27, 15, 0, 14), (15, 23, 16, 22), (17, 20, 18, 21), (23, 27, 24, 26), (10, 21, 11, 22)], (10, 21, 11, 22)], ['o9_16141', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (19, 4, 20, 5), (5, 18, 6, 19), (15, 6, 16, 7), (13, 9, 14, 8), (11, 24, 12, 25), (23, 12, 24, 13), (27, 14, 0, 15), (21, 17, 22, 16), (17, 21, 18, 20), (25, 23, 26, 22), (26, 8, 27, 7)], (26, 8, 27, 7)], ['o9_16157', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (19, 5, 20, 4), (5, 22, 6, 23), (21, 6, 22, 7), (7, 12, 8, 13), (17, 8, 18, 9), (13, 27, 14, 26), (25, 15, 26, 14), (15, 25, 16, 24), (27, 17, 0, 16), (23, 21, 24, 20), (18, 12, 19, 11)], (18, 12, 19, 11)], ['o9_16181', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (19, 4, 20, 5), (5, 18, 6, 19), (7, 26, 8, 27), (21, 9, 22, 8), (9, 23, 10, 22), (13, 11, 14, 10), (27, 14, 0, 15), (25, 17, 26, 16), (17, 25, 18, 24), (23, 21, 24, 20), (6, 16, 7, 15)], (6, 16, 7, 15)], ['o9_16319', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (15, 5, 16, 4), (5, 26, 6, 27), (19, 7, 20, 6), (7, 21, 8, 20), (17, 8, 18, 9), (11, 24, 12, 25), (23, 12, 24, 13), (13, 22, 14, 23), (25, 14, 26, 15), (21, 19, 22, 18), (16, 27, 17, 0)], (16, 27, 17, 0)], ['o9_16356', [(1, 11, 2, 10), (11, 3, 12, 2), (3, 22, 4, 23), (19, 4, 20, 5), (5, 20, 6, 21), (21, 6, 22, 7), (17, 9, 18, 8), (9, 17, 10, 16), (23, 13, 24, 12), (13, 27, 14, 26), (25, 15, 26, 14), (15, 25, 16, 24), (27, 19, 0, 18), (0, 7, 1, 8)], (0, 7, 1, 8)], ['o9_16527', [(1, 11, 2, 10), (11, 3, 12, 2), (3, 19, 4, 18), (17, 5, 18, 4), (5, 17, 6, 16), (19, 7, 20, 6), (21, 8, 22, 9), (9, 22, 10, 23), (23, 12, 24, 13), (13, 26, 14, 27), (25, 14, 26, 15), (15, 24, 16, 25), (27, 20, 0, 21), (0, 7, 1, 8)], (0, 7, 1, 8)], ['o9_16642', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (13, 5, 14, 4), (15, 6, 16, 7), (7, 18, 8, 19), (19, 8, 20, 9), (11, 25, 12, 24), (23, 13, 24, 12), (27, 15, 0, 14), (21, 16, 22, 17), (17, 20, 18, 21), (25, 22, 26, 23), (26, 6, 27, 5)], (26, 6, 27, 5)], ['o9_16748', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (13, 5, 14, 4), (5, 23, 6, 22), (15, 6, 16, 7), (7, 24, 8, 25), (19, 9, 20, 8), (9, 19, 10, 18), (17, 11, 18, 10), (23, 17, 24, 16), (25, 20, 0, 21), (14, 22, 15, 21)], (14, 22, 15, 21)], ['o9_16920', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (17, 5, 18, 4), (5, 22, 6, 23), (15, 6, 16, 7), (7, 14, 8, 15), (19, 8, 20, 9), (9, 13, 10, 12), (13, 20, 14, 21), (23, 19, 0, 18), (16, 22, 17, 21)], (16, 22, 17, 21)], ['o9_17450', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (21, 5, 22, 4), (5, 23, 6, 22), (15, 7, 16, 6), (19, 8, 20, 9), (9, 18, 10, 19), (17, 10, 18, 11), (13, 20, 14, 21), (23, 14, 24, 15), (25, 17, 0, 16), (24, 8, 25, 7)], (24, 8, 25, 7)], ['o9_18007', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (17, 5, 18, 4), (5, 24, 6, 25), (23, 6, 24, 7), (7, 17, 8, 16), (21, 9, 22, 8), (9, 21, 10, 20), (13, 11, 14, 10), (19, 15, 20, 14), (15, 23, 16, 22), (18, 25, 19, 0)], (18, 25, 19, 0)], ['o9_18209', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (5, 26, 6, 27), (25, 6, 26, 7), (7, 24, 8, 25), (21, 8, 22, 9), (9, 20, 10, 21), (19, 10, 20, 11), (11, 15, 12, 14), (15, 22, 16, 23), (23, 16, 24, 17), (27, 19, 0, 18), (4, 17, 5, 18)], (4, 17, 5, 18)], ['o9_18633', [(3, 1, 4, 0), (1, 15, 2, 14), (15, 3, 16, 2), (11, 4, 12, 5), (5, 19, 6, 18), (21, 7, 22, 6), (7, 16, 8, 17), (25, 8, 0, 9), (9, 24, 10, 25), (23, 10, 24, 11), (19, 12, 20, 13), (13, 20, 14, 21), (22, 17, 23, 18)], (22, 17, 23, 18)], ['o9_18813', [(3, 1, 4, 0), (1, 14, 2, 15), (13, 2, 14, 3), (9, 5, 10, 4), (5, 21, 6, 20), (17, 7, 18, 6), (15, 8, 16, 9), (19, 10, 20, 11), (11, 18, 12, 19), (23, 13, 0, 12), (21, 16, 22, 17), (22, 8, 23, 7)], (22, 8, 23, 7)], ['o9_19130', [(5, 1, 6, 0), (1, 20, 2, 21), (3, 12, 4, 13), (15, 4, 16, 5), (21, 7, 22, 6), (17, 8, 18, 9), (9, 18, 10, 19), (19, 10, 20, 11), (11, 16, 12, 17), (13, 23, 14, 22), (23, 15, 0, 14), (2, 8, 3, 7)], (2, 8, 3, 7)], ['o9_20219', [(1, 19, 2, 18), (9, 3, 10, 2), (3, 9, 4, 8), (19, 5, 20, 4), (11, 7, 12, 6), (7, 17, 8, 16), (17, 11, 18, 10), (21, 12, 22, 13), (13, 24, 14, 25), (23, 14, 24, 15), (15, 22, 16, 23), (25, 20, 0, 21), (0, 5, 1, 6)], (0, 5, 1, 6)], ['o9_21893', [(5, 1, 6, 0), (1, 9, 2, 8), (3, 12, 4, 13), (21, 5, 22, 4), (15, 6, 16, 7), (7, 16, 8, 17), (9, 19, 10, 18), (19, 11, 20, 10), (11, 21, 12, 20), (13, 22, 14, 23), (23, 14, 0, 15), (2, 18, 3, 17)], (2, 18, 3, 17)], ['o9_21918', [(1, 19, 2, 18), (9, 2, 10, 3), (3, 14, 4, 15), (13, 4, 14, 5), (5, 12, 6, 13), (21, 7, 22, 6), (17, 8, 18, 9), (19, 11, 20, 10), (11, 21, 12, 20), (15, 22, 16, 23), (23, 16, 0, 17), (0, 7, 1, 8)], (0, 7, 1, 8)], ['o9_22129', [(1, 18, 2, 19), (19, 2, 20, 3), (3, 20, 4, 21), (13, 4, 14, 5), (5, 14, 6, 15), (17, 6, 18, 7), (21, 9, 22, 8), (9, 23, 10, 22), (23, 11, 24, 10), (11, 25, 12, 24), (25, 13, 26, 12), (15, 27, 16, 26), (27, 17, 0, 16), (0, 7, 1, 8)], (0, 7, 1, 8)], ['o9_22477', [(7, 1, 8, 0), (9, 2, 10, 3), (3, 19, 4, 18), (13, 5, 14, 4), (5, 15, 6, 14), (23, 7, 0, 6), (17, 9, 18, 8), (19, 10, 20, 11), (11, 20, 12, 21), (21, 12, 22, 13), (15, 22, 16, 23), (16, 2, 17, 1)], (16, 2, 17, 1)], ['o9_22607', [(7, 1, 8, 0), (1, 9, 2, 8), (13, 3, 14, 2), (19, 4, 20, 5), (5, 18, 6, 19), (17, 6, 18, 7), (9, 20, 10, 21), (21, 10, 22, 11), (11, 22, 12, 23), (23, 12, 24, 13), (25, 15, 26, 14), (15, 27, 16, 26), (27, 17, 0, 16), (24, 4, 25, 3)], (24, 4, 25, 3)], ['o9_22663', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (13, 4, 14, 5), (5, 20, 6, 21), (19, 6, 20, 7), (7, 14, 8, 15), (15, 10, 16, 11), (11, 23, 12, 22), (23, 16, 0, 17), (21, 19, 22, 18), (12, 17, 13, 18)], (12, 17, 13, 18)], ['o9_22698', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (17, 4, 18, 5), (5, 24, 6, 25), (11, 6, 12, 7), (7, 14, 8, 15), (21, 12, 22, 13), (13, 20, 14, 21), (25, 16, 0, 17), (23, 19, 24, 18), (19, 23, 20, 22), (10, 16, 11, 15)], (10, 16, 11, 15)], ['o9_22925', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (13, 5, 14, 4), (5, 19, 6, 18), (17, 7, 18, 6), (7, 22, 8, 23), (11, 21, 12, 20), (19, 13, 20, 12), (23, 15, 0, 14), (21, 16, 22, 17), (8, 16, 9, 15)], (8, 16, 9, 15)], ['o9_23023', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (11, 5, 12, 4), (5, 16, 6, 17), (7, 24, 8, 25), (19, 8, 20, 9), (17, 13, 18, 12), (21, 14, 22, 15), (15, 22, 16, 23), (25, 19, 0, 18), (23, 20, 24, 21), (6, 14, 7, 13)], (6, 14, 7, 13)], ['o9_23263', [(5, 1, 6, 0), (1, 19, 2, 18), (19, 3, 20, 2), (3, 13, 4, 12), (13, 5, 14, 4), (7, 24, 8, 25), (21, 8, 22, 9), (9, 22, 10, 23), (23, 10, 24, 11), (11, 20, 12, 21), (25, 14, 26, 15), (15, 26, 16, 27), (27, 16, 0, 17), (6, 18, 7, 17)], (6, 18, 7, 17)], ['o9_23660', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (19, 4, 20, 5), (5, 18, 6, 19), (25, 7, 26, 6), (7, 25, 8, 24), (23, 9, 24, 8), (21, 10, 22, 11), (11, 20, 12, 21), (27, 14, 0, 15), (15, 26, 16, 27), (17, 23, 18, 22), (16, 9, 17, 10)], (16, 9, 17, 10)], ['o9_23955', [(7, 1, 8, 0), (1, 23, 2, 22), (3, 18, 4, 19), (17, 4, 18, 5), (5, 16, 6, 17), (25, 7, 26, 6), (27, 9, 0, 8), (9, 27, 10, 26), (19, 10, 20, 11), (11, 20, 12, 21), (21, 12, 22, 13), (23, 15, 24, 14), (15, 25, 16, 24), (2, 14, 3, 13)], (2, 14, 3, 13)], ['o9_23961', [(3, 1, 4, 0), (1, 15, 2, 14), (15, 3, 16, 2), (21, 4, 22, 5), (5, 20, 6, 21), (27, 7, 0, 6), (7, 27, 8, 26), (19, 8, 20, 9), (9, 24, 10, 25), (23, 10, 24, 11), (11, 22, 12, 23), (17, 13, 18, 12), (13, 17, 14, 16), (18, 26, 19, 25)], (18, 26, 19, 25)], ['o9_23977', [(5, 1, 6, 0), (1, 18, 2, 19), (9, 3, 10, 2), (3, 9, 4, 8), (17, 4, 18, 5), (7, 14, 8, 15), (19, 10, 20, 11), (11, 22, 12, 23), (21, 12, 22, 13), (13, 20, 14, 21), (15, 25, 16, 24), (25, 17, 0, 16), (6, 23, 7, 24)], (6, 23, 7, 24)], ['o9_24149', [(7, 1, 8, 0), (1, 11, 2, 10), (13, 3, 14, 2), (17, 4, 18, 5), (5, 18, 6, 19), (19, 6, 20, 7), (27, 9, 0, 8), (9, 27, 10, 26), (11, 20, 12, 21), (21, 12, 22, 13), (23, 15, 24, 14), (15, 25, 16, 24), (25, 17, 26, 16), (22, 4, 23, 3)], (22, 4, 23, 3)], ['o9_24183', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (11, 5, 12, 4), (5, 22, 6, 23), (21, 6, 22, 7), (7, 17, 8, 16), (17, 9, 18, 8), (27, 13, 0, 12), (13, 19, 14, 18), (25, 14, 26, 15), (15, 24, 16, 25), (23, 21, 24, 20), (26, 19, 27, 20)], (26, 19, 27, 20)], ['o9_24534', [(7, 1, 8, 0), (1, 23, 2, 22), (3, 16, 4, 17), (17, 4, 18, 5), (5, 18, 6, 19), (27, 7, 0, 6), (19, 8, 20, 9), (9, 20, 10, 21), (21, 10, 22, 11), (23, 13, 24, 12), (13, 25, 14, 24), (25, 15, 26, 14), (15, 27, 16, 26), (2, 12, 3, 11)], (2, 12, 3, 11)], ['o9_24592', [(7, 1, 8, 0), (1, 17, 2, 16), (11, 2, 12, 3), (3, 12, 4, 13), (19, 5, 20, 4), (5, 19, 6, 18), (17, 7, 18, 6), (15, 8, 16, 9), (9, 22, 10, 23), (21, 10, 22, 11), (13, 20, 14, 21), (14, 0, 15, 23)], (14, 0, 15, 23)], ['o9_24886', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (15, 5, 16, 4), (5, 17, 6, 16), (7, 22, 8, 23), (19, 9, 20, 8), (9, 21, 10, 20), (23, 10, 24, 11), (13, 24, 14, 25), (25, 14, 26, 15), (17, 26, 18, 27), (21, 19, 22, 18), (6, 27, 7, 0)], (6, 27, 7, 0)], ['o9_24889', [(3, 1, 4, 0), (1, 15, 2, 14), (15, 3, 16, 2), (19, 4, 20, 5), (5, 18, 6, 19), (27, 7, 0, 6), (7, 27, 8, 26), (17, 8, 18, 9), (9, 24, 10, 25), (21, 10, 22, 11), (11, 22, 12, 23), (23, 12, 24, 13), (13, 20, 14, 21), (16, 26, 17, 25)], (16, 26, 17, 25)], ['o9_25595', [(1, 21, 2, 20), (21, 3, 22, 2), (3, 13, 4, 12), (13, 5, 14, 4), (5, 15, 6, 14), (15, 7, 16, 6), (19, 8, 20, 9), (9, 22, 10, 23), (23, 10, 24, 11), (11, 24, 12, 25), (25, 16, 26, 17), (17, 26, 18, 27), (27, 18, 0, 19), (0, 7, 1, 8)], (0, 7, 1, 8)], ['o9_26604', [(3, 1, 4, 0), (1, 14, 2, 15), (13, 2, 14, 3), (5, 11, 6, 10), (21, 7, 22, 6), (7, 16, 8, 17), (23, 8, 0, 9), (9, 19, 10, 18), (11, 20, 12, 21), (15, 13, 16, 12), (17, 23, 18, 22), (4, 20, 5, 19)], (4, 20, 5, 19)], ['o9_26791', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (5, 24, 6, 25), (23, 6, 24, 7), (7, 17, 8, 16), (13, 9, 14, 8), (9, 20, 10, 21), (19, 10, 20, 11), (21, 15, 22, 14), (15, 23, 16, 22), (25, 19, 0, 18), (4, 17, 5, 18)], (4, 17, 5, 18)], ['o9_27155', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (15, 5, 16, 4), (5, 22, 6, 23), (13, 6, 14, 7), (7, 12, 8, 13), (19, 8, 20, 9), (21, 14, 22, 15), (23, 17, 24, 16), (17, 25, 18, 24), (25, 19, 0, 18), (20, 12, 21, 11)], (20, 12, 21, 11)], ['o9_27261', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (17, 4, 18, 5), (5, 16, 6, 17), (21, 6, 22, 7), (7, 22, 8, 23), (15, 8, 16, 9), (9, 18, 10, 19), (25, 12, 0, 13), (13, 21, 14, 20), (23, 15, 24, 14), (24, 19, 25, 20)], (24, 19, 25, 20)], ['o9_27392', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (5, 17, 6, 16), (21, 7, 22, 6), (7, 21, 8, 20), (19, 9, 20, 8), (11, 19, 12, 18), (17, 13, 18, 12), (25, 15, 26, 14), (15, 25, 16, 24), (27, 22, 0, 23), (23, 26, 24, 27), (4, 13, 5, 14)], (4, 13, 5, 14)], ['o9_27480', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (5, 24, 6, 25), (19, 6, 20, 7), (7, 20, 8, 21), (21, 8, 22, 9), (9, 13, 10, 12), (13, 22, 14, 23), (23, 14, 24, 15), (25, 17, 26, 16), (17, 27, 18, 26), (27, 19, 0, 18), (4, 15, 5, 16)], (4, 15, 5, 16)], ['o9_27737', [(3, 1, 4, 0), (1, 15, 2, 14), (15, 3, 16, 2), (21, 4, 22, 5), (5, 20, 6, 21), (19, 6, 20, 7), (7, 18, 8, 19), (27, 9, 0, 8), (9, 27, 10, 26), (25, 11, 26, 10), (11, 17, 12, 16), (23, 12, 24, 13), (13, 22, 14, 23), (24, 17, 25, 18)], (24, 17, 25, 18)], ['o9_28113', [(3, 1, 4, 0), (1, 14, 2, 15), (13, 2, 14, 3), (19, 5, 20, 4), (5, 26, 6, 27), (25, 6, 26, 7), (7, 19, 8, 18), (17, 9, 18, 8), (9, 17, 10, 16), (15, 11, 16, 10), (11, 23, 12, 22), (23, 13, 24, 12), (21, 25, 22, 24), (20, 27, 21, 0)], (20, 27, 21, 0)], ['o9_28153', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (15, 4, 16, 5), (5, 16, 6, 17), (23, 7, 24, 6), (13, 9, 14, 8), (11, 21, 12, 20), (21, 13, 22, 12), (25, 14, 0, 15), (17, 24, 18, 25), (19, 23, 20, 22), (18, 7, 19, 8)], (18, 7, 19, 8)], ['o9_28529', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (11, 5, 12, 4), (5, 22, 6, 23), (21, 6, 22, 7), (7, 15, 8, 14), (15, 9, 16, 8), (13, 19, 14, 18), (27, 16, 0, 17), (17, 26, 18, 27), (19, 25, 20, 24), (23, 21, 24, 20), (12, 25, 13, 26)], (12, 25, 13, 26)], ['o9_28592', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (5, 24, 6, 25), (13, 7, 14, 6), (7, 13, 8, 12), (21, 8, 22, 9), (9, 20, 10, 21), (27, 14, 0, 15), (15, 26, 16, 27), (25, 16, 26, 17), (23, 19, 24, 18), (19, 23, 20, 22), (4, 18, 5, 17)], (4, 18, 5, 17)], ['o9_28746', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (5, 24, 6, 25), (23, 6, 24, 7), (7, 20, 8, 21), (13, 9, 14, 8), (9, 17, 10, 16), (17, 11, 18, 10), (21, 14, 22, 15), (15, 22, 16, 23), (25, 18, 0, 19), (4, 20, 5, 19)], (4, 20, 5, 19)], ['o9_28810', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (13, 5, 14, 4), (23, 7, 24, 6), (7, 16, 8, 17), (15, 8, 16, 9), (11, 20, 12, 21), (21, 12, 22, 13), (25, 15, 0, 14), (17, 25, 18, 24), (19, 22, 20, 23), (18, 5, 19, 6)], (18, 5, 19, 6)], ['o9_29246', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (19, 4, 20, 5), (5, 18, 6, 19), (17, 6, 18, 7), (7, 22, 8, 23), (11, 21, 12, 20), (21, 13, 22, 12), (23, 14, 24, 15), (15, 27, 16, 26), (25, 17, 26, 16), (27, 24, 0, 25), (8, 13, 9, 14)], (8, 13, 9, 14)], ['o9_29436', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (19, 4, 20, 5), (5, 18, 6, 19), (15, 6, 16, 7), (21, 8, 22, 9), (9, 22, 10, 23), (23, 10, 24, 11), (11, 20, 12, 21), (27, 14, 0, 15), (25, 17, 26, 16), (17, 25, 18, 24), (26, 8, 27, 7)], (26, 8, 27, 7)], ['o9_29529', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (15, 5, 16, 4), (5, 20, 6, 21), (13, 6, 14, 7), (7, 19, 8, 18), (23, 10, 0, 11), (21, 13, 22, 12), (19, 14, 20, 15), (17, 22, 18, 23), (16, 12, 17, 11)], (16, 12, 17, 11)], ['o9_30375', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (13, 4, 14, 5), (5, 24, 6, 25), (7, 16, 8, 17), (19, 11, 20, 10), (11, 19, 12, 18), (25, 12, 0, 13), (23, 15, 24, 14), (15, 23, 16, 22), (17, 20, 18, 21), (6, 22, 7, 21)], (6, 22, 7, 21)], ['o9_30721', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (19, 4, 20, 5), (5, 20, 6, 21), (11, 7, 12, 6), (7, 16, 8, 17), (15, 8, 16, 9), (21, 12, 22, 13), (23, 15, 0, 14), (17, 23, 18, 22), (18, 14, 19, 13)], (18, 14, 19, 13)], ['o9_30790', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (5, 22, 6, 23), (25, 6, 26, 7), (7, 26, 8, 27), (19, 8, 20, 9), (9, 18, 10, 19), (27, 12, 0, 13), (13, 24, 14, 25), (23, 14, 24, 15), (21, 17, 22, 16), (17, 21, 18, 20), (4, 16, 5, 15)], (4, 16, 5, 15)], ['o9_31165', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (19, 4, 20, 5), (5, 18, 6, 19), (17, 6, 18, 7), (15, 9, 16, 8), (9, 15, 10, 14), (23, 10, 24, 11), (11, 22, 12, 23), (27, 16, 0, 17), (25, 21, 26, 20), (21, 25, 22, 24), (26, 8, 27, 7)], (26, 8, 27, 7)], ['o9_32257', [(5, 1, 6, 0), (1, 17, 2, 16), (9, 2, 10, 3), (3, 12, 4, 13), (23, 7, 0, 6), (7, 23, 8, 22), (15, 8, 16, 9), (17, 11, 18, 10), (11, 19, 12, 18), (13, 20, 14, 21), (21, 14, 22, 15), (4, 19, 5, 20)], (4, 19, 5, 20)], ['o9_33526', [(5, 1, 6, 0), (1, 10, 2, 11), (9, 2, 10, 3), (3, 15, 4, 14), (15, 5, 16, 4), (17, 6, 18, 7), (7, 12, 8, 13), (19, 9, 20, 8), (13, 20, 14, 21), (21, 16, 0, 17), (18, 11, 19, 12)], (18, 11, 19, 12)], ['o9_33585', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (13, 5, 14, 4), (5, 22, 6, 23), (21, 6, 22, 7), (7, 17, 8, 16), (19, 9, 20, 8), (11, 19, 12, 18), (17, 13, 18, 12), (23, 15, 0, 14), (20, 15, 21, 16)], (20, 15, 21, 16)], ['o9_34403', [(1, 12, 2, 13), (17, 2, 18, 3), (3, 9, 4, 8), (21, 5, 22, 4), (13, 7, 14, 6), (7, 17, 8, 16), (9, 20, 10, 21), (19, 10, 20, 11), (11, 18, 12, 19), (23, 15, 0, 14), (15, 23, 16, 22), (0, 5, 1, 6)], (0, 5, 1, 6)], ['o9_35320', [(7, 1, 8, 0), (1, 17, 2, 16), (9, 2, 10, 3), (11, 5, 12, 4), (5, 19, 6, 18), (17, 7, 18, 6), (15, 8, 16, 9), (19, 11, 20, 10), (23, 12, 0, 13), (13, 22, 14, 23), (21, 14, 22, 15), (20, 4, 21, 3)], (20, 4, 21, 3)], ['o9_35549', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (13, 5, 14, 4), (5, 16, 6, 17), (15, 6, 16, 7), (25, 9, 26, 8), (11, 22, 12, 23), (23, 12, 24, 13), (17, 15, 18, 14), (27, 19, 0, 18), (19, 27, 20, 26), (21, 24, 22, 25), (20, 7, 21, 8)], (20, 7, 21, 8)], ['o9_35682', [(1, 18, 2, 19), (11, 2, 12, 3), (3, 12, 4, 13), (17, 4, 18, 5), (21, 7, 22, 6), (7, 14, 8, 15), (13, 8, 14, 9), (9, 21, 10, 20), (19, 11, 20, 10), (15, 23, 16, 22), (23, 17, 0, 16), (0, 5, 1, 6)], (0, 5, 1, 6)], ['o9_35736', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (21, 4, 22, 5), (5, 22, 6, 23), (15, 7, 16, 6), (7, 15, 8, 14), (27, 10, 0, 11), (19, 12, 20, 13), (13, 16, 14, 17), (17, 27, 18, 26), (25, 19, 26, 18), (23, 20, 24, 21), (24, 12, 25, 11)], (24, 12, 25, 11)], ['o9_35772', [(5, 1, 6, 0), (1, 17, 2, 16), (9, 2, 10, 3), (3, 10, 4, 11), (17, 5, 18, 4), (23, 7, 0, 6), (7, 21, 8, 20), (11, 18, 12, 19), (19, 12, 20, 13), (21, 15, 22, 14), (15, 23, 16, 22), (8, 14, 9, 13)], (8, 14, 9, 13)], ['o9_37941', [(1, 19, 2, 18), (9, 2, 10, 3), (3, 12, 4, 13), (21, 5, 22, 4), (17, 6, 18, 7), (7, 14, 8, 15), (13, 8, 14, 9), (19, 11, 20, 10), (11, 21, 12, 20), (15, 22, 16, 23), (23, 16, 0, 17), (0, 5, 1, 6)], (0, 5, 1, 6)], ['o9_39394', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (17, 5, 18, 4), (5, 11, 6, 10), (19, 6, 20, 7), (7, 22, 8, 23), (23, 8, 0, 9), (11, 17, 12, 16), (21, 14, 22, 15), (15, 20, 16, 21), (18, 10, 19, 9)], (18, 10, 19, 9)], ['s086', [(5, 1, 6, 0), (13, 3, 14, 2), (3, 13, 4, 12), (17, 5, 18, 4), (15, 6, 16, 7), (7, 14, 8, 15), (21, 9, 0, 8), (9, 21, 10, 20), (19, 11, 20, 10), (11, 19, 12, 18), (16, 1, 17, 2)], (16, 1, 17, 2)], ['s294', [(5, 1, 6, 0), (1, 15, 2, 14), (7, 2, 8, 3), (9, 5, 10, 4), (13, 6, 14, 7), (15, 9, 16, 8), (19, 10, 0, 11), (11, 18, 12, 19), (17, 12, 18, 13), (16, 4, 17, 3)], (16, 4, 17, 3)], ['s301', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (15, 4, 16, 5), (5, 16, 6, 17), (17, 6, 18, 7), (13, 9, 14, 8), (11, 19, 12, 18), (19, 13, 20, 12), (21, 14, 0, 15), (20, 8, 21, 7)], (20, 8, 21, 7)], ['s308', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (13, 5, 14, 4), (17, 6, 18, 7), (7, 16, 8, 17), (15, 8, 16, 9), (11, 18, 12, 19), (19, 12, 20, 13), (21, 15, 0, 14), (20, 6, 21, 5)], (20, 6, 21, 5)], ['s336', [(5, 1, 6, 0), (1, 16, 2, 17), (7, 2, 8, 3), (3, 10, 4, 11), (13, 4, 14, 5), (15, 8, 16, 9), (9, 14, 10, 15), (11, 19, 12, 18), (19, 13, 0, 12), (6, 17, 7, 18)], (6, 17, 7, 18)], ['s346', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (17, 4, 18, 5), (5, 18, 6, 19), (13, 6, 14, 7), (7, 14, 8, 15), (21, 10, 0, 11), (19, 13, 20, 12), (15, 21, 16, 20), (16, 11, 17, 12)], (16, 11, 17, 12)], ['s367', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (5, 20, 6, 21), (15, 7, 16, 6), (7, 15, 8, 14), (13, 9, 14, 8), (11, 19, 12, 18), (19, 13, 20, 12), (21, 16, 0, 17), (4, 18, 5, 17)], (4, 18, 5, 17)], ['s407', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (13, 5, 14, 4), (15, 6, 16, 7), (7, 21, 8, 20), (21, 9, 0, 8), (9, 16, 10, 17), (17, 10, 18, 11), (19, 15, 20, 14), (18, 6, 19, 5)], (18, 6, 19, 5)], ['s582', [(1, 10, 2, 11), (9, 2, 10, 3), (3, 16, 4, 17), (15, 4, 16, 5), (11, 7, 12, 6), (7, 18, 8, 19), (17, 8, 18, 9), (21, 13, 0, 12), (13, 21, 14, 20), (19, 15, 20, 14), (0, 5, 1, 6)], (0, 5, 1, 6)], ['s665', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (13, 5, 14, 4), (5, 13, 6, 12), (17, 6, 18, 7), (7, 18, 8, 19), (11, 21, 12, 20), (21, 15, 0, 14), (19, 16, 20, 17), (10, 15, 11, 16)], (10, 15, 11, 16)], ['s684', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (13, 5, 14, 4), (5, 18, 6, 19), (11, 6, 12, 7), (7, 17, 8, 16), (17, 12, 18, 13), (19, 15, 0, 14), (10, 15, 11, 16)], (10, 15, 11, 16)], ['s800', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (5, 20, 6, 21), (19, 6, 20, 7), (7, 15, 8, 14), (13, 9, 14, 8), (9, 18, 10, 19), (17, 10, 18, 11), (21, 17, 0, 16), (4, 15, 5, 16)], (4, 15, 5, 16)], ['t00324', [(5, 1, 6, 0), (15, 3, 16, 2), (3, 15, 4, 14), (19, 5, 20, 4), (17, 6, 18, 7), (7, 16, 8, 17), (25, 9, 0, 8), (9, 25, 10, 24), (23, 11, 24, 10), (11, 23, 12, 22), (21, 13, 22, 12), (13, 21, 14, 20), (18, 1, 19, 2)], (18, 1, 19, 2)], ['t00729', [(3, 1, 4, 0), (1, 14, 2, 15), (13, 2, 14, 3), (21, 4, 22, 5), (5, 20, 6, 21), (7, 19, 8, 18), (23, 8, 24, 9), (9, 22, 10, 23), (17, 11, 18, 10), (11, 17, 12, 16), (15, 13, 16, 12), (19, 25, 20, 24), (6, 25, 7, 0)], (6, 25, 7, 0)], ['t00787', [(3, 1, 4, 0), (1, 14, 2, 15), (13, 2, 14, 3), (15, 5, 16, 4), (5, 17, 6, 16), (17, 7, 18, 6), (7, 22, 8, 23), (23, 8, 24, 9), (25, 11, 0, 10), (11, 20, 12, 21), (21, 12, 22, 13), (19, 24, 20, 25), (18, 9, 19, 10)], (18, 9, 19, 10)], ['t00826', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (19, 4, 20, 5), (5, 22, 6, 23), (17, 7, 18, 6), (7, 17, 8, 16), (21, 8, 22, 9), (9, 20, 10, 21), (13, 11, 14, 10), (25, 14, 0, 15), (15, 24, 16, 25), (18, 24, 19, 23)], (18, 24, 19, 23)], ['t00855', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (13, 5, 14, 4), (5, 20, 6, 21), (17, 6, 18, 7), (7, 23, 8, 22), (23, 9, 24, 8), (9, 18, 10, 19), (19, 10, 20, 11), (25, 15, 0, 14), (15, 25, 16, 24), (16, 21, 17, 22)], (16, 21, 17, 22)], ['t01033', [(5, 1, 6, 0), (1, 14, 2, 15), (13, 2, 14, 3), (3, 12, 4, 13), (11, 4, 12, 5), (19, 6, 20, 7), (7, 20, 8, 21), (21, 8, 22, 9), (17, 11, 18, 10), (15, 23, 16, 22), (23, 17, 24, 16), (25, 18, 0, 19), (24, 10, 25, 9)], (24, 10, 25, 9)], ['t01037', [(5, 1, 6, 0), (1, 14, 2, 15), (13, 2, 14, 3), (3, 12, 4, 13), (11, 4, 12, 5), (17, 7, 18, 6), (21, 8, 22, 9), (9, 20, 10, 21), (19, 10, 20, 11), (15, 22, 16, 23), (23, 16, 24, 17), (25, 19, 0, 18), (24, 8, 25, 7)], (24, 8, 25, 7)], ['t01125', [(5, 1, 6, 0), (1, 11, 2, 10), (11, 3, 12, 2), (3, 13, 4, 12), (13, 5, 14, 4), (21, 6, 22, 7), (7, 22, 8, 23), (17, 8, 18, 9), (9, 18, 10, 19), (25, 14, 0, 15), (23, 17, 24, 16), (19, 25, 20, 24), (20, 15, 21, 16)], (20, 15, 21, 16)], ['t01268', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (13, 5, 14, 4), (5, 18, 6, 19), (17, 6, 18, 7), (7, 22, 8, 23), (21, 8, 22, 9), (9, 13, 10, 12), (25, 15, 0, 14), (23, 16, 24, 17), (19, 24, 20, 25), (20, 16, 21, 15)], (20, 16, 21, 15)], ['t01318', [(5, 1, 6, 0), (1, 17, 2, 16), (7, 2, 8, 3), (9, 5, 10, 4), (15, 6, 16, 7), (17, 9, 18, 8), (21, 10, 22, 11), (11, 22, 12, 23), (23, 12, 0, 13), (13, 20, 14, 21), (19, 14, 20, 15), (18, 4, 19, 3)], (18, 4, 19, 3)], ['t01368', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (5, 20, 6, 21), (13, 6, 14, 7), (7, 19, 8, 18), (23, 10, 0, 11), (11, 22, 12, 23), (17, 13, 18, 12), (19, 14, 20, 15), (21, 17, 22, 16), (4, 15, 5, 16)], (4, 15, 5, 16)], ['t01422', [(5, 1, 6, 0), (1, 20, 2, 21), (7, 2, 8, 3), (3, 12, 4, 13), (15, 4, 16, 5), (19, 8, 20, 9), (9, 18, 10, 19), (17, 10, 18, 11), (11, 16, 12, 17), (13, 23, 14, 22), (23, 15, 0, 14), (6, 21, 7, 22)], (6, 21, 7, 22)], ['t01440', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (5, 24, 6, 25), (19, 7, 20, 6), (7, 19, 8, 18), (17, 9, 18, 8), (9, 17, 10, 16), (15, 11, 16, 10), (13, 23, 14, 22), (23, 15, 24, 14), (25, 20, 0, 21), (4, 22, 5, 21)], (4, 22, 5, 21)], ['t01636', [(5, 1, 6, 0), (11, 2, 12, 3), (3, 12, 4, 13), (15, 4, 16, 5), (17, 7, 18, 6), (7, 22, 8, 23), (19, 8, 20, 9), (9, 20, 10, 21), (21, 10, 22, 11), (13, 25, 14, 24), (25, 15, 0, 14), (23, 19, 24, 18), (16, 2, 17, 1)], (16, 2, 17, 1)], ['t01690', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (15, 4, 16, 5), (5, 24, 6, 25), (7, 23, 8, 22), (17, 8, 18, 9), (9, 16, 10, 17), (21, 11, 22, 10), (11, 21, 12, 20), (25, 14, 0, 15), (23, 19, 24, 18), (6, 19, 7, 20)], (6, 19, 7, 20)], ['t01757', [(3, 1, 4, 0), (1, 14, 2, 15), (13, 2, 14, 3), (19, 4, 20, 5), (17, 7, 18, 6), (7, 17, 8, 16), (23, 8, 24, 9), (9, 22, 10, 23), (21, 10, 22, 11), (11, 20, 12, 21), (15, 13, 16, 12), (25, 18, 0, 19), (24, 6, 25, 5)], (24, 6, 25, 5)], ['t01834', [(3, 1, 4, 0), (1, 14, 2, 15), (13, 2, 14, 3), (15, 5, 16, 4), (5, 22, 6, 23), (7, 25, 8, 24), (25, 9, 0, 8), (9, 18, 10, 19), (19, 10, 20, 11), (11, 20, 12, 21), (21, 12, 22, 13), (23, 17, 24, 16), (6, 18, 7, 17)], (6, 18, 7, 17)], ['t01863', [(5, 1, 6, 0), (1, 19, 2, 18), (9, 2, 10, 3), (3, 10, 4, 11), (15, 6, 16, 7), (7, 16, 8, 17), (17, 8, 18, 9), (11, 25, 12, 24), (21, 13, 22, 12), (13, 23, 14, 22), (23, 15, 24, 14), (25, 20, 0, 21), (4, 19, 5, 20)], (4, 19, 5, 20)], ['t02099', [(5, 1, 6, 0), (7, 2, 8, 3), (3, 20, 4, 21), (11, 5, 12, 4), (15, 6, 16, 7), (19, 9, 20, 8), (9, 19, 10, 18), (17, 11, 18, 10), (23, 12, 0, 13), (13, 22, 14, 23), (21, 14, 22, 15), (16, 1, 17, 2)], (16, 1, 17, 2)], ['t02104', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (19, 4, 20, 5), (5, 14, 6, 15), (15, 6, 16, 7), (7, 22, 8, 23), (9, 13, 10, 12), (13, 21, 14, 20), (21, 17, 22, 16), (23, 18, 0, 19), (8, 17, 9, 18)], (8, 17, 9, 18)], ['t02238', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (17, 4, 18, 5), (5, 16, 6, 17), (7, 15, 8, 14), (21, 8, 22, 9), (9, 20, 10, 21), (13, 11, 14, 10), (15, 25, 16, 24), (23, 19, 24, 18), (19, 23, 20, 22), (6, 25, 7, 0)], (6, 25, 7, 0)], ['t02378', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (13, 4, 14, 5), (15, 7, 16, 6), (7, 20, 8, 21), (21, 8, 22, 9), (9, 22, 10, 23), (25, 12, 0, 13), (19, 14, 20, 15), (23, 17, 24, 16), (17, 25, 18, 24), (18, 5, 19, 6)], (18, 5, 19, 6)], ['t02398', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (13, 5, 14, 4), (5, 20, 6, 21), (21, 6, 22, 7), (7, 15, 8, 14), (25, 9, 0, 8), (9, 16, 10, 17), (17, 10, 18, 11), (23, 18, 24, 19), (19, 22, 20, 23), (24, 16, 25, 15)], (24, 16, 25, 15)], ['t02404', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (15, 4, 16, 5), (5, 21, 6, 20), (21, 7, 22, 6), (7, 16, 8, 17), (11, 9, 12, 8), (17, 12, 18, 13), (13, 23, 14, 22), (19, 15, 20, 14), (18, 0, 19, 23)], (18, 0, 19, 23)], ['t02470', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (15, 4, 16, 5), (5, 18, 6, 19), (17, 6, 18, 7), (7, 16, 8, 17), (23, 9, 24, 8), (9, 23, 10, 22), (21, 13, 22, 12), (19, 14, 20, 15), (25, 21, 0, 20), (24, 13, 25, 14)], (24, 13, 25, 14)], ['t02537', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (19, 4, 20, 5), (5, 22, 6, 23), (21, 6, 22, 7), (7, 20, 8, 21), (15, 8, 16, 9), (9, 16, 10, 17), (25, 12, 0, 13), (13, 19, 14, 18), (23, 15, 24, 14), (24, 17, 25, 18)], (24, 17, 25, 18)], ['t02567', [(3, 1, 4, 0), (1, 14, 2, 15), (13, 2, 14, 3), (15, 5, 16, 4), (17, 6, 18, 7), (7, 25, 8, 24), (23, 9, 24, 8), (9, 23, 10, 22), (25, 11, 0, 10), (11, 18, 12, 19), (19, 12, 20, 13), (21, 17, 22, 16), (20, 6, 21, 5)], (20, 6, 21, 5)], ['t02639', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (11, 5, 12, 4), (5, 16, 6, 17), (15, 6, 16, 7), (7, 21, 8, 20), (21, 9, 22, 8), (25, 13, 0, 12), (13, 18, 14, 19), (23, 14, 24, 15), (19, 23, 20, 22), (24, 18, 25, 17)], (24, 18, 25, 17)], ['t03607', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (19, 4, 20, 5), (5, 20, 6, 21), (15, 6, 16, 7), (7, 16, 8, 17), (25, 10, 0, 11), (11, 24, 12, 25), (13, 23, 14, 22), (21, 15, 22, 14), (23, 18, 24, 19), (12, 18, 13, 17)], (12, 18, 13, 17)], ['t03713', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (15, 5, 16, 4), (13, 6, 14, 7), (7, 20, 8, 21), (11, 9, 12, 8), (21, 12, 22, 13), (19, 15, 20, 14), (23, 17, 0, 16), (17, 23, 18, 22), (18, 5, 19, 6)], (18, 5, 19, 6)], ['t03781', [(1, 12, 2, 13), (11, 2, 12, 3), (3, 18, 4, 19), (17, 4, 18, 5), (13, 7, 14, 6), (7, 22, 8, 23), (21, 8, 22, 9), (9, 20, 10, 21), (19, 10, 20, 11), (25, 15, 0, 14), (15, 25, 16, 24), (23, 17, 24, 16), (0, 5, 1, 6)], (0, 5, 1, 6)], ['t03864', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (11, 5, 12, 4), (5, 20, 6, 21), (13, 6, 14, 7), (15, 9, 16, 8), (21, 13, 22, 12), (19, 14, 20, 15), (23, 16, 0, 17), (17, 22, 18, 23), (18, 7, 19, 8)], (18, 7, 19, 8)], ['t03956', [(5, 1, 6, 0), (1, 15, 2, 14), (9, 3, 10, 2), (3, 9, 4, 8), (15, 5, 16, 4), (7, 22, 8, 23), (21, 10, 22, 11), (11, 20, 12, 21), (19, 12, 20, 13), (13, 18, 14, 19), (23, 16, 0, 17), (6, 18, 7, 17)], (6, 18, 7, 17)], ['t03979', [(5, 1, 6, 0), (1, 17, 2, 16), (7, 2, 8, 3), (19, 5, 20, 4), (15, 6, 16, 7), (17, 9, 18, 8), (9, 19, 10, 18), (11, 22, 12, 23), (21, 12, 22, 13), (13, 20, 14, 21), (23, 14, 0, 15), (10, 4, 11, 3)], (10, 4, 11, 3)], ['t04003', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (17, 5, 18, 4), (19, 6, 20, 7), (7, 14, 8, 15), (15, 8, 16, 9), (11, 16, 12, 17), (21, 13, 22, 12), (13, 21, 14, 20), (23, 19, 0, 18), (22, 6, 23, 5)], (22, 6, 23, 5)], ['t04019', [(5, 1, 6, 0), (1, 14, 2, 15), (13, 2, 14, 3), (3, 18, 4, 19), (17, 4, 18, 5), (25, 7, 0, 6), (7, 25, 8, 24), (23, 9, 24, 8), (21, 10, 22, 11), (11, 20, 12, 21), (19, 12, 20, 13), (15, 22, 16, 23), (16, 10, 17, 9)], (16, 10, 17, 9)], ['t04102', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (15, 4, 16, 5), (5, 14, 6, 15), (19, 6, 20, 7), (17, 9, 18, 8), (11, 17, 12, 16), (21, 12, 22, 13), (13, 20, 14, 21), (23, 18, 0, 19), (22, 8, 23, 7)], (22, 8, 23, 7)], ['t04180', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (15, 4, 16, 5), (5, 16, 6, 17), (23, 7, 24, 6), (7, 23, 8, 22), (21, 9, 22, 8), (9, 21, 10, 20), (19, 13, 20, 12), (13, 25, 14, 24), (17, 14, 18, 15), (18, 25, 19, 0)], (18, 25, 19, 0)], ['t04244', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (13, 5, 14, 4), (5, 17, 6, 16), (17, 7, 18, 6), (7, 24, 8, 25), (21, 8, 22, 9), (11, 22, 12, 23), (23, 12, 24, 13), (19, 15, 20, 14), (15, 19, 16, 18), (20, 25, 21, 0)], (20, 25, 21, 0)], ['t04382', [(3, 1, 4, 0), (1, 14, 2, 15), (13, 2, 14, 3), (15, 5, 16, 4), (5, 23, 6, 22), (17, 6, 18, 7), (7, 18, 8, 19), (23, 9, 24, 8), (9, 25, 10, 24), (19, 10, 20, 11), (11, 20, 12, 21), (25, 13, 0, 12), (16, 22, 17, 21)], (16, 22, 17, 21)], ['t04721', [(1, 21, 2, 20), (15, 3, 16, 2), (3, 15, 4, 14), (21, 5, 22, 4), (17, 6, 18, 7), (7, 18, 8, 19), (19, 8, 20, 9), (9, 16, 10, 17), (23, 11, 24, 10), (11, 25, 12, 24), (25, 13, 0, 12), (13, 23, 14, 22), (0, 5, 1, 6)], (0, 5, 1, 6)], ['t05118', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (19, 4, 20, 5), (5, 18, 6, 19), (15, 6, 16, 7), (7, 24, 8, 25), (13, 9, 14, 8), (11, 23, 12, 22), (23, 13, 24, 12), (21, 17, 22, 16), (17, 21, 18, 20), (14, 0, 15, 25)], (14, 0, 15, 25)], ['t05390', [(1, 17, 2, 16), (11, 2, 12, 3), (3, 12, 4, 13), (17, 5, 18, 4), (5, 19, 6, 18), (19, 7, 20, 6), (15, 8, 16, 9), (9, 22, 10, 23), (21, 10, 22, 11), (13, 20, 14, 21), (23, 14, 0, 15), (0, 7, 1, 8)], (0, 7, 1, 8)], ['t05425', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (17, 4, 18, 5), (5, 22, 6, 23), (21, 6, 22, 7), (7, 14, 8, 15), (13, 8, 14, 9), (9, 18, 10, 19), (15, 20, 16, 21), (23, 16, 0, 17), (12, 20, 13, 19)], (12, 20, 13, 19)], ['t05538', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (15, 5, 16, 4), (5, 22, 6, 23), (13, 6, 14, 7), (7, 21, 8, 20), (19, 9, 20, 8), (9, 19, 10, 18), (17, 13, 18, 12), (21, 14, 22, 15), (16, 23, 17, 0)], (16, 23, 17, 0)], ['t05564', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (19, 5, 20, 4), (5, 24, 6, 25), (23, 6, 24, 7), (7, 19, 8, 18), (17, 9, 18, 8), (9, 17, 10, 16), (13, 11, 14, 10), (21, 15, 22, 14), (15, 23, 16, 22), (20, 25, 21, 0)], (20, 25, 21, 0)], ['t05658', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (17, 5, 18, 4), (5, 22, 6, 23), (15, 6, 16, 7), (7, 14, 8, 15), (9, 13, 10, 12), (13, 20, 14, 21), (21, 16, 22, 17), (23, 19, 0, 18), (8, 20, 9, 19)], (8, 20, 9, 19)], ['t05674', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (19, 4, 20, 5), (5, 18, 6, 19), (17, 6, 18, 7), (7, 24, 8, 25), (11, 23, 12, 22), (21, 13, 22, 12), (13, 21, 14, 20), (23, 15, 24, 14), (25, 16, 0, 17), (8, 15, 9, 16)], (8, 15, 9, 16)], ['t05695', [(3, 1, 4, 0), (1, 14, 2, 15), (13, 2, 14, 3), (5, 24, 6, 25), (23, 6, 24, 7), (7, 17, 8, 16), (15, 9, 16, 8), (9, 22, 10, 23), (21, 10, 22, 11), (11, 20, 12, 21), (19, 12, 20, 13), (25, 19, 0, 18), (4, 17, 5, 18)], (4, 17, 5, 18)], ['t06001', [(3, 1, 4, 0), (1, 14, 2, 15), (13, 2, 14, 3), (19, 4, 20, 5), (5, 24, 6, 25), (23, 6, 24, 7), (7, 22, 8, 23), (17, 9, 18, 8), (9, 17, 10, 16), (21, 10, 22, 11), (11, 20, 12, 21), (15, 13, 16, 12), (18, 0, 19, 25)], (18, 0, 19, 25)], ['t06440', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (15, 5, 16, 4), (5, 17, 6, 16), (25, 7, 0, 6), (7, 20, 8, 21), (19, 8, 20, 9), (9, 18, 10, 19), (21, 10, 22, 11), (13, 22, 14, 23), (23, 14, 24, 15), (24, 18, 25, 17)], (24, 18, 25, 17)], ['t06463', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (13, 5, 14, 4), (5, 24, 6, 25), (17, 7, 18, 6), (7, 19, 8, 18), (15, 8, 16, 9), (11, 23, 12, 22), (21, 13, 22, 12), (19, 17, 20, 16), (23, 20, 24, 21), (14, 25, 15, 0)], (14, 25, 15, 0)], ['t06525', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (9, 4, 10, 5), (5, 20, 6, 21), (17, 6, 18, 7), (7, 23, 8, 22), (19, 11, 20, 10), (11, 19, 12, 18), (23, 14, 0, 15), (21, 17, 22, 16), (8, 15, 9, 16)], (8, 15, 9, 16)], ['t06570', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (19, 4, 20, 5), (5, 20, 6, 21), (21, 6, 22, 7), (7, 18, 8, 19), (25, 9, 0, 8), (9, 17, 10, 16), (15, 11, 16, 10), (13, 23, 14, 22), (23, 15, 24, 14), (24, 17, 25, 18)], (24, 17, 25, 18)], ['t06605', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (11, 5, 12, 4), (5, 22, 6, 23), (21, 6, 22, 7), (7, 16, 8, 17), (15, 8, 16, 9), (13, 24, 14, 25), (17, 15, 18, 14), (25, 18, 0, 19), (23, 21, 24, 20), (12, 20, 13, 19)], (12, 20, 13, 19)], ['t07348', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (9, 4, 10, 5), (5, 17, 6, 16), (19, 7, 20, 6), (21, 8, 0, 9), (17, 10, 18, 11), (11, 18, 12, 19), (15, 21, 16, 20), (14, 8, 15, 7)], (14, 8, 15, 7)], ['t08111', [(5, 1, 6, 0), (1, 10, 2, 11), (9, 2, 10, 3), (3, 16, 4, 17), (15, 4, 16, 5), (13, 7, 14, 6), (17, 8, 18, 9), (11, 18, 12, 19), (19, 12, 20, 13), (21, 15, 0, 14), (20, 8, 21, 7)], (20, 8, 21, 7)], ['t08201', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (9, 4, 10, 5), (5, 17, 6, 16), (7, 14, 8, 15), (21, 8, 0, 9), (17, 10, 18, 11), (11, 18, 12, 19), (15, 21, 16, 20), (6, 19, 7, 20)], (6, 19, 7, 20)], ['t08267', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (11, 5, 12, 4), (5, 16, 6, 17), (13, 6, 14, 7), (7, 22, 8, 23), (19, 8, 20, 9), (21, 15, 22, 14), (15, 21, 16, 20), (23, 19, 0, 18), (12, 17, 13, 18)], (12, 17, 13, 18)], ['t08403', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (15, 5, 16, 4), (5, 22, 6, 23), (11, 6, 12, 7), (7, 19, 8, 18), (19, 12, 20, 13), (13, 20, 14, 21), (21, 14, 22, 15), (23, 17, 0, 16), (10, 17, 11, 18)], (10, 17, 11, 18)], ['t09016', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (17, 4, 18, 5), (11, 6, 12, 7), (7, 14, 8, 15), (15, 10, 16, 11), (19, 13, 20, 12), (13, 21, 14, 20), (23, 16, 0, 17), (21, 19, 22, 18), (22, 6, 23, 5)], (22, 6, 23, 5)], ['t09267', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (13, 5, 14, 4), (17, 6, 18, 7), (7, 18, 8, 19), (19, 8, 20, 9), (11, 20, 12, 21), (21, 12, 22, 13), (23, 15, 24, 14), (15, 25, 16, 24), (25, 17, 0, 16), (22, 6, 23, 5)], (22, 6, 23, 5)], ['t09313', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (13, 5, 14, 4), (5, 13, 6, 12), (19, 6, 20, 7), (7, 20, 8, 21), (11, 16, 12, 17), (25, 15, 0, 14), (15, 25, 16, 24), (17, 22, 18, 23), (21, 18, 22, 19), (10, 24, 11, 23)], (10, 24, 11, 23)], ['t09455', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (19, 4, 20, 5), (5, 18, 6, 19), (17, 6, 18, 7), (13, 9, 14, 8), (11, 21, 12, 20), (21, 13, 22, 12), (23, 14, 24, 15), (15, 24, 16, 25), (25, 16, 0, 17), (22, 8, 23, 7)], (22, 8, 23, 7)], ['t09580', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (5, 22, 6, 23), (15, 7, 16, 6), (7, 15, 8, 14), (21, 8, 22, 9), (9, 18, 10, 19), (19, 12, 20, 13), (13, 20, 14, 21), (23, 16, 0, 17), (4, 18, 5, 17)], (4, 18, 5, 17)], ['t09704', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (15, 5, 16, 4), (5, 15, 6, 14), (19, 6, 20, 7), (7, 20, 8, 21), (25, 10, 0, 11), (11, 16, 12, 17), (23, 13, 24, 12), (13, 23, 14, 22), (21, 18, 22, 19), (24, 18, 25, 17)], (24, 18, 25, 17)], ['t09852', [(1, 16, 2, 17), (9, 3, 10, 2), (3, 9, 4, 8), (15, 4, 16, 5), (19, 7, 20, 6), (7, 12, 8, 13), (17, 10, 18, 11), (11, 18, 12, 19), (13, 21, 14, 20), (21, 15, 0, 14), (0, 5, 1, 6)], (0, 5, 1, 6)], ['t09954', [(5, 1, 6, 0), (1, 12, 2, 13), (3, 11, 4, 10), (15, 4, 16, 5), (13, 7, 14, 6), (7, 18, 8, 19), (19, 8, 20, 9), (9, 20, 10, 21), (11, 17, 12, 16), (21, 15, 0, 14), (2, 17, 3, 18)], (2, 17, 3, 18)], ['t10230', [(5, 1, 6, 0), (1, 16, 2, 17), (9, 3, 10, 2), (3, 9, 4, 8), (15, 4, 16, 5), (19, 7, 20, 6), (17, 10, 18, 11), (11, 18, 12, 19), (13, 21, 14, 20), (21, 15, 0, 14), (12, 8, 13, 7)], (12, 8, 13, 7)], ['t10462', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (5, 22, 6, 23), (17, 6, 18, 7), (7, 18, 8, 19), (21, 8, 22, 9), (9, 15, 10, 14), (19, 13, 20, 12), (13, 21, 14, 20), (23, 17, 0, 16), (4, 15, 5, 16)], (4, 15, 5, 16)], ['t10643', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (21, 5, 22, 4), (5, 21, 6, 20), (19, 7, 20, 6), (7, 17, 8, 16), (11, 18, 12, 19), (17, 12, 18, 13), (23, 15, 24, 14), (15, 25, 16, 24), (25, 23, 0, 22), (8, 13, 9, 14)], (8, 13, 9, 14)], ['t10681', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (19, 4, 20, 5), (5, 20, 6, 21), (13, 6, 14, 7), (7, 14, 8, 15), (25, 10, 0, 11), (17, 12, 18, 13), (15, 25, 16, 24), (23, 17, 24, 16), (21, 18, 22, 19), (22, 12, 23, 11)], (22, 12, 23, 11)], ['t10985', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (13, 5, 14, 4), (5, 20, 6, 21), (21, 6, 22, 7), (7, 15, 8, 14), (15, 9, 16, 8), (9, 19, 10, 18), (25, 17, 0, 16), (17, 25, 18, 24), (19, 22, 20, 23), (10, 23, 11, 24)], (10, 23, 11, 24)], ['t11852', [(5, 1, 6, 0), (1, 18, 2, 19), (11, 3, 12, 2), (3, 11, 4, 10), (17, 4, 18, 5), (13, 7, 14, 6), (7, 15, 8, 14), (21, 9, 0, 8), (9, 16, 10, 17), (19, 12, 20, 13), (20, 16, 21, 15)], (20, 16, 21, 15)], ['v0165', [(5, 1, 6, 0), (1, 16, 2, 17), (11, 2, 12, 3), (3, 12, 4, 13), (21, 7, 22, 6), (7, 21, 8, 20), (19, 9, 20, 8), (9, 19, 10, 18), (17, 11, 18, 10), (13, 23, 14, 22), (23, 15, 0, 14), (4, 16, 5, 15)], (4, 16, 5, 15)], ['v0330', [(5, 1, 6, 0), (1, 17, 2, 16), (17, 3, 18, 2), (3, 13, 4, 12), (13, 5, 14, 4), (7, 18, 8, 19), (19, 8, 20, 9), (9, 20, 10, 21), (21, 10, 22, 11), (11, 22, 12, 23), (23, 14, 0, 15), (6, 16, 7, 15)], (6, 16, 7, 15)], ['v0398', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (19, 4, 20, 5), (5, 18, 6, 19), (7, 17, 8, 16), (21, 8, 22, 9), (9, 20, 10, 21), (15, 11, 16, 10), (11, 15, 12, 14), (17, 23, 18, 22), (6, 23, 7, 0)], (6, 23, 7, 0)], ['v0407', [(1, 16, 2, 17), (11, 2, 12, 3), (3, 12, 4, 13), (15, 4, 16, 5), (19, 7, 20, 6), (7, 21, 8, 20), (21, 9, 22, 8), (9, 19, 10, 18), (17, 11, 18, 10), (13, 23, 14, 22), (23, 15, 0, 14), (0, 5, 1, 6)], (0, 5, 1, 6)], ['v0434', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (17, 4, 18, 5), (5, 20, 6, 21), (15, 7, 16, 6), (7, 15, 8, 14), (19, 8, 20, 9), (9, 18, 10, 19), (23, 12, 0, 13), (13, 22, 14, 23), (16, 22, 17, 21)], (16, 22, 17, 21)], ['v0554', [(7, 1, 8, 0), (1, 18, 2, 19), (9, 2, 10, 3), (3, 10, 4, 11), (15, 4, 16, 5), (5, 16, 6, 17), (17, 6, 18, 7), (11, 21, 12, 20), (21, 13, 22, 12), (13, 23, 14, 22), (23, 15, 0, 14), (8, 19, 9, 20)], (8, 19, 9, 20)], ['v0570', [(7, 1, 8, 0), (9, 2, 10, 3), (3, 10, 4, 11), (17, 4, 18, 5), (5, 16, 6, 17), (15, 6, 16, 7), (19, 9, 20, 8), (11, 23, 12, 22), (21, 13, 22, 12), (13, 21, 14, 20), (23, 15, 0, 14), (18, 2, 19, 1)], (18, 2, 19, 1)], ['v0573', [(7, 1, 8, 0), (1, 19, 2, 18), (9, 2, 10, 3), (3, 14, 4, 15), (15, 4, 16, 5), (5, 16, 6, 17), (23, 7, 0, 6), (19, 11, 20, 10), (11, 23, 12, 22), (21, 13, 22, 12), (13, 21, 14, 20), (8, 18, 9, 17)], (8, 18, 9, 17)], ['v0707', [(5, 1, 6, 0), (1, 18, 2, 19), (7, 2, 8, 3), (3, 15, 4, 14), (11, 5, 12, 4), (17, 8, 18, 9), (9, 16, 10, 17), (15, 10, 16, 11), (21, 12, 0, 13), (13, 20, 14, 21), (6, 19, 7, 20)], (6, 19, 7, 20)], ['v0740', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (17, 4, 18, 5), (5, 16, 6, 17), (23, 7, 0, 6), (7, 23, 8, 22), (21, 9, 22, 8), (9, 21, 10, 20), (15, 10, 16, 11), (19, 14, 20, 15), (18, 12, 19, 11)], (18, 12, 19, 11)], ['v0759', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (17, 4, 18, 5), (5, 18, 6, 19), (19, 6, 20, 7), (7, 22, 8, 23), (9, 13, 10, 12), (13, 21, 14, 20), (21, 15, 22, 14), (23, 16, 0, 17), (8, 15, 9, 16)], (8, 15, 9, 16)], ['v0765', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (17, 4, 18, 5), (15, 7, 16, 6), (7, 15, 8, 14), (21, 8, 22, 9), (9, 20, 10, 21), (19, 10, 20, 11), (11, 18, 12, 19), (23, 16, 0, 17), (22, 6, 23, 5)], (22, 6, 23, 5)], ['v0939', [(5, 1, 6, 0), (7, 2, 8, 3), (3, 15, 4, 14), (11, 5, 12, 4), (19, 7, 20, 6), (15, 8, 16, 9), (9, 16, 10, 17), (17, 10, 18, 11), (21, 12, 0, 13), (13, 20, 14, 21), (18, 2, 19, 1)], (18, 2, 19, 1)], ['v0945', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (15, 4, 16, 5), (5, 14, 6, 15), (23, 7, 0, 6), (7, 13, 8, 12), (19, 8, 20, 9), (9, 18, 10, 19), (21, 17, 22, 16), (17, 21, 18, 20), (22, 13, 23, 14)], (22, 13, 23, 14)], ['v1077', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (13, 5, 14, 4), (5, 16, 6, 17), (15, 6, 16, 7), (19, 8, 20, 9), (11, 20, 12, 21), (21, 12, 22, 13), (17, 15, 18, 14), (23, 19, 0, 18), (22, 8, 23, 7)], (22, 8, 23, 7)], ['v1109', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (19, 5, 20, 4), (5, 19, 6, 18), (13, 6, 14, 7), (7, 14, 8, 15), (23, 10, 0, 11), (11, 17, 12, 16), (21, 13, 22, 12), (17, 20, 18, 21), (22, 15, 23, 16)], (22, 15, 23, 16)], ['v1392', [(5, 1, 6, 0), (1, 16, 2, 17), (9, 3, 10, 2), (3, 9, 4, 8), (13, 7, 14, 6), (7, 20, 8, 21), (17, 10, 18, 11), (11, 18, 12, 19), (19, 12, 20, 13), (21, 15, 0, 14), (4, 16, 5, 15)], (4, 16, 5, 15)], ['v1547', [(5, 1, 6, 0), (1, 10, 2, 11), (9, 2, 10, 3), (3, 16, 4, 17), (15, 4, 16, 5), (19, 7, 20, 6), (7, 12, 8, 13), (17, 8, 18, 9), (13, 21, 14, 20), (21, 15, 0, 14), (18, 12, 19, 11)], (18, 12, 19, 11)], ['v1620', [(1, 17, 2, 16), (17, 3, 18, 2), (3, 11, 4, 10), (11, 5, 12, 4), (15, 6, 16, 7), (7, 20, 8, 21), (19, 8, 20, 9), (9, 18, 10, 19), (21, 12, 22, 13), (13, 22, 14, 23), (23, 14, 0, 15), (0, 5, 1, 6)], (0, 5, 1, 6)], ['v1690', [(3, 1, 4, 0), (1, 9, 2, 8), (9, 3, 10, 2), (13, 5, 14, 4), (11, 6, 12, 7), (7, 18, 8, 19), (19, 10, 20, 11), (17, 13, 18, 12), (21, 15, 0, 14), (15, 21, 16, 20), (16, 5, 17, 6)], (16, 5, 17, 6)], ['v1709', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (11, 5, 12, 4), (5, 19, 6, 18), (13, 6, 14, 7), (7, 20, 8, 21), (15, 9, 16, 8), (19, 15, 20, 14), (21, 16, 0, 17), (12, 18, 13, 17)], (12, 18, 13, 17)], ['v1716', [(5, 1, 6, 0), (1, 9, 2, 8), (9, 3, 10, 2), (3, 18, 4, 19), (17, 4, 18, 5), (13, 6, 14, 7), (7, 14, 8, 15), (19, 11, 20, 10), (21, 12, 0, 13), (15, 20, 16, 21), (16, 11, 17, 12)], (16, 11, 17, 12)], ['v1718', [(5, 1, 6, 0), (1, 17, 2, 16), (17, 3, 18, 2), (3, 13, 4, 12), (13, 5, 14, 4), (23, 7, 0, 6), (19, 8, 20, 9), (9, 20, 10, 21), (21, 10, 22, 11), (11, 18, 12, 19), (15, 23, 16, 22), (14, 7, 15, 8)], (14, 7, 15, 8)], ['v1728', [(5, 1, 6, 0), (9, 2, 10, 3), (3, 15, 4, 14), (11, 5, 12, 4), (21, 7, 0, 6), (7, 21, 8, 20), (17, 9, 18, 8), (15, 10, 16, 11), (19, 12, 20, 13), (13, 18, 14, 19), (16, 2, 17, 1)], (16, 2, 17, 1)], ['v1810', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (5, 22, 6, 23), (17, 6, 18, 7), (7, 16, 8, 17), (15, 8, 16, 9), (11, 21, 12, 20), (19, 13, 20, 12), (23, 15, 0, 14), (21, 18, 22, 19), (4, 13, 5, 14)], (4, 13, 5, 14)], ['v1832', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (15, 5, 16, 4), (5, 20, 6, 21), (13, 6, 14, 7), (7, 12, 8, 13), (17, 8, 18, 9), (19, 14, 20, 15), (21, 17, 0, 16), (18, 12, 19, 11)], (18, 12, 19, 11)], ['v1839', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (11, 5, 12, 4), (5, 20, 6, 21), (19, 6, 20, 7), (7, 15, 8, 14), (15, 9, 16, 8), (13, 22, 14, 23), (23, 16, 0, 17), (21, 19, 22, 18), (12, 18, 13, 17)], (12, 18, 13, 17)], ['v1921', [(5, 1, 6, 0), (1, 11, 2, 10), (11, 3, 12, 2), (3, 15, 4, 14), (15, 5, 16, 4), (17, 6, 18, 7), (7, 20, 8, 21), (19, 8, 20, 9), (9, 18, 10, 19), (21, 12, 22, 13), (13, 22, 14, 23), (16, 0, 17, 23)], (16, 0, 17, 23)], ['v1980', [(1, 15, 2, 14), (7, 2, 8, 3), (3, 10, 4, 11), (17, 5, 18, 4), (13, 6, 14, 7), (15, 9, 16, 8), (9, 17, 10, 16), (11, 18, 12, 19), (19, 12, 0, 13), (0, 5, 1, 6)], (0, 5, 1, 6)], ['v1986', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (17, 5, 18, 4), (5, 22, 6, 23), (21, 6, 22, 7), (7, 17, 8, 16), (15, 9, 16, 8), (9, 20, 10, 21), (19, 10, 20, 11), (11, 15, 12, 14), (18, 23, 19, 0)], (18, 23, 19, 0)], ['v2024', [(3, 1, 4, 0), (1, 13, 2, 12), (13, 3, 14, 2), (17, 4, 18, 5), (5, 18, 6, 19), (19, 6, 20, 7), (7, 16, 8, 17), (9, 15, 10, 14), (21, 10, 22, 11), (11, 20, 12, 21), (15, 23, 16, 22), (8, 23, 9, 0)], (8, 23, 9, 0)], ['v2090', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (13, 5, 14, 4), (5, 22, 6, 23), (17, 7, 18, 6), (7, 19, 8, 18), (15, 8, 16, 9), (11, 20, 12, 21), (21, 12, 22, 13), (19, 17, 20, 16), (14, 23, 15, 0)], (14, 23, 15, 0)], ['v2215', [(3, 1, 4, 0), (1, 10, 2, 11), (9, 2, 10, 3), (5, 20, 6, 21), (13, 7, 14, 6), (7, 19, 8, 18), (17, 9, 18, 8), (11, 17, 12, 16), (19, 13, 20, 12), (21, 14, 0, 15), (4, 16, 5, 15)], (4, 16, 5, 15)], ['v2325', [(3, 1, 4, 0), (1, 11, 2, 10), (11, 3, 12, 2), (15, 4, 16, 5), (5, 22, 6, 23), (21, 6, 22, 7), (7, 18, 8, 19), (17, 8, 18, 9), (9, 16, 10, 17), (19, 12, 20, 13), (13, 20, 14, 21), (14, 0, 15, 23)], (14, 0, 15, 23)], ['v2759', [(5, 1, 6, 0), (1, 17, 2, 16), (17, 3, 18, 2), (3, 11, 4, 10), (11, 5, 12, 4), (7, 20, 8, 21), (19, 8, 20, 9), (9, 18, 10, 19), (21, 12, 22, 13), (13, 22, 14, 23), (23, 14, 0, 15), (6, 16, 7, 15)], (6, 16, 7, 15)], ['v2930', [(7, 1, 8, 0), (1, 9, 2, 8), (11, 3, 12, 2), (15, 4, 16, 5), (5, 16, 6, 17), (17, 6, 18, 7), (9, 18, 10, 19), (19, 10, 20, 11), (21, 13, 22, 12), (13, 23, 14, 22), (23, 15, 0, 14), (20, 4, 21, 3)], (20, 4, 21, 3)], ['v3354', [(3, 1, 4, 0), (1, 12, 2, 13), (11, 2, 12, 3), (15, 5, 16, 4), (5, 22, 6, 23), (21, 6, 22, 7), (7, 15, 8, 14), (13, 9, 14, 8), (9, 19, 10, 18), (19, 11, 20, 10), (17, 21, 18, 20), (16, 23, 17, 0)], (16, 23, 17, 0)]]
len(PD_codes_of_tangle_exteriors)

381

In [6]:
L_space_knots_with_QA=[x for x in Lspace_census if x not in asymmetric+pos_no_QA_surgery+[y[0] for y in PD_codes_of_tangle_exteriors]]
len(L_space_knots_with_QA)

236

In [7]:
start_time = time.time()
TANGLE_EXTERIORS=[]

for knot in pos_no_QA_surgery:
    #try:
    T=find_tangle_exterior(knot)
    #except ValueError:
        #T=find_tangle_exterior(knot,slopes=[(0,1),(-1,2),(-1,1)])
    print(T)
    TANGLE_EXTERIORS.append(T)
    PD_codes_of_tangle_exteriors.append([T[0],T[1].PD_code(),T[2]])

print("--- Time taken: %s minutes ---" % ((time.time() - start_time)/60))   

['v2871', <Link: 1 comp; 14 cross>, (13, 4, 14, 5), True, True]
['t09847', <Link: 1 comp; 14 cross>, (23, 6, 24, 7), True, True]
['t09882', <Link: 1 comp; 14 cross>, (12, 21, 13, 22), True, True]
['o9_30634', <Link: 1 comp; 15 cross>, (17, 8, 18, 9), True, True]
['o9_32044', <Link: 1 comp; 14 cross>, (17, 6, 18, 7), True, True]
['o9_32964', <Link: 1 comp; 15 cross>, (17, 27, 18, 26), True, True]
--- Time taken: 6.501451961199442 minutes ---


In [8]:
small_branching_sets=[]
for x in TANGLE_EXTERIORS:
    knot=x[0]
    print(knot)
    PD=x[1].PD_code()
    cros=x[2]    
    result=create_braid_words_of_branching_sets(knot,PD,cros)
    small_branching_sets.append(result)
    if result[1]==False:
        print(result)

v2871
t09847
t09882
o9_30634
o9_32044
o9_32964


We transform these data into something which is readable by mathematica. Then we transfer the data to mathematica to compute the Khovanov homology.

In [9]:
Math_str='{'

for [knot,codes] in small_branching_sets:
    Math_str=Math_str+'{'
    Math_str=Math_str+'"'+knot+'"'+','
    Math_str=Math_str+'{'
    for x in codes:
        Math_str=Math_str+'{'
        Math_str=Math_str+str(braid_index(x[1]))+','
        Math_str=Math_str+braid_word_to_Mathematica(x[1])
        Math_str=Math_str+'},'
    Math_str=Math_str[:-1]
    Math_str=Math_str+'}},'
    
Math_str=Math_str[:-1]
Math_str=Math_str+'}'

Math_str

'{{"v2871",{{5,{-1, 2, -3, 2, 1, -2, 3, -2, 3, 4, -3, 2, -3, -4, 3, 3, 2, 2, 2, 2}},{9,{-1, 2, 3, -4, -5, -4, 6, -3, -5, -7, -2, 4, -6, -8, 1, 3, 7, -2, -4, 5, -4, 6, 3, -5, 4, 6, -5, -7, 6, 8, -5, 7, -4, -6, -3, 5, 2, -4, -3, 4, 4}},{5,{-1, 2, -3, 2, 1, -2, 3, -2, 3, 4, -3, 2, -3, -4, 3, 3, 2, 2}},{7,{-1, 2, -3, -4, -5, -4, -6, 5, 4, 4, 3, -2, 4, 1, 3, -5, 2, -4, 6, 3, -4, 5, -4, -3, -2, 3, -4}},{6,{-1, 2, 2, -3, 4, -5, 4, 3, -2, 3, -4, 3, 5, -2, 1, -2, 3, -2, -4, 3, 2}},{7,{-1, 2, -3, 2, -4, 1, 3, -4, 5, 4, -6, 3, -5, 2, -4, -3, -4, 5, 4, 6, 4, -5, -4, 3, -2}},{5,{-1, 2, -3, 2, 1, -2, 3, -2, 3, 4, -3, 2, -3, -4, 3, 3, -2, -2}},{5,{-1, -2, 3, -2, 3, 4, -3, 2, -3, -4, 3, 3, -2, -2, -2, 1, 2, -3, 2}},{5,{-1, 2, -3, 2, 1, -2, 3, -2, 3, 4, -3, 2, -3, -4, 3, 3, -2, -2, -2, -2}}}},{"t09847",{{8,{-1, -2, 3, -4, -5, 6, 5, 7, -4, -6, -3, 5, 2, -4, 1, -3, -2, -4, -3, -5, 6, 5, -7, 4, -3, 5, 2, -4, -6, 3, -4, -5, -5, -5, -5, -4}},{8,{-1, -2, 3, -4, -5, 6, 5, 7, -4, -6, -3, 5, 2, -4, 1, -3, -2, -

We continue by finding branching sets of all other strongly invertible L-space census knots. For that we sort out the above 6 and the asymmetric L-space knots and then apply the same code.

In [10]:
start_time = time.time()

unclear=[]

for knot in L_space_knots_with_QA:
    try:
        T=find_tangle_exterior(knot)
        if T is None:
            print('Found none at:',knot)
            unclear.append([knot,False])
        else:
            print(T)
            PD_codes_of_tangle_exteriors.append([T[0],T[1].PD_code(),T[2]])
    except ValueError:
        print('Value Error at:',knot)
        unclear.append([knot,False])
    except RuntimeError:
        print('Runtime Error at:',knot)
        unclear.append([knot,False])
    except AttributeError:
        print('Attribute Error at:',knot)
        unclear.append([knot,False])

print("--- Time taken: %s minutes ---" % ((time.time() - start_time)/60))   

['m211', <Link: 1 comp; 12 cross>, (5, 22, 6, 23), True, True]
['m223', <Link: 1 comp; 10 cross>, (9, 2, 10, 3), True, True]
['s384', <Link: 1 comp; 11 cross>, (19, 12, 20, 13), True, True]
['s560', <Link: 1 comp; 14 cross>, (8, 1, 9, 2), True, True]
['s652', <Link: 1 comp; 11 cross>, (12, 8, 13, 7), True, True]
['s682', <Link: 1 comp; 13 cross>, (15, 6, 16, 7), True, True]
['s849', <Link: 1 comp; 11 cross>, (0, 5, 1, 6), True, True]
['v0249', <Link: 1 comp; 14 cross>, (5, 26, 6, 27), True, True]
['v0319', <Link: 1 comp; 13 cross>, (14, 20, 15, 19), True, True]
['v0545', <Link: 1 comp; 14 cross>, (5, 16, 6, 17), True, True]
['v0830', <Link: 1 comp; 13 cross>, (4, 15, 5, 16), True, True]
['v0959', <Link: 1 comp; 12 cross>, (9, 2, 10, 3), True, True]
['v1269', <Link: 1 comp; 15 cross>, (8, 1, 9, 2), True, True]
['v1359', <Link: 1 comp; 13 cross>, (6, 14, 7, 13), True, True]
['v1423', <Link: 1 comp; 14 cross>, (14, 19, 15, 20), True, True]
['v1425', <Link: 1 comp; 13 cross>, (7, 24, 8, 25

['o9_21620', <Link: 1 comp; 15 cross>, (7, 24, 8, 25), True, True]
['o9_22252', <Link: 1 comp; 14 cross>, (17, 26, 18, 27), True, True]
['o9_23032', <Link: 1 comp; 13 cross>, (6, 15, 7, 16), True, True]
['o9_23179', <Link: 1 comp; 13 cross>, (4, 20, 5, 19), True, True]
['o9_23461', <Link: 1 comp; 15 cross>, (25, 12, 26, 13), True, True]
['o9_23723', <Link: 1 comp; 14 cross>, (26, 7, 27, 8), True, True]
['o9_23971', <Link: 1 comp; 13 cross>, (4, 17, 5, 18), True, True]
['o9_24069', <Link: 1 comp; 15 cross>, (7, 26, 8, 27), True, True]
['o9_24126', False]
Attribute Error at: o9_24126
['o9_24290', <Link: 1 comp; 14 cross>, (25, 16, 26, 17), True, True]
['o9_24401', <Link: 1 comp; 14 cross>, (23, 10, 24, 11), True, True]
['o9_24407', <Link: 1 comp; 15 cross>, (5, 21, 6, 20), True, True]
['o9_24779', <Link: 1 comp; 14 cross>, (4, 15, 5, 16), True, True]
['o9_24946', <Link: 1 comp; 14 cross>, (14, 5, 15, 6), True, True]
['o9_25110', <Link: 1 comp; 15 cross>, (8, 1, 9, 2), True, True]
['o9_25

In [12]:
len(PD_codes_of_tangle_exteriors)

571

In [13]:
len(unclear)

52

In [14]:
print(unclear)

[['v2925', False], ['t03710', False], ['t03843', False], ['t04756', False], ['t04927', False], ['t09500', False], ['t09633', False], ['t10188', False], ['t11556', False], ['t12753', False], ['o9_01175', False], ['o9_01936', False], ['o9_02383', False], ['o9_02909', False], ['o9_07152', False], ['o9_07167', False], ['o9_07195', False], ['o9_07401', False], ['o9_08402', False], ['o9_08497', False], ['o9_09271', False], ['o9_09372', False], ['o9_09731', False], ['o9_11002', False], ['o9_11100', False], ['o9_11537', False], ['o9_11556', False], ['o9_11658', False], ['o9_12079', False], ['o9_12253', False], ['o9_14108', False], ['o9_17382', False], ['o9_24126', False], ['o9_27429', False], ['o9_28751', False], ['o9_30142', False], ['o9_32132', False], ['o9_32150', False], ['o9_32588', False], ['o9_33430', False], ['o9_33959', False], ['o9_34689', False], ['o9_35720', False], ['o9_37551', False], ['o9_37754', False], ['o9_39451', False], ['o9_40026', False], ['o9_40179', False], ['o9_43001',

For the remaining unclear we just use other triples of slopes to find the tangle exteriors.

In [15]:
start_time = time.time()

unclear2=[]

for knot in unclear:
    try:
        T=find_tangle_exterior(knot[0],slopes=[(-1,1),(-1,2),(0,1)])
        if T is None:
            print('Found none at:',knot)
            unclear2.append([knot,False])
        else:
            print(T)
            PD_codes_of_tangle_exteriors.append([T[0],T[1].PD_code(),T[2]])
    except ValueError:
        print('Value Error at:',knot)
        unclear2.append([knot,False])
    except RuntimeError:
        print('Runtime Error at:',knot)
        unclear2.append([knot,False])
    except AttributeError:
        print('Attribute Error at:',knot)
        unclear2.append([knot,False])

print("--- Time taken: %s minutes ---" % ((time.time() - start_time)/60))   


['v2925', <Link: 1 comp; 11 cross>, (20, 8, 21, 7), True, True]
['t03710', <Link: 1 comp; 15 cross>, (5, 28, 6, 29), True, True]
['t03843', <Link: 1 comp; 15 cross>, (25, 17, 26, 16), True, True]
['t04756', <Link: 1 comp; 15 cross>, (5, 28, 6, 29), True, True]
['t04927', <Link: 1 comp; 15 cross>, (7, 12, 8, 13), True, True]
['t09500', <Link: 1 comp; 12 cross>, (12, 0, 13, 23), True, True]
Found none at: ['t09633', False]
['t10188', False]
Attribute Error at: ['t10188', False]
['t11556', False]
Attribute Error at: ['t11556', False]
['t12753', False]
Attribute Error at: ['t12753', False]
['o9_01175', False]
Attribute Error at: ['o9_01175', False]
['o9_01936', <Link: 1 comp; 15 cross>, (15, 29, 16, 28), True, True]
['o9_02383', <Link: 1 comp; 15 cross>, (5, 12, 6, 13), True, True]
['o9_02909', <Link: 1 comp; 15 cross>, (24, 2, 25, 1), True, True]
['o9_07152', False]
Attribute Error at: ['o9_07152', False]
['o9_07167', False]
Attribute Error at: ['o9_07167', False]
['o9_07195', False]
Attr

In [16]:
len(unclear2)

31

In [19]:
unclear2=[x[0] for x in unclear2]

In [21]:
start_time = time.time()

unclear3=[]

for knot in unclear2:
    try:
        T=find_tangle_exterior(knot[0],slopes=[(-2,1),(-3,2),(-1,1)])
        if T is None:
            print('Found none at:',knot)
            unclear3.append([knot,False])
        else:
            print(T)
            PD_codes_of_tangle_exteriors.append([T[0],T[1].PD_code(),T[2]])
    except ValueError:
        print('Value Error at:',knot)
        unclear3.append([knot,False])
    except RuntimeError:
        print('Runtime Error at:',knot)
        unclear3.append([knot,False])
    except AttributeError:
        print('Attribute Error at:',knot)
        unclear3.append([knot,False])

print("--- Time taken: %s minutes ---" % ((time.time() - start_time)/60))   


['t09633', <Link: 1 comp; 15 cross>, (25, 10, 26, 11), True, True]
Found none at: ['t10188', False]
['t11556', False]
Attribute Error at: ['t11556', False]
['t12753', <Link: 1 comp; 14 cross>, (15, 4, 16, 5), True, True]
['o9_01175', False]
Attribute Error at: ['o9_01175', False]
['o9_07152', False]
Attribute Error at: ['o9_07152', False]
['o9_07167', False]
Attribute Error at: ['o9_07167', False]
['o9_07195', False]
Attribute Error at: ['o9_07195', False]
['o9_07401', False]
Attribute Error at: ['o9_07401', False]
['o9_08402', False]
Attribute Error at: ['o9_08402', False]
['o9_08497', False]
Attribute Error at: ['o9_08497', False]
['o9_09271', False]
Attribute Error at: ['o9_09271', False]
['o9_09372', False]
Attribute Error at: ['o9_09372', False]
['o9_09731', False]
Attribute Error at: ['o9_09731', False]
['o9_11002', False]
Attribute Error at: ['o9_11002', False]
['o9_11537', False]
Attribute Error at: ['o9_11537', False]
['o9_11658', False]
Attribute Error at: ['o9_11658', False]

In [22]:
len(unclear3)

23

In [23]:
unclear3=[x[0] for x in unclear3]

In [24]:
start_time = time.time()

unclear4=[]

for knot in unclear3:
    try:
        T=find_tangle_exterior(knot[0],slopes=[(2,1),(3,2),(1,1)])
        if T is None:
            print('Found none at:',knot)
            unclear4.append([knot,False])
        else:
            print(T)
            PD_codes_of_tangle_exteriors.append([T[0],T[1].PD_code(),T[2]])
    except ValueError:
        print('Value Error at:',knot)
        unclear4.append([knot,False])
    except RuntimeError:
        print('Runtime Error at:',knot)
        unclear4.append([knot,False])
    except AttributeError:
        print('Attribute Error at:',knot)
        unclear4.append([knot,False])

print("--- Time taken: %s minutes ---" % ((time.time() - start_time)/60))   


['t10188', <Link: 1 comp; 13 cross>, (21, 4, 22, 5), True, True]
Found none at: ['t11556', False]
['o9_01175', False]
Attribute Error at: ['o9_01175', False]
['o9_07152', <Link: 1 comp; 15 cross>, (0, 5, 1, 6), True, True]
['o9_07167', <Link: 1 comp; 15 cross>, (20, 16, 21, 15), True, True]
['o9_07195', <Link: 1 comp; 15 cross>, (26, 21, 27, 22), True, True]
['o9_07401', <Link: 1 comp; 15 cross>, (4, 10, 5, 9), True, True]
['o9_08402', <Link: 1 comp; 15 cross>, (0, 5, 1, 6), True, True]
['o9_08497', <Link: 1 comp; 15 cross>, (7, 28, 8, 29), True, True]
['o9_09271', <Link: 1 comp; 15 cross>, (0, 5, 1, 6), True, True]
['o9_09372', <Link: 1 comp; 15 cross>, (24, 0, 25, 29), True, True]
['o9_09731', <Link: 1 comp; 15 cross>, (19, 29, 20, 28), True, True]
['o9_11002', <Link: 1 comp; 15 cross>, (24, 19, 25, 20), True, True]
['o9_11537', <Link: 1 comp; 15 cross>, (0, 5, 1, 6), True, True]
['o9_11658', <Link: 1 comp; 15 cross>, (0, 5, 1, 6), True, True]
['o9_12079', <Link: 1 comp; 15 cross>, (

In [25]:
len(unclear4)

5

In [28]:
len(PD_codes_of_tangle_exteriors)

618

In [30]:
unclear4=[x[0] for x in unclear4]

In [31]:
start_time = time.time()

unclear5=[]

for knot in unclear4:
    try:
        T=find_tangle_exterior(knot[0],slopes=[(3,1),(5,2),(2,1)])
        if T is None:
            print('Found none at:',knot)
            unclear5.append([knot,False])
        else:
            print(T)
            PD_codes_of_tangle_exteriors.append([T[0],T[1].PD_code(),T[2]])
    except ValueError:
        print('Value Error at:',knot)
        unclear5.append([knot,False])
    except RuntimeError:
        print('Runtime Error at:',knot)
        unclear5.append([knot,False])
    except AttributeError:
        print('Attribute Error at:',knot)
        unclear5.append([knot,False])

print("--- Time taken: %s minutes ---" % ((time.time() - start_time)/60))   

['t11556', <Link: 1 comp; 15 cross>, (8, 1, 9, 2), True, True]
['o9_01175', False]
Attribute Error at: ['o9_01175', False]
['o9_24126', False]
Attribute Error at: ['o9_24126', False]
['o9_37754', <Link: 1 comp; 14 cross>, (6, 24, 7, 23), True, True]
['o9_40026', False]
Attribute Error at: ['o9_40026', False]
--- Time taken: 26.162654427687325 minutes ---


In [32]:
unclear5=[x[0] for x in unclear5]

In [33]:
start_time = time.time()

unclear6=[]

for knot in unclear5:
    try:
        T=find_tangle_exterior(knot[0],slopes=[(3,1),(5,2),(2,1)])
        if T is None:
            print('Found none at:',knot)
            unclear6.append([knot,False])
        else:
            print(T)
            PD_codes_of_tangle_exteriors.append([T[0],T[1].PD_code(),T[2]])
    except ValueError:
        print('Value Error at:',knot)
        unclear6.append([knot,False])
    except RuntimeError:
        print('Runtime Error at:',knot)
        unclear6.append([knot,False])
    except AttributeError:
        print('Attribute Error at:',knot)
        unclear6.append([knot,False])

print("--- Time taken: %s minutes ---" % ((time.time() - start_time)/60))   

['o9_01175', False]
Attribute Error at: ['o9_01175', False]
['o9_24126', False]
Attribute Error at: ['o9_24126', False]
['o9_40026', False]
Attribute Error at: ['o9_40026', False]
--- Time taken: 25.888400574525196 minutes ---


For the last remaining 3 knots we could work a bit harder by including 16 crossing branching sets or searching a bit hard. However, it is fast to just work by hand. For the remaining 3 knots, we build surgery diagrams along strongly invertible links by drilling out short geodesics and then we take the quotient by hand. By doing that we get the following 3 branching sets, which we confirm below.

In [28]:
PD_codes_of_tangle_exteriors.append(['o9_01175',
  [(35,47,0,36),
   (0,37,1,36),
   (16,2,17,1),
  (2,25,3,26),
  (24,3,25,4),
  (4,23,5,24),
  (22,5,23,6),
  (42,7,43,6),
  (7,46,8,45),
  (8,34,9,33),
  (34,10,35,9),
  (46,11,47,10),
  (37,11,38,12),
  (12,38,13,39),
  (39,13,40,14),
  (14,40,15,41),
  (41,15,42,16),
  (17,30,18,31),
  (29,18,30,19),
  (19,28,20,29),
  (27,20,28,21),
  (21,26,22,27),
  (43,32,44,31),
  (32,45,33,44)],
  (35,47,0,36)])
PD_codes_of_tangle_exteriors.append(['o9_24126',
  [(49,7,0,6),
   (0,5,1,6),
  (34,1,35,2),
  (21,3,22,2),
  (3,40,4,41),
  (4,10,5,9),
  (7,30,8,31),
  (31,8,32,9),
  (10,17,11,18),
  (16,11,17,12),
  (12,15,13,16),
  (13,39,14,38),
   (39,15,40,14),
   (18,38,19,37),
  (36,20,37,19),
  (20,36,21,35),
  (22,33,23,34),
  (23,42,24,43),
  (43,24,44,25),
  (25,44,26,45),
  (26,48,27,47),
  (46,28,47,27),
  (28,46,29,45),
  (48,30,49,29),
   (32,42,33,41)],
  (49,7,0,6)])
PD_codes_of_tangle_exteriors.append(['o9_40026',
  [(21,22,0,23),
   (23,0,24,1),
  (38,1,39,2),
  (2,37,3,38),
  (36,3,37,4),
  (4,35,5,36),
  (16,6,17,5),
  (45,7,46,6),
  (7,13,8,12),
  (47,8,48,9),
  (18,9,19,10),
  (10,17,11,18),
   (11,46,12,47),
   (32,13,33,14),
  (43,14,44,15),
  (15,34,16,35),
  (19,31,20,30),
  (29,21,30,20),
  (39,24,40,25),
  (25,40,26,41),
  (41,26,42,27),
  (27,42,28,43),
  (28,22,29,49),
  (48,32,49,31),
   (44,33,45,34)],
  (21,22,0,23)])

In [35]:
len(PD_codes_of_tangle_exteriors)

623

Now that we have all tangle exteriors we continue by creating braid words of the branching sets. For that we first sort the knots according to their S3-fillings:

In [18]:
str_inv_Lspacecensus=[x for x in Lspace_census if x not in asymmetric]
len(str_inv_Lspacecensus)

623

In [19]:
start_time = time.time()
standard_meridian=[]
other_meridian=[]

for knot in str_inv_Lspacecensus:
    slope=find_S3_filling(knot)
    if slope==(1,0) or slope==(-1,0):
        standard_meridian.append(knot)
    else:
        other_meridian.append([knot,slope])
        
print('Number of knots with standard meridian:',len(standard_meridian))
print('Number of knots with other meridian:',len(other_meridian)) 
print("--- Time taken: %s minutes ---" % ((time.time() - start_time)/60))   

Number of knots with standard meridian: 542
Number of knots with other meridian: 81
--- Time taken: 0.7808093269666035 minutes ---


In [15]:
len(PD_codes_of_tangle_exteriors)

623

In [16]:
len(Lspace_census)

632

We start with the knots that have the standard meridian and create now braid words of their branching sets.

In [20]:
PD_codes_of_tangle_exteriors_std_meridians=[x for x in PD_codes_of_tangle_exteriors if x[0] in standard_meridian]
len(PD_codes_of_tangle_exteriors_std_meridians)

542

In [42]:
start_time = time.time()
small_branching_sets=[]
unclear=[]

for [knot,PD,cros] in PD_codes_of_tangle_exteriors_std_meridians:  
    result=create_braid_words_of_branching_sets(knot,PD,cros)
    if result!=False:
        small_branching_sets.append(result)  
    else:
        unclear.append([knot,PD,cros])

print('Number of knots where we have all braid words of tangle fillings:',len(small_branching_sets))
print('Number of unclear knots:',len(unclear))
print("--- Time taken: %s minutes ---" % ((time.time() - start_time)/60)) 

Number of knots where we have all braid words of tangle fillings: 540
Number of unclear knots: 2
--- Time taken: 22.72244195540746 minutes ---


The two unclear knots are those for which we have created the branching sets by hand. This is because the basis there is different. We create braid words of the branching sets for these as well.

In [69]:
unclear

[['o9_01175',
  [(35, 47, 0, 36),
   (0, 37, 1, 36),
   (16, 2, 17, 1),
   (2, 25, 3, 26),
   (24, 3, 25, 4),
   (4, 23, 5, 24),
   (22, 5, 23, 6),
   (42, 7, 43, 6),
   (7, 46, 8, 45),
   (8, 34, 9, 33),
   (34, 10, 35, 9),
   (46, 11, 47, 10),
   (37, 11, 38, 12),
   (12, 38, 13, 39),
   (39, 13, 40, 14),
   (14, 40, 15, 41),
   (41, 15, 42, 16),
   (17, 30, 18, 31),
   (29, 18, 30, 19),
   (19, 28, 20, 29),
   (27, 20, 28, 21),
   (21, 26, 22, 27),
   (43, 32, 44, 31),
   (32, 45, 33, 44)],
  (35, 47, 0, 36)],
 ['o9_40026',
  [(21, 22, 0, 23),
   (23, 0, 24, 1),
   (38, 1, 39, 2),
   (2, 37, 3, 38),
   (36, 3, 37, 4),
   (4, 35, 5, 36),
   (16, 6, 17, 5),
   (45, 7, 46, 6),
   (7, 13, 8, 12),
   (47, 8, 48, 9),
   (18, 9, 19, 10),
   (10, 17, 11, 18),
   (11, 46, 12, 47),
   (32, 13, 33, 14),
   (43, 14, 44, 15),
   (15, 34, 16, 35),
   (19, 31, 20, 30),
   (29, 21, 30, 20),
   (39, 24, 40, 25),
   (25, 40, 26, 41),
   (41, 26, 42, 27),
   (27, 42, 28, 43),
   (28, 22, 29, 49),
   (

In [80]:
braid_words=[]
[knot,PD,cros]=unclear[0]
K=snappy.Manifold(knot)
for s in [(-5,1),(-4,1),(-3,1),(-2,1),(-1,1),(0,1),(1,1),(2,1),(3,1),(4,1),(5,1)]:
    K.dehn_fill(s)
    K_hom=K.homology()
    for p in range(-20,21):
        B=snappy.Link(tangle_filling(PD,cros,(p,1)))
        C=double_branched_cover(B.exterior())
        if K_hom==C.homology():
            B.simplify('global')
            braid_words.append(B.braid_word())
small_branching_sets.append([knot,braid_words])

In [82]:
braid_words=[]
[knot,PD,cros]=unclear[1]
K=snappy.Manifold(knot)
for s in [(-5,1),(-4,1),(-3,1),(-2,1),(-1,1),(0,1),(1,1),(2,1),(3,1),(4,1),(5,1)]:
    K.dehn_fill(s)
    K_hom=K.homology()
    for p in range(-20,21):
        B=snappy.Link(tangle_filling(PD,cros,(p,1)))
        C=double_branched_cover(B.exterior())
        if K_hom==C.homology():
            B.simplify('global')
            braid_words.append(B.braid_word())
small_branching_sets.append([knot,braid_words])

In [83]:
len(small_branching_sets)

542

In [85]:
len(standard_meridian)

542

We transform this into something which is readable by mathematica.

In [91]:
Math_str='{'

for [knot,words] in small_branching_sets:
    Math_str=Math_str+'{'
    Math_str=Math_str+'"'+knot+'"'+','
    Math_str=Math_str+'{'
    for x in words:
        Math_str=Math_str+'{'
        Math_str=Math_str+str(braid_index(x))+','
        Math_str=Math_str+braid_word_to_Mathematica(x)
        Math_str=Math_str+'},'
    Math_str=Math_str[:-1]
    Math_str=Math_str+'}},'
    
Math_str=Math_str[:-1]
Math_str=Math_str+'}'

Math_str

'{{"m016",{{5,{-1, -2, 3, -4, -3, -2, -3, -2, -2, -2, 1, -2, -3, 4, 3, -2}},{5,{-1, -2, 3, -2, -4, -3, -2, -2, -2, 1, -2, -3, 4, 3, -2}},{5,{-1, -2, 3, -4, -3, -2, -3, -2, 1, -2, -3, 4, 3, -2}},{3,{1, 2, 2, 1, -2, 1, -2, -2, 1}},{4,{1, -2, 3, -2, 1, -2, -2, -3, -2}},{3,{1, 1, -2, 1, -2, -2, 1}},{4,{-1, -2, 3, -2, -2, -1, 2, 3, 2}},{6,{-1, 2, -3, 4, -3, -5, -2, 1, -3, 4, 3, -2, 3, -4, 3, 5, 3, -4, 3, 2, -3, -3}},{6,{-1, -2, 3, -4, -3, 5, -3, -4, -3, 2, 1, 3, 3, 3, -2, -3, 4, -3, -5, 2, 3, 4, 3}},{8,{-1, 2, -3, 4, -5, 4, 6, 4, 7, -5, 4, 6, 3, -2, -4, 1, -4, 3, -2, 3, -4, 5, -4, -3, 2, -4, -5, -6, 5, -7, -4, -6, -3, -4, 5, 4}},{6,{-1, -2, 3, -4, -3, 5, -3, -4, -3, 2, 1, 3, 3, 3, 3, 3, -2, -3, 4, -3, -5, 2, 3, 4, 3}}}},{"m071",{{7,{-1, 2, -3, -4, -5, -6, 5, -4, 3, -2, 1, 3, -2, 3, -4, -5, 4, 6, 4, 4, 4, 3, 2, 4, 3, 4, 5, -4}},{5,{-1, -2, 3, -2, 3, 3, 3, 3, -2, -2, 1, -3, 2, -3, 4, -3, -2, -3, -4}},{7,{-1, 2, -3, -4, -5, -6, 5, -4, 3, -2, 1, 3, -2, 3, -4, -5, 4, 6, 4, 3, 2, 4, 3, 4, 5, -4}}

We can do the same for the census L-space knots where the meridian is not standard. Here we need first to recompute the integer slopes in the snappy basis, this can be done for example with the following function.

In [44]:
def Seifert_slopes_to_geometric_slopes(knot,S3_filling,Seifert_slope):
    '''
    Takes as input a Seifert slope and returns the slope measured with respect to the SnapPy basis.
    '''
    (a,b)=S3_filling
    K=snappy.Manifold(knot)
    (c,d)=K.homological_longitude()
    (p,q)=Seifert_slope
    return (p*a+q*c,p*b+q*d)